## What this notebook does
This notebook demonstrates a reproducible **data-processing pipeline** for a study on **time-of-day (ToD) effects in working memory (WM)**. It ingests multi-source data, performs cleaning and harmonization, and outputs analysis-ready tables for downstream statistical modeling.

This script produces an excel file, containing multiple sheets/databases. These databases/sheets contain the following information:

**1. Screening database**: includes information that was used to evaluate whether participants met the study’s inclusion and exclusion criteria.

**2. Sleep Diary database**: includes information that was used to evaluate whether participants complied with the study’s mandatory sleep schedule.

**3. Acitivity Diary database**: includes information that was used to evaluate whether participants complied with the study’s mandatory restrictions.

**4. Actigraphy database**: 

**5. WM tasks Practice database**:

**6. WM tasks Experimental database**:

# 0 - Imports librarys

In [ ]:
import pandas as pd
import os, datetime
from pathlib import Path
from datetime import timedelta, datetime
import time, re
import warnings

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 1. Screening database

The dataset produced includes information that was used to evaluate whether participants met the study’s inclusion and exclusion criteria. 

It contains:

- **Sociodemographic data.**  
  *e.g., age, nationality, educational background*
- **Morningness–Eveningness Questionnaire (MEQ)** scores.  
- **Brief Symptom Inventory (BSI)** scores.

---

This section:

- Loads the **raw screening data** (Excel file + MEQ item files).  
- Removes redundant variables.  
- Standardizes column names.  
- Performs extensive **data cleaning and recoding**.  
- Converts numeric categorical codes into **meaningful labels**.  
  *(e.g., sex, education level, caffeine/alcohol consumption)*.  
- Computes and integrates:  
  - **MEQ chronotype classification.**  
  - **BSI symptom dimensions.**  
  - **Global indice**.

## 1.1. Importing data

In [ ]:
#Opens the excel file where data from participants are stored.
# Go up one folder (from /notebook to project root)
PROJ_ROOT = Path("..").resolve()

# Build the path safely
excel_part_data_path = PROJ_ROOT / "Screening_Data" / "data.xlsx"

# Read the Excel file
df_excel_Screening = pd.read_excel(excel_part_data_path)
##Drops redundant columns (empty columns regarding Q1,Q7,Q14,Q15) in the dataframe df_excel_Screening.
df_excel_Screening.drop(df_excel_Screening.columns[[33, *range(104,108)]], axis=1, inplace=True)
df_excel_Screening.columns = [
    'subject_nr', 'Idade', 'Sexo', 'Nacionalidade', 'FreqEnsSup',
    'CicloEstudosEnsSup_1','CicloEstudosEnsSup_2','CicloEstudosEnsSup_3','CicloEstudosEnsSup_4',
    'HabAcademicas_1','HabAcademicas_2','HabAcademicas_3','HabAcademicas_4','HabAcademicas_5','HabAcademicas_6',
    'Curso','SituacaoLaboral','SituacaoLaboralTurnos_1','SituacaoLaboralTurnos_2',
    'PresencaDoencasAnter','ListaDoencasAnter','MedicacaoPsicotropica','ListaMedicacaoPsicotropica',
    'Fumador','NrCigarros','ProdutosSessaoTabagica_1','ProdutosSessaoTabagica_2',
    'ConsomeCafeina','QuantidadeDiaCafeina','ConsomeAlcool','QuantidadeDiaAlcool','ConsomeDrogas','Viagens',
    'Pergunta1','Pergunta2','Pergunta3','Pergunta4','Pergunta5','Pergunta6','Pergunta7','Pergunta8',
    'Pergunta9','Pergunta10','Pergunta11','Pergunta12','Pergunta13','Pergunta14','Pergunta15','Pergunta16',
    'MEQscore',
    *[f'BSI:{i}' for i in range(1,54)],
    'TIME_start','TIME_end','TIME_total'
]

# Sort participants by completion time (earliest to latest)
df_excel_Screening.sort_values(by='TIME_end', kind='mergesort', ascending=True, inplace=True)

# Reset the index after sorting
df_excel_Screening.reset_index(drop=True, inplace=True)

# Display all columns when printing the DataFrame
pd.set_option("display.max_columns", None)

# Show the cleaned screening dataset
df_excel_Screening

## 1.2. Transformation of raw codes into meaningful attributes

In [ ]:
df_excel_Screening[['Pergunta1', 'Pergunta7','Pergunta14','Pergunta15','Sexo','Nacionalidade','FreqEnsSup','CicloEstudosEnsSup_1','CicloEstudosEnsSup_2','CicloEstudosEnsSup_3','CicloEstudosEnsSup_4','HabAcademicas_1','HabAcademicas_2','HabAcademicas_3','HabAcademicas_4','HabAcademicas_5','HabAcademicas_6','SituacaoLaboral','SituacaoLaboralTurnos_1','SituacaoLaboralTurnos_2','PresencaDoencasAnter','MedicacaoPsicotropica','Fumador','ProdutosSessaoTabagica_1','ProdutosSessaoTabagica_2','ConsomeCafeina','ConsomeAlcool','ConsomeDrogas','Viagens']]

In [ ]:
# Resolve project root and point to MEQ item files directory
PROJ_ROOT = Path("..").resolve()
meq_dir = PROJ_ROOT / "Screening_Data" / "experiment_data"

# Initialize mapping: filename -> integer response value
file_to_value = {}
# Read each MEQ *.txt file, extract first token as int, and store in mapping (skip unreadable files)
for fp in sorted(meq_dir.glob("*.txt")):
    try:
        # Plain-text read is most robust for these tiny files
        token = fp.read_text(encoding="utf-8", errors="ignore").strip().split()[0]
        file_to_value[fp.name] = int(token)
    except Exception as e:
        print(f"Skipping {fp.name}: {e}")

# Replace filenames in MEQ question columns with their numeric answers (fallback to original if missing)
for col in ['Pergunta1','Pergunta7','Pergunta14','Pergunta15']:
    # Map filenames in the column to their numeric value; keep original if not found
    df_excel_Screening[col] = df_excel_Screening[col].map(file_to_value).fillna(df_excel_Screening[col])

# Recode sex codes to labels (1=Masculino, 2=Feminino, else=Outro)
for i in range(0,len(df_excel_Screening['Sexo'])):
        if df_excel_Screening.loc[i,'Sexo'] == 1:
            df_excel_Screening.loc[i,'Sexo'] = "Masculino"
        elif df_excel_Screening.loc[i,'Sexo'] == 2:
            df_excel_Screening.loc[i,'Sexo'] = "Feminino"
        else:
            df_excel_Screening.loc[i,'Sexo'] = "Outro"

# Recode nationality (1=PT, else=Outra)
for i in range(0,len(df_excel_Screening['Nacionalidade'])):
        if df_excel_Screening.loc[i,'Nacionalidade'] == 1:
            df_excel_Screening.loc[i,'Nacionalidade'] = "PT"
        else:
            df_excel_Screening.loc[i,'Nacionalidade'] = "Outra"

# Recode higher-education attendance (1=Sim, 0/other=Não)
for i in range(0,len(df_excel_Screening['FreqEnsSup'])):
        if df_excel_Screening.loc[i,'FreqEnsSup'] == 1:
            df_excel_Screening.loc[i,'FreqEnsSup'] = "Sim"
        else :
            df_excel_Screening.loc[i,'FreqEnsSup'] = "Não"

# Collapse one-hot study-cycle columns into a single 'CicloEstudosEnsSup' label
CicloEstudosEnsSup = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
        indexx = 0
        if df_excel_Screening.loc[i,'CicloEstudosEnsSup_1'] == 1:
            aaa = "Licenciatura"
            CicloEstudosEnsSup.append(aaa)
            indexx += 1
        elif df_excel_Screening.loc[i,'CicloEstudosEnsSup_2'] == 1:
            aaa = "Mestrado"
            CicloEstudosEnsSup.append(aaa)
            indexx += 1
        elif df_excel_Screening.loc[i,'CicloEstudosEnsSup_3'] == 1:
            aaa = "Doutoramento"
            CicloEstudosEnsSup.append(aaa)
            indexx += 1
        elif df_excel_Screening.loc[i,'CicloEstudosEnsSup_4'] == 1:
            aaa = "Outro Ciclo"
            CicloEstudosEnsSup.append(aaa)
            indexx +=1
        elif indexx == 0:
            aaa = ""
            CicloEstudosEnsSup.append(aaa)

# Drop study-cycle one-hot columns
df_excel_Screening.drop(columns=['CicloEstudosEnsSup_1','CicloEstudosEnsSup_2','CicloEstudosEnsSup_3','CicloEstudosEnsSup_4'],inplace=True)
# Attach consolidated study-cycle label
df_excel_Screening['CicloEstudosEnsSup'] = CicloEstudosEnsSup
# Move study-cycle column to position 5
TransCol = df_excel_Screening.pop('CicloEstudosEnsSup')
df_excel_Screening.insert(5,"CicloEstudosEnsSup",TransCol)

# Derive highest academic qualification based on attendance + cycle indicators
HabAcademicas = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
        if df_excel_Screening.loc[i,"FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_1'] == 1:
            aaa = "Ensino Obrigatório"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_2'] == 1:
            aaa = "Licenciatura"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_3'] == 1:
            aaa = "Pós-graduação"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_4'] == 1:
            aaa = "Mestrado"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_5'] == 1:
            aaa = "Doutoramento"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Não" and df_excel_Screening.loc[i,'HabAcademicas_6'] == 1:
            aaa = "Nenhuma das opções anteriores se aplica"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Sim" and df_excel_Screening.loc[i,'CicloEstudosEnsSup'] == "Licenciatura":
            aaa = "Ensino Obrigatório"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Sim" and df_excel_Screening.loc[i,'CicloEstudosEnsSup'] == "Mestrado":
            aaa = "Licenciatura"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Sim" and df_excel_Screening.loc[i,'CicloEstudosEnsSup'] == "Doutoramento":
            aaa = "Mestrado"
            HabAcademicas.append(aaa)
        elif df_excel_Screening.loc[i, "FreqEnsSup"] == "Sim" and df_excel_Screening.loc[i,'CicloEstudosEnsSup'] == "Outro Ciclo":
            aaa = "Outro Ciclo"
            HabAcademicas.append(aaa)

# Drop academic one-hot columns
df_excel_Screening.drop(columns=['HabAcademicas_1','HabAcademicas_2','HabAcademicas_3','HabAcademicas_4','HabAcademicas_5','HabAcademicas_6'],inplace=True)
# Attach derived academic label
df_excel_Screening['HabAcademicas'] = HabAcademicas
# Move academic column to position 6
TransCol = df_excel_Screening.pop('HabAcademicas')
df_excel_Screening.insert(6,"HabAcademicas",TransCol)

# Recode employment status (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['SituacaoLaboral'])):
        if df_excel_Screening.loc[i,'SituacaoLaboral'] == 1:
            df_excel_Screening.loc[i,'SituacaoLaboral'] = "Sim"
        else:
            df_excel_Screening.loc[i,'SituacaoLaboral'] = "Não"

# Derive shift-work indicator from two binary columns
SituacaoLaboralTurnos = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
        if df_excel_Screening.loc[i,'SituacaoLaboralTurnos_1'] == 1:
            aaa = "Sim"
            SituacaoLaboralTurnos.append(aaa)
        elif df_excel_Screening.loc[i, 'SituacaoLaboralTurnos_1'] == 0:
            aaa = "Não"
            SituacaoLaboralTurnos.append(aaa)
        elif df_excel_Screening.loc[i,'SituacaoLaboralTurnos_2'] == 1 or df_excel_Screening.loc[i,'SituacaoLaboralTurnos_2'] == 0:
            aaa = "Não"
            SituacaoLaboralTurnos.append(aaa)

# Drop shift-work source columns
df_excel_Screening.drop(columns=['SituacaoLaboralTurnos_1','SituacaoLaboralTurnos_2'],inplace=True)
# Attach derived shift-work label and place at column 9
df_excel_Screening['SituacaoLaboralTurnos'] = SituacaoLaboralTurnos
TransCol = df_excel_Screening.pop('SituacaoLaboralTurnos')
df_excel_Screening.insert(9,"SituacaoLaboralTurnos",TransCol)

# Recode prior disease presence (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['PresencaDoencasAnter'])):
        if df_excel_Screening.loc[i,'PresencaDoencasAnter'] == 1:
            df_excel_Screening.loc[i,'PresencaDoencasAnter'] = "Sim"
        else:
            df_excel_Screening.loc[i,'PresencaDoencasAnter'] = "Não"

# Recode psychotropic medication use (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['MedicacaoPsicotropica'])):
        if df_excel_Screening.loc[i,'MedicacaoPsicotropica'] == 1:
            df_excel_Screening.loc[i,'MedicacaoPsicotropica'] = "Sim"
        else:
            df_excel_Screening.loc[i,'MedicacaoPsicotropica'] = "Não"

# Recode smoking status into descriptive labels
for i in range(0,len(df_excel_Screening['Fumador'])):
        if df_excel_Screening.loc[i,'Fumador'] == 1:
            df_excel_Screening.loc[i,'Fumador'] = "Sou fumador"
        elif df_excel_Screening.loc[i,'Fumador'] == 2:
            df_excel_Screening.loc[i,'Fumador'] = "Deixei de fumar há menos de 3 meses"
        else:
            df_excel_Screening.loc[i,'Fumador'] = "Não sou fumador nem deixei de fumar há menos de 3 meses"

# Merge two tabacco-session flags into a single labeled column
ProdutosSessaoTabagica = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
        if df_excel_Screening.loc[i,'ProdutosSessaoTabagica_1'] == 1:
            aaa = "Sim"
            ProdutosSessaoTabagica.append(aaa)
        elif df_excel_Screening.loc[i,'ProdutosSessaoTabagica_2'] == 1:
            aaa = "Não"
            ProdutosSessaoTabagica.append(aaa)
        elif df_excel_Screening.loc[i,'ProdutosSessaoTabagica_1'] == 0 and df_excel_Screening.loc[i,'ProdutosSessaoTabagica_2'] == 0:
            aaa = None
            ProdutosSessaoTabagica.append(aaa)

# Drop tobacco-session source columns
df_excel_Screening.drop(columns=['ProdutosSessaoTabagica_1','ProdutosSessaoTabagica_2'],inplace=True)

# Attach consolidated tobacco-session label and place at column 16
df_excel_Screening['ProdutosSessaoTabagica'] = ProdutosSessaoTabagica
TransCol = df_excel_Screening.pop('ProdutosSessaoTabagica')
df_excel_Screening.insert(16,"ProdutosSessaoTabagica",TransCol)

# Recode caffeine consumption (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['ConsomeCafeina'])):
        if df_excel_Screening.loc[i,'ConsomeCafeina'] == 1:
            df_excel_Screening.loc[i,'ConsomeCafeina'] = "Sim"
        else:
            df_excel_Screening.loc[i,'ConsomeCafeina'] = "Não"

# Recode alcohol consumption (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['ConsomeAlcool'])):
        if df_excel_Screening.loc[i,'ConsomeAlcool'] == 1:
            df_excel_Screening.loc[i,'ConsomeAlcool'] = "Sim"
        else:
            df_excel_Screening.loc[i,'ConsomeAlcool'] = "Não"

# Recode drug use (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['ConsomeDrogas'])):
        if df_excel_Screening.loc[i,'ConsomeDrogas'] == 1:
            df_excel_Screening.loc[i,'ConsomeDrogas'] = "Sim"
        else:
            df_excel_Screening.loc[i,'ConsomeDrogas'] = "Não"

# Recode recent travel (1=Sim, else=Não)
for i in range(0,len(df_excel_Screening['Viagens'])):
        if df_excel_Screening.loc[i,'Viagens'] == 1:
            df_excel_Screening.loc[i,'Viagens'] = "Sim"
        else:
            df_excel_Screening.loc[i,'Viagens'] = "Não"

# Final QC preview: key recoded and derived screening fields
df_excel_Screening[['Pergunta1', 'Pergunta7','Pergunta14','Pergunta15','Sexo','Nacionalidade','FreqEnsSup','CicloEstudosEnsSup',
                    'HabAcademicas','SituacaoLaboral','SituacaoLaboralTurnos','PresencaDoencasAnter','MedicacaoPsicotropica',
                    'Fumador','ProdutosSessaoTabagica','ConsomeCafeina','ConsomeAlcool','ConsomeDrogas','Viagens']]

## 1.3. Calculating MEQ score

In [ ]:
# Compute chronotype category for each participant based on MEQ score
colChronotype = []
for i in range(0,len(df_excel_Screening['MEQscore'])):
    if df_excel_Screening.loc[i,'MEQscore'] < 31:
        temp_hold_chrono = 'Definitivamente Vespertino'
    elif df_excel_Screening.loc[i,'MEQscore'] >=  31 and df_excel_Screening.loc[i,'MEQscore'] <= 42:
        temp_hold_chrono = 'Moderadamente Vespertino'
    elif df_excel_Screening.loc[i,'MEQscore'] >=  43 and df_excel_Screening.loc[i,'MEQscore'] <= 53:
        temp_hold_chrono = 'Intermédio'
    elif df_excel_Screening.loc[i,'MEQscore'] >=  54 and df_excel_Screening.loc[i,'MEQscore'] <= 59:
        temp_hold_chrono = 'Moderadamente Matutino'
    elif df_excel_Screening.loc[i,'MEQscore'] > 59:
        temp_hold_chrono = 'Definitivamente Matutino'
    colChronotype.append(temp_hold_chrono)

# Add chronotype column to the dataframe
df_excel_Screening['Cronotipo'] = colChronotype

# Move chronotype column to position 40
TransCol = df_excel_Screening.pop('Cronotipo')
df_excel_Screening.insert(40,"Cronotipo",TransCol)

# Display chronotype column
df_excel_Screening[['Cronotipo']]

## 1.4. Calculating symptom dimensions and global indeces for the BSI

In [ ]:
#############Somatizacao
# Compute Somatization factor (mean of selected BSI items) per participant
FatorSomatizacao = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_somatizacao = (df_excel_Screening.loc[i,'BSI:2'] + df_excel_Screening.loc[i,'BSI:7'] + df_excel_Screening.loc[i,'BSI:23'] + df_excel_Screening.loc[i,'BSI:29'] + df_excel_Screening.loc[i,'BSI:30'] + df_excel_Screening.loc[i,'BSI:33'] + df_excel_Screening.loc[i,'BSI:37'])/7
    FatorSomatizacao.append(temp_somatizacao)

# Add Somatizacao column and position it at index 94
df_excel_Screening['Somatizacao'] = FatorSomatizacao
TransCol = df_excel_Screening.pop('Somatizacao')
df_excel_Screening.insert(94,"Somatizacao",TransCol)

##############FatorObsessoesCompulsoes
# Compute Obsessions–Compulsions factor (mean of selected BSI items)
FatorObsessoesCompulsoes = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorObsessoesCompulsoes = (df_excel_Screening.loc[i,'BSI:5'] + df_excel_Screening.loc[i,'BSI:15'] + df_excel_Screening.loc[i,'BSI:26'] + df_excel_Screening.loc[i,'BSI:27'] + df_excel_Screening.loc[i,'BSI:32'] + df_excel_Screening.loc[i,'BSI:36'])/6
    FatorObsessoesCompulsoes.append(temp_FatorObsessoesCompulsoes)

# Add factor column and place it at index 95
df_excel_Screening['FatorObsessoesCompulsoes'] = FatorObsessoesCompulsoes
TransCol = df_excel_Screening.pop('FatorObsessoesCompulsoes')
df_excel_Screening.insert(95,"FatorObsessoesCompulsoes",TransCol)

##############FatorSensibilidadeInterpessoal
# Compute Interpersonal Sensitivity factor (mean of selected BSI items)
FatorSensibilidadeInterpessoal = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorSensibilidadeInterpessoal = (df_excel_Screening.loc[i,'BSI:20'] + df_excel_Screening.loc[i,'BSI:21'] + df_excel_Screening.loc[i,'BSI:22'] + df_excel_Screening.loc[i,'BSI:42'])/4
    FatorSensibilidadeInterpessoal.append(temp_FatorSensibilidadeInterpessoal)

# Add factor column and place it at index 96
df_excel_Screening['FatorSensibilidadeInterpessoal'] = FatorSensibilidadeInterpessoal
TransCol = df_excel_Screening.pop('FatorSensibilidadeInterpessoal')
df_excel_Screening.insert(96,"FatorSensibilidadeInterpessoal",TransCol)

##############FatorDepressao
# Compute Depression factor (mean of selected BSI items)
FatorDepressao = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorDepressao = (df_excel_Screening.loc[i,'BSI:9'] + df_excel_Screening.loc[i,'BSI:16'] + df_excel_Screening.loc[i,'BSI:17'] + df_excel_Screening.loc[i,'BSI:18'] + df_excel_Screening.loc[i,'BSI:35'] + df_excel_Screening.loc[i,'BSI:50'])/6
    FatorDepressao.append(temp_FatorDepressao)

# Add factor column and place it at index 97
df_excel_Screening['FatorDepressao'] = FatorDepressao
TransCol = df_excel_Screening.pop('FatorDepressao')
df_excel_Screening.insert(97,"FatorDepressao",TransCol)

##############FatorAnsiedade
# Compute Anxiety factor (mean of selected BSI items)
FatorAnsiedade = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorAnsiedade = (df_excel_Screening.loc[i,'BSI:1'] + df_excel_Screening.loc[i,'BSI:12'] + df_excel_Screening.loc[i,'BSI:19'] + df_excel_Screening.loc[i,'BSI:38'] + df_excel_Screening.loc[i,'BSI:45'] + df_excel_Screening.loc[i,'BSI:49'])/6
    FatorAnsiedade.append(temp_FatorAnsiedade)

# Add factor column and place it at index 98
df_excel_Screening['FatorAnsiedade'] = FatorAnsiedade
TransCol = df_excel_Screening.pop('FatorAnsiedade')
df_excel_Screening.insert(98,"FatorAnsiedade",TransCol)

##############FatorHostilidade
# Compute Hostility factor (mean of selected BSI items)
FatorHostilidade = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorHostilidade = (df_excel_Screening.loc[i,'BSI:6'] + df_excel_Screening.loc[i,'BSI:13'] + df_excel_Screening.loc[i,'BSI:40'] + df_excel_Screening.loc[i,'BSI:41'] + df_excel_Screening.loc[i,'BSI:46'])/5
    FatorHostilidade.append(temp_FatorHostilidade)

# Add factor column and place it at index 99
df_excel_Screening['FatorHostilidade'] = FatorHostilidade
TransCol = df_excel_Screening.pop('FatorHostilidade')
df_excel_Screening.insert(99,"FatorHostilidade",TransCol)

##############FatorAnsiedadeFobica
# Compute Phobic Anxiety factor (mean of selected BSI items)
FatorAnsiedadeFobica = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorAnsiedadeFobica = (df_excel_Screening.loc[i,'BSI:8'] + df_excel_Screening.loc[i,'BSI:28'] + df_excel_Screening.loc[i,'BSI:31'] + df_excel_Screening.loc[i,'BSI:43'] + df_excel_Screening.loc[i,'BSI:47'])/5
    FatorAnsiedadeFobica.append(temp_FatorAnsiedadeFobica)

# Add factor column and place it at index 100
df_excel_Screening['FatorAnsiedadeFobica'] = FatorAnsiedadeFobica
TransCol = df_excel_Screening.pop('FatorAnsiedadeFobica')
df_excel_Screening.insert(100,"FatorAnsiedadeFobica",TransCol)

##############FatorIdeacaoParanoide
# Compute Paranoid Ideation factor (mean of selected BSI items)
FatorIdeacaoParanoide = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorIdeacaoParanoide = (df_excel_Screening.loc[i,'BSI:4'] + df_excel_Screening.loc[i,'BSI:10'] + df_excel_Screening.loc[i,'BSI:24'] + df_excel_Screening.loc[i,'BSI:48'] + df_excel_Screening.loc[i,'BSI:51'])/5
    FatorIdeacaoParanoide.append(temp_FatorIdeacaoParanoide)

# Add factor column and place it at index 101
df_excel_Screening['FatorIdeacaoParanoide'] = FatorIdeacaoParanoide
TransCol = df_excel_Screening.pop('FatorIdeacaoParanoide')
df_excel_Screening.insert(101,"FatorIdeacaoParanoide",TransCol)

##############FatorPsicoticismo
# Compute Psychoticism factor (mean of selected BSI items)
FatorPsicoticismo = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_FatorPsicoticismo = (df_excel_Screening.loc[i,'BSI:3'] + df_excel_Screening.loc[i,'BSI:14'] + df_excel_Screening.loc[i,'BSI:34'] + df_excel_Screening.loc[i,'BSI:44'] + df_excel_Screening.loc[i,'BSI:53'])/5
    FatorPsicoticismo.append(temp_FatorPsicoticismo)

# Add factor column and place it at index 102
df_excel_Screening['FatorPsicoticismo'] = FatorPsicoticismo
TransCol = df_excel_Screening.pop('FatorPsicoticismo')
df_excel_Screening.insert(102,"FatorPsicoticismo",TransCol)

##############IGS
# Compute Global Severity Index (IGS) as the mean of all 53 BSI items
IGS = []
for i in range(0,len(df_excel_Screening['subject_nr'])):
    temp_IGS = (df_excel_Screening.loc[i,'BSI:1'] +	df_excel_Screening.loc[i,'BSI:2'] +	df_excel_Screening.loc[i,'BSI:3'] +	df_excel_Screening.loc[i,'BSI:4'] +	df_excel_Screening.loc[i,'BSI:5'] +	df_excel_Screening.loc[i,'BSI:6'] +	df_excel_Screening.loc[i,'BSI:7'] +	df_excel_Screening.loc[i,'BSI:8'] +	df_excel_Screening.loc[i,'BSI:9'] +	df_excel_Screening.loc[i,'BSI:10'] + df_excel_Screening.loc[i,'BSI:11'] + df_excel_Screening.loc[i,'BSI:12'] + df_excel_Screening.loc[i,'BSI:13'] +	df_excel_Screening.loc[i,'BSI:14'] +	df_excel_Screening.loc[i,'BSI:15'] +	df_excel_Screening.loc[i,'BSI:16'] +	df_excel_Screening.loc[i,'BSI:17'] +	df_excel_Screening.loc[i,'BSI:18'] +	df_excel_Screening.loc[i,'BSI:19'] +	df_excel_Screening.loc[i,'BSI:20'] +	df_excel_Screening.loc[i,'BSI:21'] +	df_excel_Screening.loc[i,'BSI:22'] +	df_excel_Screening.loc[i,'BSI:23'] +	df_excel_Screening.loc[i,'BSI:24'] +	df_excel_Screening.loc[i,'BSI:25'] +	df_excel_Screening.loc[i,'BSI:26'] +	df_excel_Screening.loc[i,'BSI:27'] +	df_excel_Screening.loc[i,'BSI:28'] +	df_excel_Screening.loc[i,'BSI:29'] +	df_excel_Screening.loc[i,'BSI:30'] +	df_excel_Screening.loc[i,'BSI:31'] +	df_excel_Screening.loc[i,'BSI:32'] +	df_excel_Screening.loc[i,'BSI:33'] +	df_excel_Screening.loc[i,'BSI:34'] +	df_excel_Screening.loc[i,'BSI:35'] +	df_excel_Screening.loc[i,'BSI:36'] +	df_excel_Screening.loc[i,'BSI:37'] +	df_excel_Screening.loc[i,'BSI:38'] +	df_excel_Screening.loc[i,'BSI:39'] +	df_excel_Screening.loc[i,'BSI:40'] +	df_excel_Screening.loc[i,'BSI:41'] +	df_excel_Screening.loc[i,'BSI:42'] +	df_excel_Screening.loc[i,'BSI:43'] +	df_excel_Screening.loc[i,'BSI:44'] +	df_excel_Screening.loc[i,'BSI:45'] +	df_excel_Screening.loc[i,'BSI:46'] +	df_excel_Screening.loc[i,'BSI:47'] +	df_excel_Screening.loc[i,'BSI:48'] +	df_excel_Screening.loc[i,'BSI:49'] +	df_excel_Screening.loc[i,'BSI:50'] + df_excel_Screening.loc[i,'BSI:51'] + df_excel_Screening.loc[i,'BSI:52'] + df_excel_Screening.loc[i,'BSI:53'])/53
    IGS.append(temp_IGS)

# Add IGS column and place it at index 103
df_excel_Screening['IGS'] = IGS
TransCol = df_excel_Screening.pop('IGS')
df_excel_Screening.insert(103,"IGS",TransCol)

# Round selected BSI factor scores and IGS to two decimals
cols_to_round = [
    "Somatizacao",
    "FatorObsessoesCompulsoes",
    "FatorSensibilidadeInterpessoal",
    "FatorDepressao",
    "FatorAnsiedade",
    "FatorHostilidade",
    "FatorAnsiedadeFobica",
    "FatorIdeacaoParanoide",
    "FatorPsicoticismo",
    "IGS"
]
df_excel_Screening[cols_to_round] = df_excel_Screening[cols_to_round].round(2)

# Preview selected factor scores and IGS
df_excel_Screening[['Somatizacao','FatorObsessoesCompulsoes','FatorSensibilidadeInterpessoal','FatorDepressao',
                    'FatorAnsiedade','FatorHostilidade','FatorAnsiedadeFobica','FatorIdeacaoParanoide','FatorPsicoticismo',
                    'IGS']]

# Sort participants by subject number for consistent ordering
df_excel_Screening.sort_values(by=['subject_nr'], kind='mergesort', inplace=True,ascending=True)


## 1.5. View final screening database

In [ ]:
# Display all dataframe columns without truncation
pd.set_option("display.max_columns", None)

# Show the full dataframe
df_excel_Screening

# 2. Activity and sleep diaries databases
The following sections load, clean, transform, and preprocess the data collected through the participants’ sleep and activity diaries. These steps ensure that all variables are standardized, structured, and ready for subsequent analysis.

## **2.1. Sleep diary database**

In this subsection, we load, clean, and preprocess the data collected through the sleep diaries. These daily logs were used to verify compliance with the study’s mandatory sleep schedule: sleeping between 23:00h and 01:00h and waking between 07:00h and 09:00h.

---

### **Sleep Diary Questions**

The sleep diary contained the following questions: 

SD_Q1. A que hora foi para a cama?

SD_Q2. A que horas começou a tentar adormecer

SD_Q3. Quanto tempo acha que demorou a adormecer?

SD_Q4. Quantas vezes acordou e voltou a adormecer?

SD_Q5. Caso tenha acordado a meio da noite e voltado a adormecer quanto tempo é que esteve acordado até voltar a adormecer? Se experienciou mais do que um episódio destes some o tempo total que esteve acordado durante estes episódios. Por favor, indique esta estimativa em minutos (p. ex., 75).

SD_Q6a. A que horas acordou? Se acordou a meio da noite e voltou a adormecer indique a hora em que acordou pela última vez antes de se levantar. Por favor, utilize o formato hh:mm (p. ex., 08:30).

SD_Q6b. Depois de acordar quanto tempo passou na cama a tentar adormecer novamente? Por favor, indique esta estimativa em minutos (p. ex., 75).

SD_Q6c. Acordou antes da hora que tinha planificado?

SD_Q6d. Caso esta situação tenha ocorrido, indique quanto tempo antes da hora planificada acordou? Por favor, indique esta estimativa em minutos (p. ex., 75)Se isto não lhe aconteceu avance para a próxima pergunta.

SD_Q7. A que horas se levantou/saiu da cama? Se acordou a meio da noite, levantou-se, voltou para a cama e voltou a adormecer, indique qual foi a hora em que se levantou/saiu da cama pela última vez antes de iniciar a sua rotina. Por favor, utilize o formato hh:mm (p. ex., 08:30).

SD_Q8. No total, estima que dormiu quanto tempo (p. ex., 6h30)?

SD_Q9. Como classifica a qualidade do seu sono?

SD_Q10. O quão descansado(a)/restabelecido(a) se sentiu quando acordou e iniciou a sua rotina?

SD_Q11. Comentários. Adicione qualquer informação adicional que considere que possa ter condicionado o seu sono (p. ex., estar doente, ter experienciado alguma emergência). Caso não queira adicionar nenhuma informação deixe a caixa de texto em branco.

---

### **Derived Sleep-Efficiency Indices**

The script derives a set of sleep-efficiency indices from the raw outputs, which are standard in sleep research and allow quantitative evaluation of sleep continuity and restoration:

- **SOL (Sleep Onset Latency):** Time elapsed between going to bed and falling asleep.

- **TASAFA (Time Attempting to Sleep After Final Awakening):** Minutes spent trying to return to sleep after the final awakening.

- **WASO (Wake After Sleep Onset):** Total time awake during the sleep period after initially falling asleep.

- **TST (Total Sleep Time):** Total number of minutes actually spent sleeping during the sleep episode.

- **DSE (Duration of the Sleep Episode):** Total length of the sleep period, including SOL, WASO, TST, and TASAFA.

- **SE (Sleep Efficiency):** Ratio between total sleep time (TST) and the full sleep episode duration (DSE), expressed as a percentage.

### 2.1.1. Importing Data

In [ ]:
# Resolve project root directory
PROJ_ROOT = Path("..").resolve()

# Build full path to the sleep diary Excel file
SD_excel_part_data_path = PROJ_ROOT / "DiariosSonoAtividade" / "dataSono.xlsx"

# Load Excel sleep diary data into a dataframe
SD_df_excel_data_part = pd.read_excel(SD_excel_part_data_path)

## Drop redundant (empty) session-related columns
SD_df_excel_data_part.drop(SD_df_excel_data_part.columns[3:11],axis=1,inplace=True)

## Replace original column names with correct labels
SD_df_excel_data_part.columns = ['participant', 'Subject_Nr', 'Session_Nr', 'Data', 'SD_Q1', 'SD_Q2', 'SD_Q3', 'SD_Q4', 'SD_Q5', 'SD_Q6a',
                               'SD_Q6b', 'SD_Q6c', 'SD_Q6d', 'SD_Q7', 'SD_Q8', 'SD_Q9', 'SD_Q10', 'SD_Q11', 'TIME_start', 'TIME_end','TIME_total']

## Sort diary entries by participant and session number
SD_df_excel_data_part.sort_values(by=['Subject_Nr','Session_Nr'], kind='mergesort', inplace=True,ascending=True)
SD_df_excel_data_part.reset_index(drop=True,inplace=True)

# Display all dataframe columns and show the result
pd.set_option("display.max_columns", None)
SD_df_excel_data_part

### 2.1.2. Transformation of raw codes into meaningful attributes

In [ ]:
SD_df_excel_data_part[['SD_Q6c', 'SD_Q9','SD_Q10']]

In [ ]:
## Substitui valores numéricos por labels correspondentes nas colunas selecionadas
for i in range(0,len(SD_df_excel_data_part['SD_Q6c'])):
        if SD_df_excel_data_part.loc[i,'SD_Q6c'] == 1:
            SD_df_excel_data_part.loc[i,'SD_Q6c'] = "Sim"
        else :
            SD_df_excel_data_part.loc[i,'SD_Q6c'] = "Não"

# Map numeric sleep quality ratings (SD_Q9) to descriptive labels
for i in range(0,len(SD_df_excel_data_part['SD_Q9'])):
        if SD_df_excel_data_part.loc[i,'SD_Q9'] == 1:
            SD_df_excel_data_part.loc[i,'SD_Q9'] = "Muito Pobre"
        elif SD_df_excel_data_part.loc[i,'SD_Q9'] == 2:
            SD_df_excel_data_part.loc[i,'SD_Q9'] = "Pobre"
        elif SD_df_excel_data_part.loc[i,'SD_Q9'] == 3:
            SD_df_excel_data_part.loc[i,'SD_Q9'] = "Aceitável"
        elif SD_df_excel_data_part.loc[i,'SD_Q9'] == 4:
            SD_df_excel_data_part.loc[i,'SD_Q9'] = "Boa"
        else :
            SD_df_excel_data_part.loc[i,'SD_Q9'] = "Muito Boa"

# Convert numeric restfulness ratings (SD_Q10) to descriptive labels
for i in range(0,len(SD_df_excel_data_part['SD_Q10'])):
        if SD_df_excel_data_part.loc[i,'SD_Q10'] == 1:
            SD_df_excel_data_part.loc[i,'SD_Q10'] = "Nada descansado(a)/restabelecido(a)"
        elif SD_df_excel_data_part.loc[i,'SD_Q10'] == 2:
            SD_df_excel_data_part.loc[i,'SD_Q10'] = "Ligeiramente descansado(a)/restabelecido(a)"
        elif SD_df_excel_data_part.loc[i,'SD_Q10'] == 3:
            SD_df_excel_data_part.loc[i,'SD_Q10'] = "Aceitavelmente descansado(a)/restabelecido(a)"
        elif SD_df_excel_data_part.loc[i,'SD_Q10'] == 4:
            SD_df_excel_data_part.loc[i,'SD_Q10'] = "Bem descansado(a)/restabelecido(a)"
        else :
            SD_df_excel_data_part.loc[i,'SD_Q10'] = "Muito bem descansado(a)/restabelecido(a)"

# Display updated categorical columns
SD_df_excel_data_part[['SD_Q6c', 'SD_Q9','SD_Q10']]

### 2.1.3. Calculates and appends sleep efficiency indices (e.g., Total Sleep Time (TST), Duration of Sleep Episode (DSE), and Sleep Efficiency (SE))

In [ ]:
# Import standard libraries used for time calculations and timestamps
import os, datetime, time

# Create SOL (sleep onset latency) column from SD_Q3
SD_df_excel_data_part['SOL'] = SD_df_excel_data_part['SD_Q3']

# Create TASAFA (time attempting to sleep after final awakening) column from SD_Q6b
SD_df_excel_data_part['TASAFA'] = SD_df_excel_data_part['SD_Q6b']

# Create WASO (wake after sleep onset) column from SD_Q5
SD_df_excel_data_part['WASO'] = SD_df_excel_data_part['SD_Q5']

# Replace missing WASO values with 0 (participants who skipped the question)
for i in range(0,len(SD_df_excel_data_part['WASO'])):
    if pd.isnull(SD_df_excel_data_part.loc[i,'WASO']):
        SD_df_excel_data_part.loc[i,'WASO'] = 0

# Prepare to compute Total Sleep Time (TST) from sleep attempt to rise time
temp_sleep_time = SD_df_excel_data_part['SD_Q2'].copy(deep=True)

# Parse bedtime strings to datetime
for i in range(0,len(temp_sleep_time)):
    temp_sleep_time[i] = pd.to_datetime(str(temp_sleep_time[i]))

# Copy rise times for aligned processing
temp_rising_time = SD_df_excel_data_part['SD_Q6a'].copy(deep=True)

# Parse rise time strings to datetime
for i in range(0,len(temp_rising_time)):
    temp_rising_time[i] = pd.to_datetime(str(temp_rising_time[i]))

# Compute elapsed time between rise time and bedtime
temp_diff_ris_sleep = temp_rising_time - temp_sleep_time

# Keep only the HH:MM portion (drop date component)
for i in range(0,len(temp_diff_ris_sleep)):
    temp_diff_ris_sleep[i] = str(temp_diff_ris_sleep[i])
    aaa = (str(temp_diff_ris_sleep[i]))
    aaa = aaa[-8:-3]
    temp_diff_ris_sleep[i] = aaa

# Compute TST in minutes: (Wakeup - Bedtime) - WASO - SOL, and also format HHhMM for display
temp_TST = []
TST_for_DSE = []
for i in range(0,len(temp_diff_ris_sleep)):
    ##Converte o formato hh:mm em minutos e subtrai o valor do WASo e do SOL para calcular o TST.
    ##Volta a converter o TST num forato HHhMM
    aaa = float(temp_diff_ris_sleep[i][0:2])
    aaa = aaa*60
    bbb = float(temp_diff_ris_sleep[i][3:6])
    aaa = aaa + bbb
    aaa = aaa - SD_df_excel_data_part.loc[i,'WASO'] - SD_df_excel_data_part.loc[i,'SOL']
    TST_for_DSE.append(aaa)
    aaa = aaa/60
    aaa = '{0:02.0f}:{1:02.0f}'.format(*divmod(aaa * 60, 60))
    aaa = str(aaa)
    aaa = aaa.replace(":","h")
    temp_TST.append(aaa)

# Store formatted TST (HHhMM)
SD_df_excel_data_part['TST'] = temp_TST

# Compute DSE (duration of sleep episode) = SOL + WASO + TST + TASAFA; store formatted HHhMM
# The DSE is calculating by suming the SOL, WASO, TST, and the TASAFA.
temp_DSE = []
DSE_for_SE = []
for i in range(0,len(SD_df_excel_data_part['participant'])):
    aaa = TST_for_DSE[i] + SD_df_excel_data_part.loc[i,'WASO'] + SD_df_excel_data_part.loc[i,'SOL'] + SD_df_excel_data_part.loc[i,'TASAFA']
    DSE_for_SE.append(aaa)
    aaa = aaa/60
    aaa = '{0:02.0f}:{1:02.0f}'.format(*divmod(aaa * 60, 60))
    aaa = str(aaa)
    aaa = aaa.replace(":","h")
    temp_DSE.append(aaa)

# Store formatted DSE (HHhMM)
SD_df_excel_data_part['DSE'] = temp_DSE

# Compute SE (sleep efficiency) as TST/DSE * 100
# The SE is the ratio of total sleep time (TST) to duration of sleep episode (SDE).
temp_SE = []
for i in range(0,len(SD_df_excel_data_part['participant'])):
    aaa = round((TST_for_DSE[i]/DSE_for_SE[i])*100,2)
    temp_SE.append(aaa)

# Store SE (%)
SD_df_excel_data_part['SE'] = temp_SE

# Define helper constants (hours and time thresholds)
twentythree = 82800
twentythree = datetime.time(23,0,0)
one = 3600
one = datetime.time(1,0,0)
seven = 25200
seven = datetime.time(7,0,0)
nine = datetime.time(9,0,0)
TSTsix = 360
TSTnine = 540

# Show all columns and preview key derived sleep variables
pd.set_option("display.max_columns", None)
SD_df_excel_data_part[['SOL', 'TASAFA', 'WASO', 'TST', 'DSE', 'SE']]

### 2.1.4. View final sleep diary database

In [ ]:
# Reorder columns to produce the final structured sleep diary dataset
SD_df_excel_data_part = SD_df_excel_data_part[['Subject_Nr', 'Session_Nr', 'Data', 'SD_Q1', 'SD_Q2',
       'SD_Q3', 'SD_Q4', 'SD_Q5', 'SD_Q6a', 'SD_Q6b', 'SD_Q6c', 'SD_Q6d',
       'SD_Q7', 'SD_Q8', 'SD_Q9', 'SD_Q10', 'SD_Q11', 'SOL', 'TASAFA', 'WASO', 'TST', 'DSE', 'SE', 'TIME_start', 'TIME_end',
       'TIME_total']]

# Display all columns and show the final dataframe
pd.set_option("display.max_columns", None)
SD_df_excel_data_part

## **2.1. Acitivity diary database**

In this subsection, we load, clean, and preprocess the data collected through the activity diaries. These daily logs were used to verify compliance with the study’s mandatory restrictions: participants were asked to limit their daily intake of caffeinated beverages (≤3) and alcoholic drinks (≤2) and to refrain from using any sleep aids, including herbal supplements like valerian.

---

### **Activity diary questions**

The activity diary contained the following questions: 

AD_Q1.1 Quantas sestas fez hoje (p. ex., 1)?

AD_Q1.2 Caso tenha feito sesta(s), quanto tempo dormiu no total (p. ex., 02h15)? Se fez mais do que uma sesta, some o tempo total que dormiu em todas as sestas.Caso não tenha feito sesta avance para a questão seguinte.

AD_Q3. Quantas bebidas alcoólicas consumiu hoje (p. ex., 0)?

AD_Q4. Sinalize o(s) horário(s) em que consumiu bebidas alcoólicas.<br>Caso não tenha consumido bebidas alcoólicas avance para a questão seguinte.

AD_Q5. Quantas bebidas cafeinadas (café, bebidas energéticas, chá) consumiu hoje (p. ex., 2)?

AD_Q6. Sinalize o(s) horário(s) em que consumiu bebidas cafeinadas.<br>Caso não tenha consumido bebidas cafeinadas avance para a questão seguinte.

AD_Q7. Tomou alguma medicação para o(a) ajudar a dormir?

AD_Q8. Caso tenha respondido afirmativamente na última questão liste a medicação que tomou.<br>Se não tomou nenhuma medicação avance para a questão seguinte.

AD_Q9. Sinalize o(s) horário(s) em que tomou medicação para dormir.<br>Caso não tenha tomado nenhuma medicação avance para a questão seguinte.

AD_Q10. Realizou algum tipo de desporto ou atividade física intensa?

AD_Q11. Sinalize o(s) horário(s) em que realizou desporto ou atividade física intensa.<br>Caso não tenha realizado desporto ou atividade física intensa avance para a questão seguinte.

AD_Q12. Comentários. Adicione qualquer informação adicional que considere que possa ter condicionado/alterado a sua rotina (p. ex., estar doente, ter experienciado alguma emergência). Caso não queira adicionar mais nenhuma informação deixe a caixa de texto em branco.

### 2.2.1. Importing data

In [ ]:
# Resolve the project root directory (one level above the notebook)
PROJ_ROOT = Path("..").resolve()

# Build the full path to the activity diary Excel file
AD_excel_part_data_path = PROJ_ROOT / "DiariosSonoAtividade" / "dataAtividade.xlsx"

# Load the raw activity diary dataset
AD_df_excel_data_part = pd.read_excel(AD_excel_part_data_path)

# Drop redundant/empty columns related to unused session fields
AD_df_excel_data_part.drop(AD_df_excel_data_part.columns[3:11], axis=1, inplace=True)

# Replace default column names with the correct questionnaire labels
AD_df_excel_data_part.columns = ["participant","Subject_Nr","Session_Nr","Data","AD_Q1","AD_Q2","AD_Q3","AD_Q4:1","AD_Q4:2","AD_Q4:3",
                                 "AD_Q4:4","AD_Q4:5","AD_Q4:6","AD_Q4:7","AD_Q4:8","AD_Q4:9","AD_Q4:10","AD_Q4:11","AD_Q4:12","AD_Q4:13",
                                 "AD_Q4:14","AD_Q4:15","AD_Q4:16","AD_Q4:17","AD_Q4:18","AD_Q4:19","AD_Q4:20","AD_Q4:21","AD_Q4:22",
                                 "AD_Q4:23","AD_Q4:24","AD_Q5","AD_Q6:1","AD_Q6:2","AD_Q6:3","AD_Q6:4","AD_Q6:5","AD_Q6:6","AD_Q6:7",
                                 "AD_Q6:8","AD_Q6:9","AD_Q6:10","AD_Q6:11","AD_Q6:12","AD_Q6:13","AD_Q6:14","AD_Q6:15","AD_Q6:16",
                                 "AD_Q6:17","AD_Q6:18","AD_Q6:19","AD_Q6:20","AD_Q6:21","AD_Q6:22","AD_Q6:23","AD_Q6:24","AD_Q7",
                                 "AD_Q8","AD_Q9:1","AD_Q9:2","AD_Q9:3","AD_Q9:4","AD_Q9:5","AD_Q9:6","AD_Q9:7","AD_Q9:8","AD_Q9:9",
                                 "AD_Q9:10","AD_Q9:11","AD_Q9:12","AD_Q9:13","AD_Q9:14","AD_Q9:15","AD_Q9:16","AD_Q9:17","AD_Q9:18",
                                 "AD_Q9:19","AD_Q9:20","AD_Q9:21","AD_Q9:22","AD_Q9:23","AD_Q9:24","AD_Q10","AD_Q11:1","AD_Q11:2",
                                 "AD_Q11:3","AD_Q11:4","AD_Q11:5","AD_Q11:6","AD_Q11:7","AD_Q11:8","AD_Q11:9","AD_Q11:10","AD_Q11:11",
                                 "AD_Q11:12","AD_Q11:13","AD_Q11:14","AD_Q11:15","AD_Q11:16","AD_Q11:17","AD_Q11:18","AD_Q11:19",
                                 "AD_Q11:20","AD_Q11:21","AD_Q11:22","AD_Q11:23","AD_Q11:24","AD_Q12","TIME_start","TIME_end",
                                 "TIME_total"]

# Sort records by participant and session number in ascending order
AD_df_excel_data_part.sort_values(by=["Subject_Nr", "Session_Nr"], kind='mergesort', inplace=True, ascending=True)
AD_df_excel_data_part.reset_index(drop=True, inplace=True)

# Ensure full column display and show the processed dataset
pd.set_option("display.max_columns", None)
AD_df_excel_data_part

### 2.2.2. Transformation of raw codes into meaningful attributes

In [ ]:
##Substitui os valores númericos que codificam horas nas colunas referentes às questões 4,6,9 11 pelos labels corretos (horas
#correspondenes, p. ex., 04h00)
for i in range(0,len(AD_df_excel_data_part['AD_Q4:1'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:1'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:1'] = '00h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:2'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:2'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:2'] = '01h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:3'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:3'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:3'] = '02h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:4'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:4'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:4'] = '03h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:5'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:5'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:5'] = '04h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:6'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:6'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:6'] = '05h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:7'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:7'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:7'] = '06h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:8'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:8'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:8'] = '07h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:9'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:9'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:9'] = '08h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:10'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:10'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:10'] = '09h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:11'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:11'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:11'] = '10h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:12'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:12'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:12'] = '11h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:13'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:13'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:13'] = '12h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:14'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:14'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:14'] = '13h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:15'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:15'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:15'] = '14h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:16'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:16'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:16'] = '15h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:17'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:17'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:17'] = '16h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:18'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:18'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:18'] = '17h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:19'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:19'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:19'] = '18h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:20'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:20'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:20'] = '19h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:21'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:21'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:21'] = '20h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:22'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:22'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:22'] = '21h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:23'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:23'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:23'] = '22h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q4:24'])):
    if AD_df_excel_data_part.loc[i,'AD_Q4:24'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q4:24'] = '23h00'

for i in range(0,len(AD_df_excel_data_part['AD_Q6:1'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:1'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:1'] = '00h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:2'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:2'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:2'] = '01h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:3'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:3'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:3'] = '02h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:4'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:4'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:4'] = '03h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:5'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:5'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:5'] = '04h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:6'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:6'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:6'] = '05h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:7'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:7'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:7'] = '06h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:8'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:8'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:8'] = '07h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:9'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:9'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:9'] = '08h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:10'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:10'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:10'] = '09h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:11'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:11'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:11'] = '10h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:12'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:12'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:12'] = '11h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:13'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:13'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:13'] = '12h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:14'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:14'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:14'] = '13h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:15'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:15'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:15'] = '14h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:16'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:16'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:16'] = '15h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:17'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:17'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:17'] = '16h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:18'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:18'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:18'] = '17h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:19'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:19'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:19'] = '18h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:20'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:20'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:20'] = '19h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:21'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:21'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:21'] = '20h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:22'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:22'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:22'] = '21h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:23'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:23'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:23'] = '22h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q6:24'])):
    if AD_df_excel_data_part.loc[i,'AD_Q6:24'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q6:24'] = '23h00'


for i in range(0,len(AD_df_excel_data_part['AD_Q9:1'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:1'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:1'] = '00h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:2'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:2'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:2'] = '01h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:3'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:3'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:3'] = '02h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:4'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:4'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:4'] = '03h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:5'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:5'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:5'] = '04h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:6'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:6'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:6'] = '05h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:7'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:7'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:7'] = '06h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:8'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:8'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:8'] = '07h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:9'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:9'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:9'] = '08h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:10'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:10'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:10'] = '09h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:11'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:11'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:11'] = '10h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:12'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:12'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:12'] = '11h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:13'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:13'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:13'] = '12h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:14'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:14'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:14'] = '13h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:15'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:15'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:15'] = '14h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:16'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:16'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:16'] = '15h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:17'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:17'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:17'] = '16h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:18'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:18'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:18'] = '17h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:19'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:19'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:19'] = '18h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:20'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:20'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:20'] = '19h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:21'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:21'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:21'] = '20h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:22'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:22'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:22'] = '21h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:23'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:23'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:23'] = '22h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q9:24'])):
    if AD_df_excel_data_part.loc[i,'AD_Q9:24'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q9:24'] = '23h00'


for i in range(0,len(AD_df_excel_data_part['AD_Q11:1'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:1'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:1'] = '00h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:2'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:2'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:2'] = '01h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:3'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:3'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:3'] = '02h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:4'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:4'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:4'] = '03h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:5'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:5'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:5'] = '04h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:6'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:6'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:6'] = '05h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:7'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:7'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:7'] = '06h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:8'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:8'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:8'] = '07h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:9'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:9'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:9'] = '08h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:10'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:10'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:10'] = '09h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:11'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:11'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:11'] = '10h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:12'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:12'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:12'] = '11h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:13'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:13'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:13'] = '12h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:14'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:14'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:14'] = '13h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:15'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:15'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:15'] = '14h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:16'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:16'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:16'] = '15h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:17'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:17'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:17'] = '16h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:18'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:18'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:18'] = '17h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:19'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:19'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:19'] = '18h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:20'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:20'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:20'] = '19h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:21'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:21'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:21'] = '20h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:22'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:22'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:22'] = '21h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:23'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:23'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:23'] = '22h00'
for i in range(0,len(AD_df_excel_data_part['AD_Q11:24'])):
    if AD_df_excel_data_part.loc[i,'AD_Q11:24'] == 1:
        AD_df_excel_data_part.loc[i, 'AD_Q11:24'] = '23h00'


for i in range(0,len(AD_df_excel_data_part['AD_Q7'])):
        if AD_df_excel_data_part.loc[i,'AD_Q7'] == 1:
            AD_df_excel_data_part.loc[i,'AD_Q7'] = "Sim"
        else:
            AD_df_excel_data_part.loc[i, 'AD_Q7'] = "Não"

for i in range(0,len(AD_df_excel_data_part['AD_Q10'])):
        if AD_df_excel_data_part.loc[i,'AD_Q10'] == 1:
            AD_df_excel_data_part.loc[i,'AD_Q10'] = "Sim"
        else:
            AD_df_excel_data_part.loc[i, 'AD_Q10'] = "Não"

##Cria listas com os horários em que os participantes consumiram bebidas alcoólicas (Q4), bebidas cafeinadas (Q6), tomaram medicação
##para dormir (Q9), ou fizeram exercício (Q11) e guarda essas listas em colunas na dataframe 'AD_df_excel_data_part'.
major_list = []
for j in range(0,len(AD_df_excel_data_part['participant'])):
    temp_list = []
    for i in range(7,31):
        if AD_df_excel_data_part.iloc[j,i] != 0:
            temp_list.append(AD_df_excel_data_part.iloc[j,i])
    major_list.append(temp_list)
for i in range(0,len(major_list)):
    if major_list[i] == []:
        major_list[i] = None
AD_df_excel_data_part["AD_Q4"] = major_list

major_list = []
for j in range(0,len(AD_df_excel_data_part['participant'])):
    temp_list = []
    for i in range(32,56):
        if AD_df_excel_data_part.iloc[j,i] != 0:
            temp_list.append(AD_df_excel_data_part.iloc[j,i])
    major_list.append(temp_list)
for i in range(0,len(major_list)):
    if major_list[i] == []:
        major_list[i] = None
AD_df_excel_data_part["AD_Q6"] = major_list

major_list = []
for j in range(0,len(AD_df_excel_data_part['participant'])):
    temp_list = []
    for i in range(58,82):
        if AD_df_excel_data_part.iloc[j,i] != 0:
            temp_list.append(AD_df_excel_data_part.iloc[j,i])
    major_list.append(temp_list)
for i in range(0,len(major_list)):
    if major_list[i] == []:
        major_list[i] = None
AD_df_excel_data_part["AD_Q9"] = major_list

major_list = []
for j in range(0,len(AD_df_excel_data_part['participant'])):
    temp_list = []
    for i in range(83,107):
        if AD_df_excel_data_part.iloc[j,i] != 0:
            temp_list.append(AD_df_excel_data_part.iloc[j,i])
    major_list.append(temp_list)
for i in range(0,len(major_list)):
    if major_list[i] == []:
        major_list[i] = None
AD_df_excel_data_part["AD_Q11"] = major_list

##Elimina colunas que já não são úteis relacionadas com as questões 4, 6, 9 e 11.
AD_df_excel_data_part.drop(AD_df_excel_data_part.columns[83:107],axis=1,inplace=True)
AD_df_excel_data_part.drop(AD_df_excel_data_part.columns[58:82],axis=1,inplace=True)
AD_df_excel_data_part.drop(AD_df_excel_data_part.columns[32:56],axis=1,inplace=True)
AD_df_excel_data_part.drop(AD_df_excel_data_part.columns[7:31],axis=1,inplace=True)


cols_to_clean = ["AD_Q4","AD_Q6", "AD_Q11"]  # columns where [''] appears
for col in cols_to_clean:
    AD_df_excel_data_part[col] = (
        AD_df_excel_data_part[col]
        .astype(str)
        .str.replace(r"[\[\]']", "", regex=True)  # remove [, ], '
        .str.strip()
    )
    
AD_df_excel_data_part = AD_df_excel_data_part[
    ["Subject_Nr", "Session_Nr","Data", "AD_Q1","AD_Q2","AD_Q3","AD_Q4", "AD_Q5","AD_Q6", "AD_Q7","AD_Q8","AD_Q9","AD_Q10", 
     "AD_Q11","AD_Q12","TIME_start", "TIME_end","TIME_total"]
]

pd.set_option("display.max_columns", None)
AD_df_excel_data_part

### 2.2.3. View final activity diary database

In [ ]:
# Reorder columns to the canonical activity-diary layout
AD_df_excel_data_part = AD_df_excel_data_part[['Subject_Nr', 'Session_Nr', 'Data', 'AD_Q1', 'AD_Q2', 'AD_Q3', 'AD_Q4',
                                               'AD_Q5', 'AD_Q6', 'AD_Q7', 'AD_Q8', 'AD_Q9', 'AD_Q10', 'AD_Q11', 'AD_Q12', 'TIME_start',
                                               'TIME_end', 'TIME_total']]

# Show all columns without truncation
pd.set_option("display.max_columns", None)

# Display the reordered dataframe
AD_df_excel_data_part

# 3. Actigraphy database
Generates DB with the actigraphy data collected.

## 3.1. Importing data

In [ ]:
# Resolve project root path (one directory up from /notebook)
PROJ_ROOT = Path("..").resolve()

# Build path to the day-summary CSV safely
csv_part_data_path = PROJ_ROOT / "Actigraphy" / "part5_daysummary_WW_L30M100V400_T5A5.csv"

# Load day-summary CSV into a DataFrame
df_Actigraphy = pd.read_csv(csv_part_data_path, engine = 'python',sep = ',')

# Create a view sorted by ID (no assignment; actual sort happens later)
df_Actigraphy.sort_values("ID")

# Keep relevant columns from GeneActive day summary
df_Actigraphy = df_Actigraphy[["ID","filename","sleeplog_used","guider","cleaningcode","daysleeper","night_number","calendar_date","weekday",
                                     "nonwear_perc_day_spt","sleeponset_ts","wakeup_ts","dur_day_spt_min","dur_day_min","dur_spt_min",
                                     "dur_spt_wake_IN_min","dur_spt_wake_LIG_min","dur_spt_wake_MOD_min","dur_spt_wake_VIG_min","dur_day_IN_unbt_min",
                                     "dur_day_LIG_unbt_min","dur_day_MOD_unbt_min","dur_day_VIG_unbt_min","dur_spt_sleep_min","sleep_efficiency"]]


######################################
# Resolve project root path again (explicitly)
PROJ_ROOT = Path("..").resolve()

# Build path to the night-summary CSV safely
csv_part_data_path2 = PROJ_ROOT / "Actigraphy" / "part4_nightsummary_sleep_cleaned.csv"

# Load night-summary CSV into a DataFrame
df_Actigraphy2 = pd.read_csv(csv_part_data_path2, engine = 'python',sep = ',')

# Create a view sorted by ID (no assignment)
df_Actigraphy2.sort_values("ID")

# Keep only first-night records
df_Actigraphy2 = df_Actigraphy2[df_Actigraphy2['night'] == 1]

# Preserve a deep copy of first-night data
df_Actigraphy3 = df_Actigraphy2.copy(deep=True)

# Select common metadata and timing columns
df_Actigraphy2 = df_Actigraphy2[["ID","filename","sleeplog_used","guider","cleaningcode","daysleeper","night","calendar_date","weekday",
                                     "nonwear_perc_spt","sleeponset_ts","wakeup_ts"]]

# Add missing day-summary columns (empty strings) to align schemas
df_Actigraphy2 = df_Actigraphy2.assign(dur_day_spt_min="",dur_day_min="",dur_spt_min="",dur_spt_wake_IN_min="",
                                             dur_spt_wake_LIG_min="",dur_spt_wake_MOD_min="",dur_spt_wake_VIG_min="",
                                             dur_day_IN_unbt_min="",dur_day_LIG_unbt_min="",dur_day_MOD_unbt_min="",
                                             dur_day_VIG_unbt_min="",dur_spt_sleep_min="",sleep_efficiency="")

# Harmonize column names with day-summary schema
df_Actigraphy2.rename(columns={'night':'night_number','nonwear_perc_spt':'nonwear_perc_day_spt'},inplace=True)

# Reorder columns to exactly match the day-summary layout
df_Actigraphy2 = df_Actigraphy2[["ID","filename","sleeplog_used","guider","cleaningcode","daysleeper","night_number","calendar_date","weekday",
                                     "nonwear_perc_day_spt","sleeponset_ts","wakeup_ts","dur_day_spt_min","dur_day_min","dur_spt_min",
                                     "dur_spt_wake_IN_min","dur_spt_wake_LIG_min","dur_spt_wake_MOD_min","dur_spt_wake_VIG_min","dur_day_IN_unbt_min",
                                     "dur_day_LIG_unbt_min","dur_day_MOD_unbt_min","dur_day_VIG_unbt_min","dur_spt_sleep_min","sleep_efficiency"]]

# Gather day and night DataFrames for concatenation
frames = [df_Actigraphy,df_Actigraphy2]

# Concatenate day and first-night records into one DataFrame
df_Actigraphy = pd.concat(frames)

# Sort by participant ID and night number
df_Actigraphy = df_Actigraphy.sort_values(by=['ID','night_number'],ascending=[True,True])

# Reset index after sorting
df_Actigraphy.reset_index(drop=True,inplace=True)

# Regex pattern to capture text between underscores (first match)
pattern2 = "_(.*?)_"

# Prepare container for device location parsed from filename
List_actigraph_location = []

# Extract device location from each filename using regex and collect
for i in range(len(df_Actigraphy["filename"])):
    substring = re.search(pattern2, df_Actigraphy["filename"][i]).group(1)
    List_actigraph_location.append(substring)

# Convert collected locations to a Series
LocationAcel = pd.Series(List_actigraph_location)

# Insert parsed location as the third column
df_Actigraphy.insert(2,"LocationAcel",LocationAcel)

# Show all columns when displaying the DataFrame
pd.set_option("display.max_columns", None)

# Display the final DataFrame
df_Actigraphy

## 3.2. Transformation of raw codes into meaningful attributes

In [ ]:
# Display all columns when printing DataFrames
pd.set_option("display.max_columns", None)

# Show selected metadata columns from the Actigraphy DataFrame
df_Actigraphy[["daysleeper","sleeplog_used","cleaningcode","calendar_date"]]

In [ ]:
# Map 0/1 in 'daysleeper' to human-readable labels across all rows
for i in range(0,len(df_Actigraphy["daysleeper"])):
    # If coded as 0, label as day sleeper
    if df_Actigraphy.loc[i,"daysleeper"] == 0:
        df_Actigraphy.loc[i,"daysleeper"] = "day sleeper"
    # If coded as 1, label as night sleeper
    elif df_Actigraphy.loc[i,"daysleeper"] == 1:
        df_Actigraphy.loc[i,"daysleeper"] = "night sleeper"

# Convert 0/1 in 'sleeplog_used' to "No"/"Yes" across all rows
for i in range(0,len(df_Actigraphy["sleeplog_used"])):
    # Replace 0 with "No"
    if df_Actigraphy.loc[i,"sleeplog_used"] == 0:
        df_Actigraphy.loc[i,"sleeplog_used"] = "No"
    # Replace 1 with "Yes"
    elif df_Actigraphy.loc[i,"sleeplog_used"] == 1:
        df_Actigraphy.loc[i,"sleeplog_used"] = "Yes"

# Expand 'cleaningcode' value 1 to its descriptive text across all rows
for i in range(0,len(df_Actigraphy["cleaningcode"])):
    # If cleaning code is 1, set a detailed explanation
    if df_Actigraphy.loc[i,"cleaningcode"] == 1:
        df_Actigraphy.loc[i,"cleaningcode"] = "1: GGIR sleep log was not used. Thus, HDCZA guider was used. Only Sleep Period Time (SPT) was identified " \
                                                  "(it was not possible to indentify Time in Bed (TIB))."

# Reformat 'calendar_date' to zero-padded YYYY-MM-DD for night 1 rows
for i in range(0,len(df_Actigraphy["calendar_date"])):
    # Only transform dates where night_number equals 1
    if df_Actigraphy.loc[i,"night_number"] == 1:
        # Split existing date by "/" into components
        x = df_Actigraphy.loc[i,"calendar_date"].split("/")
        # Prepare reversed list for Y-M-D ordering
        y = []
        # Reverse component order
        for j in x:
            y.insert(0,j)
        # Zero-pad single-digit components
        for j in range(0,len(y)):
            if int(y[j]) < 10:
                y[j] = '0' + y[j]
        # Convert list to string
        y = str(y)
        # Strip brackets
        y = y.replace("[","")
        y = y.replace("]","")
        # Remove quotes
        y = y.replace("'","")
        # Remove spaces
        y = y.replace(" ","")
        # Replace commas with hyphens
        y = y.replace(",","-")
        # Write back normalized date
        df_Actigraphy.loc[i,"calendar_date"] = y

# Build WASO list using night 1 from df_Actigraphy3, else sum wake mins within SPT
List_WASO = []
for i in range(0,len(df_Actigraphy)):
    # For night 1, convert WASO hours to minutes and round
    if df_Actigraphy.loc[i,"night_number"] == 1:
        List_WASO.append(round(df_Actigraphy3.loc[i,"WASO"]*60,3))
    # For other nights, sum wake durations in different intensity bands
    else:
        ind = df_Actigraphy.loc[i,"dur_spt_wake_IN_min"] + df_Actigraphy.loc[i,"dur_spt_wake_LIG_min"] + df_Actigraphy.loc[i,"dur_spt_wake_MOD_min"] + df_Actigraphy.loc[i,"dur_spt_wake_VIG_min"]
        List_WASO.append(ind)

# Get DataFrame shape to locate insertion position
row,col = df_Actigraphy.shape
# Convert accumulated WASO list to a Series
List_WASO = pd.Series(List_WASO)
# Insert WASO as a new column before the last existing column
df_Actigraphy.insert(col-1,"WASO",List_WASO)

# Ensure all columns display when printing
pd.set_option("display.max_columns", None)
# Preview selected columns including the new WASO
df_Actigraphy[["daysleeper","sleeplog_used","cleaningcode","calendar_date","WASO"]]

## 3.3. Standardizing time variables and calculating sleep-related indices

In [ ]:
df_Actigraphy[["dur_day_spt_min","dur_day_min","dur_spt_min","dur_spt_sleep_min","dur_spt_wake_IN_min",
                    "dur_spt_wake_LIG_min","dur_spt_wake_MOD_min","dur_spt_wake_VIG_min","dur_day_IN_unbt_min","dur_day_LIG_unbt_min",
                    "dur_day_MOD_unbt_min","dur_day_VIG_unbt_min","WASO"]]

In [ ]:
# Import the base datetime module
import datetime
# Import timedelta and datetime classes for time arithmetic and parsing
from datetime import timedelta, datetime

# Define target columns that will be converted to timedelta values
List_target_col = ["dur_day_spt_min","dur_day_min","dur_spt_min","dur_spt_sleep_min","dur_spt_wake_IN_min",
                    "dur_spt_wake_LIG_min","dur_spt_wake_MOD_min","dur_spt_wake_VIG_min","dur_day_IN_unbt_min","dur_day_LIG_unbt_min",
                    "dur_day_MOD_unbt_min","dur_day_VIG_unbt_min","WASO"]

# Loop over each target column to normalize values to timedeltas
for i in List_target_col:
    # Iterate over all rows in the actigraphy dataframe
    for l in range(0,len(df_Actigraphy)):
        # Process only rows that are not the first night
        if df_Actigraphy.loc[l,"night_number"] != 1:
            # If the value is zero, set it explicitly to a zero timedelta
            if df_Actigraphy.loc[l,i] == 0:
                df_Actigraphy.loc[l,i] = timedelta(hours=00, minutes=00, seconds=00)
            # Otherwise convert minutes to a timedelta
            else:
                df_Actigraphy.loc[l,i] = timedelta(minutes=df_Actigraphy.loc[l,i])

# Initialize storage for (wakeup - bedtime) durations for first-night rows
wakeupminusbedtime = []
# Iterate through rows to compute sleep duration and (wakeup - bedtime) for night 1
for i in range(0,len(df_Actigraphy)):
    # Operate only on first-night entries
    if df_Actigraphy.loc[i,"night_number"] == 1:
        # Parse sleep onset string to datetime, then to timedelta since midnight
        aaa = datetime.strptime(df_Actigraphy.loc[i,"sleeponset_ts"], "%H:%M:%S")
        aaa = timedelta(hours=aaa.hour, minutes=aaa.minute, seconds=aaa.second)
        # If sleep onset is between 21:00 and 23:59:59, compute remaining day span to midnight+1s
        if timedelta(hours=9,minutes=0,seconds=0) <= aaa <= timedelta(hours=23,minutes=59,seconds=59):
            bbb = timedelta(hours=23,minutes=59,seconds=59) - aaa + timedelta(hours=0,minutes=0,seconds=1)
        # Otherwise there is no cross-midnight portion
        else:
            bbb = timedelta(hours=00,minutes=0,seconds=0)
        # Parse wakeup time string and convert to timedelta since midnight
        ccc = datetime.strptime(df_Actigraphy.loc[i,"wakeup_ts"], "%H:%M:%S")
        ccc = timedelta(hours=ccc.hour, minutes=ccc.minute, seconds=ccc.second)
        # Convert WASO (in minutes) to a timedelta in seconds
        ddd = df_Actigraphy.loc[i,"WASO"]*60
        ddd = timedelta(seconds=ddd)
        # If onset is in late evening, include cross-midnight span when computing sleep duration
        if timedelta(hours=9,minutes=0,seconds=0) <= aaa <= timedelta(hours=23,minutes=59,seconds=59):
            df_Actigraphy.loc[i,"dur_spt_sleep_min"] = ccc + bbb - ddd
            wakeupminusbedtime.append(ccc + bbb)
        # Otherwise compute simple difference without cross-midnight adjustment
        else:
            df_Actigraphy.loc[i,"dur_spt_sleep_min"] = ccc - aaa - ddd
            wakeupminusbedtime.append(ccc - aaa)
        # Overwrite WASO column with timedelta version
        df_Actigraphy.loc[i,"WASO"] = ddd

# Make deep copies of the participant daily logs dataframe for processing
Daily_Logs_df_excel_data_part = SD_df_excel_data_part.copy(deep=True)
# Second copy for potential separate use
Daily_Logs_df_excel_data_part2 = SD_df_excel_data_part.copy(deep=True)

# Keep only the bedtime and risetime columns
Daily_Logs_df_excel_data_part = Daily_Logs_df_excel_data_part[["SD_Q2","SD_Q7"]]
# List of daily log time columns to convert to timedeltas
List_columns_Daily_Logs = ["SD_Q2","SD_Q7"]
# Convert each selected time field to a timedelta since midnight
for i in List_columns_Daily_Logs:
    # Iterate all rows to transform time objects into timedeltas
    for l in range(0,len(Daily_Logs_df_excel_data_part)):
        # Read time value and rebuild as timedelta for consistency
        t = Daily_Logs_df_excel_data_part.loc[l,i]
        Daily_Logs_df_excel_data_part.loc[l,i] = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
# Insert Bedtime timedelta into the actigraphy dataframe at column index 11
df_Actigraphy.insert(11,"Bedtime",Daily_Logs_df_excel_data_part["SD_Q2"])
# Insert Risetime timedelta into the actigraphy dataframe at column index 14
df_Actigraphy.insert(14,"Risetime",Daily_Logs_df_excel_data_part["SD_Q7"])

# Columns with HH:MM:SS strings to be converted into timedeltas
List_columns_sleeponset_wakeup = ["sleeponset_ts","wakeup_ts"]
# Convert sleeponset and wakeup time strings to timedeltas
for i in List_columns_sleeponset_wakeup:
    # Iterate over rows to parse and convert each time string
    for l in range(0,len(df_Actigraphy)):
        # Parse string to datetime and convert to timedelta since midnight
        t = datetime.strptime(df_Actigraphy.loc[l,i], "%H:%M:%S")
        df_Actigraphy.loc[l,i] = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)

# Define a zero timedelta for reuse
aaa = timedelta(hours=00, minutes=00, seconds=00)
# Define 07:00 threshold used to detect post-midnight times
bbb = timedelta(hours=7, minutes=00, seconds=00)
# Define 24:00 timedelta for day wrapping
ccc = timedelta(hours=24, minutes=00, seconds=00)

# Temporary holder for adjusted sleep onset per row
temp_sleep_onset = aaa
# Temporary holder for adjusted bedtime per row
temp_Bedtime = aaa
# Collector for Sleep Onset Latency (SOL) per row
List_SOL = []
# Ensure Bedtime is a timedelta, coerce invalids to NaT
df_Actigraphy["Bedtime"] = pd.to_timedelta(df_Actigraphy["Bedtime"], errors="coerce")
# Ensure Risetime is a timedelta, coerce invalids to NaT
df_Actigraphy["Risetime"] = pd.to_timedelta(df_Actigraphy["Risetime"], errors="coerce")
# Compute SOL per row, adjusting for cross-midnight where needed
for i in range(0,len(df_Actigraphy)):
    # If sleep onset is before 07:00, treat it as next-day by adding 24h
    if df_Actigraphy.loc[i,"sleeponset_ts"] < bbb:
        temp_sleep_onset = df_Actigraphy.loc[i,"sleeponset_ts"] + ccc
    # Otherwise keep sleep onset as-is
    else:
        temp_sleep_onset = df_Actigraphy.loc[i,"sleeponset_ts"]
    # If bedtime is before 07:00, treat it as next-day by adding 24h
    if df_Actigraphy.loc[i,"Bedtime"] < bbb:
        temp_Bedtime = df_Actigraphy.loc[i, "Bedtime"] + ccc
    # Otherwise keep bedtime as-is
    else:
        temp_Bedtime = df_Actigraphy.loc[i, "Bedtime"]
    # If adjusted sleep onset precedes adjusted bedtime, clamp onset to bedtime
    if temp_sleep_onset < temp_Bedtime:
        df_Actigraphy.loc[i, "sleeponset_ts"] = df_Actigraphy.loc[i, "Bedtime"]
    # Compute SOL as (adjusted sleep onset - adjusted bedtime)
    temp_SOL = temp_sleep_onset - temp_Bedtime
    # Do not allow negative SOL; floor at zero
    if temp_SOL < aaa:
        temp_SOL = aaa
    # Store computed SOL
    List_SOL.append(temp_SOL)
# Insert SOL column at provided index `col`
df_Actigraphy.insert(col,"SOL",List_SOL)

# If wakeup exceeds reported risetime, cap wakeup at risetime
for i in range(0,len(df_Actigraphy)):
    # Enforce wakeup <= risetime to avoid negative wake-after-wakeup
    if df_Actigraphy.loc[i,"wakeup_ts"] > df_Actigraphy.loc[i,"Risetime"]:
        df_Actigraphy.loc[i,"wakeup_ts"] = df_Actigraphy.loc[i,"Risetime"]

# Prepare list for Time Awake Since Awakening (TASAFA)
List_TASAFA = []
# Compute TASAFA as risetime minus wakeup, floored at zero
for i in range(0,len(df_Actigraphy)):
    # Raw TASAFA difference
    temp_TASAFA = df_Actigraphy.loc[i, "Risetime"] - df_Actigraphy.loc[i, "wakeup_ts"]
    # Floor negative values to zero
    if temp_TASAFA < aaa:
        temp_TASAFA = aaa
    # Store computed TASAFA
    List_TASAFA.append(temp_TASAFA)
# Insert TASAFA at `col+3`
df_Actigraphy.insert(col+3,"TASAFA",List_TASAFA)

# Prepare list for Daytime Sleep Episode (DSE) durations
List_DSE = []
# Compute DSE as SOL + sleep duration + WASO + TASAFA
for i in range(0,len(df_Actigraphy)):
    # Sum components to form DSE
    temp_DSE = df_Actigraphy.loc[i,"SOL"] + df_Actigraphy.loc[i,"dur_spt_sleep_min"] + df_Actigraphy.loc[i,"WASO"] + df_Actigraphy.loc[i,"TASAFA"]
    # Store computed DSE
    List_DSE.append(temp_DSE)
# Assign DSE column
df_Actigraphy["DSE"] = List_DSE

# Prepare list for sleep efficiency based on TST(Act)/DSE(Act)
List_SEF = []
# Compute per-row sleep efficiency and round to 3 decimals
for i in range(0,len(df_Actigraphy)):
    # Sleep efficiency as ratio of TST to DSE
    temp_SEF = round(df_Actigraphy.loc[i,"dur_spt_sleep_min"] / df_Actigraphy.loc[i,"DSE"],3)
    # Store computed efficiency
    List_SEF.append(temp_SEF)
# Assign computed sleep efficiency column
df_Actigraphy["Sleep Efficiency TST(Act)/DSE(Act)"] = List_SEF

# Desired column order for the actigraphy dataframe
columns_re = ["ID","filename","LocationAcel","sleeplog_used","guider","cleaningcode","daysleeper","night_number","calendar_date",
              "weekday","nonwear_perc_day_spt","Bedtime","sleeponset_ts","wakeup_ts","Risetime","SOL","dur_spt_sleep_min","WASO",
              "TASAFA","DSE","sleep_efficiency","Sleep Efficiency TST(Act)/DSE(Act)","dur_day_spt_min","dur_day_min","dur_spt_min",
              "dur_spt_wake_IN_min","dur_spt_wake_LIG_min","dur_spt_wake_MOD_min","dur_spt_wake_VIG_min","dur_day_IN_unbt_min",
              "dur_day_LIG_unbt_min","dur_day_MOD_unbt_min","dur_day_VIG_unbt_min"]

# Reindex dataframe to the desired column order
df_Actigraphy = df_Actigraphy.reindex(columns=columns_re)
# Human-readable column names for final output
col_names = ["ID","filename","LocationAcel","sleeplog_used","guider","cleaningcode","daysleeper","night_number","calendar_date",
              "weekday","nonwear_perc","Bedtime","Sleep Onset","Wakeup Time","Rise Time","SOL","TST","WASO",
              "TASAFA","DSE","Sleep Efficiency GGIR (TST/(Wakeup-Sleep Onset))","Sleep Efficiency TST(Act)/DSE(Act)",
             "Duration Day+Sleep episodes","Duration Day Episode","Duration Sleep Episode (Wakeup-Sleep Onset)",
             "Duration Inactive in Sleep Episode","Duration Light Activity in Sleep Episode",
             "Duration Moderate Activity in Sleep Episode","Duration Vigorous Activity in Sleep Episode",
             "Duration Inactive in Day Episode","Duration Light Activity in Day Episode",
             "Duration Moderate Activity in Day Episode","Duration Vigorous Activity in Day Episode"]

# Apply readable column names to the dataframe
df_Actigraphy.columns = col_names

# Index pointer for iterating through first-night wakeup-minus-bedtime values
indexx = 0
# Compute GGIR sleep efficiency and sleep episode durations row-wise
for i in range(0,len(df_Actigraphy)):
    # For first-night rows, use precomputed (wakeup - bedtime) duration
    if df_Actigraphy.loc[i,"night_number"] == 1:
        df_Actigraphy.loc[i,"Sleep Efficiency GGIR (TST/(Wakeup-Sleep Onset))"] = round(df_Actigraphy.loc[i,"TST"]/wakeupminusbedtime[indexx],3)
        df_Actigraphy.loc[i,"Duration Sleep Episode (Wakeup-Sleep Onset)"] = wakeupminusbedtime[indexx]
        indexx += 1
    # For other nights, handle potential cross-midnight onset
    else:
        # If sleep onset is late evening, wrap to next day for total duration
        if df_Actigraphy.loc[i,"Sleep Onset"] > bbb:
            temp_sleep_onset = ccc - df_Actigraphy.loc[i,"Sleep Onset"]
            df_Actigraphy.loc[i,"Sleep Efficiency GGIR (TST/(Wakeup-Sleep Onset))"] = round(df_Actigraphy.loc[i,"TST"]/(df_Actigraphy.loc[i,"Wakeup Time"] + temp_sleep_onset),3)
        # Otherwise compute duration within the same day
        else:
            df_Actigraphy.loc[i, "Sleep Efficiency GGIR (TST/(Wakeup-Sleep Onset))"] = round(df_Actigraphy.loc[i, "TST"]/(df_Actigraphy.loc[i, "Wakeup Time"] - df_Actigraphy.loc[i,"Sleep Onset"]), 3)

In [ ]:
# Allow full column display in pandas output
pd.set_option("display.max_columns", None)

# Display selected duration-related actigraphy columns including WASO
df_Actigraphy[["Duration Day+Sleep episodes","Duration Day Episode","Duration Sleep Episode (Wakeup-Sleep Onset)",
             "Duration Inactive in Sleep Episode","Duration Light Activity in Sleep Episode",
             "Duration Moderate Activity in Sleep Episode","Duration Vigorous Activity in Sleep Episode",
             "Duration Inactive in Day Episode","Duration Light Activity in Day Episode",
             "Duration Moderate Activity in Day Episode","Duration Vigorous Activity in Day Episode","WASO"]]

## 3.4. Round time and sleep efficiency metrics

In [ ]:
# List of columns containing time-like values that require formatting
time_cols = [
    "Bedtime",
    "Sleep Onset",
    "Wakeup Time",
    "Rise Time",
    "SOL",
    "TST",
    "WASO",
    "TASAFA",
    "DSE",
    "Duration Day+Sleep episodes",
    "Duration Day Episode",
    "Duration Sleep Episode (Wakeup-Sleep Onset)",
    "Duration Inactive in Sleep Episode",
    "Duration Light Activity in Sleep Episode",
    "Duration Moderate Activity in Sleep Episode",
    "Duration Vigorous Activity in Sleep Episode",
    "Duration Inactive in Day Episode",
    "Duration Light Activity in Day Episode",
    "Duration Moderate Activity in Day Episode",
    "Duration Vigorous Activity in Day Episode"
]

# Convert selected columns to timedelta format, coercing invalid values
df_Actigraphy[time_cols] = df_Actigraphy[time_cols].apply(pd.to_timedelta, errors="coerce")

# Loop through each time column to convert timedelta values into HH:MM:SS strings
for col in time_cols:
    df_Actigraphy[col] = df_Actigraphy[col].apply(
        lambda x: (
            f"{int(x.total_seconds() // 3600):02d}:"
            f"{int((x.total_seconds() % 3600) // 60):02d}:"
            f"{int(x.total_seconds() % 60):02d}"
        ) if pd.notna(x) else ""
    )

# List of numeric columns that must be rounded to 2 decimal places
cols_to_round = [
    "nonwear_perc",
    "Sleep Efficiency GGIR (TST/(Wakeup-Sleep Onset))",
    "Sleep Efficiency TST(Act)/DSE(Act)"
]

# Round numeric percentage and efficiency columns
df_Actigraphy[cols_to_round] = df_Actigraphy[cols_to_round].round(2)

# Allow unlimited column width when printing the dataframe
pd.set_option("display.max_columns", None)

# Display selected duration-related columns including WASO
df_Actigraphy[["Duration Day+Sleep episodes","Duration Day Episode","Duration Sleep Episode (Wakeup-Sleep Onset)",
             "Duration Inactive in Sleep Episode","Duration Light Activity in Sleep Episode",
             "Duration Moderate Activity in Sleep Episode","Duration Vigorous Activity in Sleep Episode",
             "Duration Inactive in Day Episode","Duration Light Activity in Day Episode",
             "Duration Moderate Activity in Day Episode","Duration Vigorous Activity in Day Episode","WASO"]]

## 3.5. View actigraphy database

In [ ]:
# Enable full display of all dataframe columns
pd.set_option("display.max_columns", None)

# Display the entire df_Actigraphy dataframe
df_Actigraphy

# 4. Working memory (WM) tasks practice session database
Generates DB with the performance data in the WM tasks collected in the practice sessions.

## 4.1. Importing data

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Import file path utilities
from pathlib import Path
# Import pandas for data handling
import pandas as pd

# Set project root directory one level above current script
PROJ_ROOT = Path("..").resolve()
# Define directory containing participant CSV files
data_dir = PROJ_ROOT / "data_participants_practice"

# Collect and sort all CSV files in the target directory
csv_files = sorted(data_dir.glob("*.csv"))

# Attempt to read a CSV file using multiple common encoding and delimiter configurations
def try_read(p):
    # List of parameter combinations to try for reading CSV files across OS/Excel formats
    tries = [
        dict(sep=None, engine="python", encoding="utf-8-sig"),
        dict(sep=";",  engine="python", encoding="utf-8-sig"),
        dict(sep=",",  engine="python", encoding="utf-8-sig"),

        dict(sep=None, engine="python", encoding="latin-1"),
        dict(sep=";",  engine="python", encoding="latin-1"),
        dict(sep=",",  engine="python", encoding="latin-1"),

        dict(sep="\t", engine="python", encoding="utf-16"),
        dict(sep=",",  engine="python", encoding="utf-16"),
        dict(sep=";",  engine="python", encoding="utf-16"),
    ]

    # Store most recent error to raise if all attempts fail
    last_err = None
    # Try reading the file with each configuration
    for kw in tries:
        try:
            return pd.read_csv(str(p), error_bad_lines=False, warn_bad_lines=False, **kw)
        except TypeError:
            # Retry using only supported kwargs if pandas version is old
            try:
                return pd.read_csv(str(p), **{k:v for k,v in kw.items() if k in ("sep","engine","encoding")})
            except Exception as e:
                last_err = e
        except Exception as e:
            last_err = e
    # Raise final error after exhausting all attempts
    raise last_err

# List to accumulate successfully read dataframes
dfs = []
# Iterate through all CSV file paths and read them into dataframes
for p in csv_files:
    df = try_read(p)
    dfs.append(df)

# Concatenate all dataframes into a single combined dataframe
df_total_part = pd.concat(dfs, ignore_index=True)

# Reset index to ensure proper sequential indexing
df_total_part.reset_index(inplace=True)

# Replace occurrences of the string 'undefined' with missing values 'Nan'
df_total_part = df_total_part.replace('undefined','Nan')

# Move Task_Name column to first position and subject_nr to second position
first_column = df_total_part.pop('Task_Name')
second_column = df_total_part.pop('subject_nr')
df_total_part.insert(0, 'Task_Name', first_column)
df_total_part.insert(1, 'subject_nr', second_column)

# Display all columns when printing dataframe
pd.set_option("display.max_columns", None)

# Display final combined participants dataframe
df_total_part

## 4.2. Extracts and cleans practice-session data for each WM task

In [ ]:
# Filter Reading Span practice trials
df_Reading_Span_Practice = df_total_part[df_total_part['Task_Name'] == 'Reading Span']
# Sort Reading Span rows by participant number
df_Reading_Span_Practice = df_Reading_Span_Practice.sort_values(by=['subject_nr'], kind='mergesort')

# Filter Working Memory Updating Task practice trials
df_WMU_Task_Practice = df_total_part[df_total_part['Task_Name'] == 'Working Memory Updating Task']
# Sort WMU rows by participant number
df_WMU_Task_Practice = df_WMU_Task_Practice.sort_values(by=['subject_nr'], kind='mergesort')

# Filter Symmetry Span practice trials
df_Symmetry_Span_Practice = df_total_part[df_total_part['Task_Name'] == 'Symmetry Span']
# Sort Symmetry Span rows by participant number
df_Symmetry_Span_Practice = df_Symmetry_Span_Practice.sort_values(by=['subject_nr'], kind='mergesort')

# Filter Binding Task practice trials
df_Binding_Task_Practice = df_total_part[df_total_part['Task_Name'] == 'Binding Task']
# Sort Binding Task rows by participant number
df_Binding_Task_Practice = df_Binding_Task_Practice.sort_values(by=['subject_nr'], kind='mergesort')

# Filter Operation Span practice trials
df_Operation_Span_Practice = df_total_part[df_total_part['Task_Name'] == 'Operation Span']
# Sort Operation Span rows by participant number
df_Operation_Span_Practice = df_Operation_Span_Practice.sort_values(by=['subject_nr'], kind='mergesort')

########################################################################################################
# Select relevant columns for Reading Span, convert formats, clean strings, and rename columns for clarity
df_Reading_Span_Practice = df_Reading_Span_Practice[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'Frase', 'height', 'letter', 'List_Prev_Letter',
     'List_responses_memory', 'live_row', 'logfile', 'response_average_time_memory', 'response_memory',
     'response_processing', 'response_time_memory', 'response_time_processing', 'response_total_time_memory',
     'RP_part_process_time', 'score_practice', 'score_reading_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4',
     'score_subblock_5', 'score_subblock_6', 'Tipo', 'total_correct',
     'total_response_time', 'total_responses', 'width']]
# Replace commas with dots before numeric conversion
df_Reading_Span_Practice[['acc', 'avg_rt']] = df_Reading_Span_Practice[['acc', 'avg_rt']].replace(',', '.')
# Convert selected text columns to appropriate numeric/string types
df_Reading_Span_Practice = df_Reading_Span_Practice.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
# Identify a recurring unwanted string in processing responses
example_b = df_Reading_Span_Practice["response_processing"].iloc[2]
# Remove the identified unwanted string from the dataframe
df_Reading_Span_Practice = df_Reading_Span_Practice.replace(example_b, '')
# Rename all columns to practice-session–specific labels
df_Reading_Span_Practice.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                             'SubTaskName','acc_Practice_Sess','avg_rt_Practice_Sess','BlockChoice_Practice_Sess','correct_Practice_Sess','correct_response_Practice_Sess',
                             'Frase_Practice_Sess','height_Practice_Sess','letter_Practice_Sess','List_Prev_Letter_Practice_Sess','List_responses_memory_Practice_Sess',
                             'live_row_Practice_Sess','logfile_Practice_Sess','response_average_time_memory_Practice_Sess','response_memory_Practice_Sess',
                             'response_processing_Practice_Sess','response_time_memory_Practice_Sess','response_time_processing_Practice_Sess',
                             'response_total_time_memory_Practice_Sess','RP_part_process_time_Practice_Sess','score_practice_Practice_Sess',
                             'score_reading_span_Practice_Sess','score_subblock_2_Practice_Sess','score_subblock_3_Practice_Sess','score_subblock_4_Practice_Sess',
                             'score_subblock_5_Practice_Sess','score_subblock_6_Practice_Sess','Tipo_Practice_Sess','total_correct_Practice_Sess',
                             'total_response_time_Practice_Sess','total_responses_Practice_Sess','width_Practice_Sess']

############################################################################################
############################################################################################
# Select WMU task columns, clean missing RTs, and rename columns for consistency
df_WMU_Task_Practice = df_WMU_Task_Practice[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'correct1', 'correct2', 'correct3', 'digit1', 'digit2', 'digit3', 'height',
     'Index_List', 'live_row', 'logfile', 'response1', 'response2', 'response3', 'response_time1', 'responseavgRT',
     'total_correct_trial', 'TotalRtBlock',  'WMUExperimentalScore', 'WMUPracticeScore', 'width']]
# Rename primary response time column
df_WMU_Task_Practice = df_WMU_Task_Practice.rename(columns={'response_time1': 'response_time'})

# Replace "0" average RT entries with empty strings
for i in range(0, len(df_WMU_Task_Practice['responseavgRT'])):
    if df_WMU_Task_Practice['responseavgRT'].iloc[i] == 0:
        df_WMU_Task_Practice['responseavgRT'].iloc[i] = ''
# Assign cleaned and standardized column names
df_WMU_Task_Practice.columns = ['subject_nr','CB_ref','practice','TrialNumber','correct1_Practice_Sess',
                         'correct2_Practice_Sess','correct3_Practice_Sess','digit1_Practice_Sess','digit2_Practice_Sess','digit3_Practice_Sess','height_Practice_Sess',
                         'Index_List_Practice_Sess','live_row_Practice_Sess','logfile_Practice_Sess','response1_Practice_Sess','response2_Practice_Sess','response3_Practice_Sess',
                         'response_time1_Practice_Sess','responseavgRT_Practice_Sess','total_correct_trial_Practice_Sess','TotalRtBlock_Practice_Sess',
                         'WMUExperimentalScore_Practice_Sess','WMUPracticeScore_Practice_Sess','width_Practice_Sess']

############################################################################################
# Select Symmetry Span columns, clean text fields, and rename columns
df_Symmetry_Span_Practice = df_Symmetry_Span_Practice[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName',
     'aggregated_score_memory', 'average_response_time_processing', 'average_total_time_memory', 'correct',
     'correct_response', 'countDys', 'countSym', 'height', 'LeftHalfPos', 'List_SS_button', 'List_SS_Pos', 'live_row',
     'logfile', 'maxDys', 'maxSym', 'pressed_buttons', 'response_memory', 'response_processing', 'response_time_memory',
     'response_time_processing', 'response_total_time_memory', 'response_total_time_memory_full_task', 'RightHalfPos',
     'SP_part_process_time', 'SS_practice_score', 'score_symmetry_span', 'score_subblock_2', 'score_subblock_3',
     'score_subblock_4', 'score_subblock_5', 'score_subblock_6', 'SymType',
     'total_correct_processing', 'total_response_time_processing', 'width']]
# Convert text-like response columns to string type
df_Symmetry_Span_Practice = df_Symmetry_Span_Practice.astype({'correct_response': 'str', 'response_processing': 'str'})
# Identify unwanted repeated processing string
example_d = df_Symmetry_Span_Practice["response_processing"].iloc[2]
# Remove occurrences of the unwanted string
df_Symmetry_Span_Practice = df_Symmetry_Span_Practice.replace(example_d, '')
# Rename columns to practice-session format
df_Symmetry_Span_Practice.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                              'SubTaskName','aggregated_score_memory_Practice_Sess','average_response_time_processing_Practice_Sess',
                              'average_total_time_memory_Practice_Sess','correct_Practice_Sess','correct_response_Practice_Sess','countDys_Practice_Sess',
                              'countSym_Practice_Sess','height_Practice_Sess','LeftHalfPos_Practice_Sess','List_SS_button_Practice_Sess','List_SS_Pos_Practice_Sess',
                              'live_row_Practice_Sess','logfile_Practice_Sess','maxDys_Practice_Sess','maxSym_Practice_Sess','pressed_buttons_Practice_Sess',
                              'response_memory_Practice_Sess','response_processing_Practice_Sess','response_time_memory_Practice_Sess',
                              'response_time_processing_Practice_Sess','response_total_time_memory_Practice_Sess',
                              'response_total_time_memory_full_task_Practice_Sess','RightHalfPos_Practice_Sess','SP_part_process_time_Practice_Sess',
                              'SS_practice_score_Practice_Sess','score_symmetry_span_Practice_Sess','score_subblock_2_Practice_Sess','score_subblock_3_Practice_Sess',
                              'score_subblock_4_Practice_Sess','score_subblock_5_Practice_Sess','score_subblock_6_Practice_Sess','SymType_Practice_Sess',
                              'total_correct_processing_Practice_Sess','total_response_time_processing_Practice_Sess','width_Practice_Sess']

############################################################################################
############################################################################################
# Select Binding Task columns, convert numeric formats, and rename for consistency
df_Binding_Task_Practice = df_Binding_Task_Practice[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'acc', 'average_response_time',
     'BindingRawScore', 'correct', 'correct_response', 'counter', 'Delay', 'eightsec_accuracy', 'FalseAlarms', 'height',
     'Hits', 'live_row', 'logfile', 'match_1s_accuracy', 'match_1s_avg_rt', 'match_8s_accuracy', 'match_8s_avg_rt',
     'mismatch_1s_accuracy', 'mismatch_1s_avg_rt', 'mismatch_8s_accuracy', 'mismatch_8s_avg_rt', 'NNonResponses',
     'Omissions', 'onesec_accuracy', 'QuinetteAccuracyScore', 'QuinetteProcessingScore', 'response',
     'response_time', 'ResponsesGiven', 'total_correct', 'total_match_1s_rt', 'total_match_8s_rt',
     'total_mismatch_1s_rt', 'total_mismatch_8s_rt', 'total_response_time', 'total_responses', 'width']]
# Replace commas with dots before float casting
df_Binding_Task_Practice[['acc', 'average_response_time']] = df_Binding_Task_Practice[['acc', 'average_response_time']].replace(',', '.')
# Convert numerical and text fields to proper types
df_Binding_Task_Practice = df_Binding_Task_Practice.astype(
    {'acc': 'float64', 'average_response_time': 'float64', 'correct_response': 'str', 'response': 'str'})
# Rename Binding Task columns to practice-session format
df_Binding_Task_Practice.columns = ['subject_nr','CB_ref','practice','TrialNumber','acc_Practice_Sess',
                             'average_response_time_Practice_Sess','BindingRawScore_Practice_Sess','correct_Practice_Sess','correct_response_Practice_Sess',
                             'counter_Practice_Sess','Delay_Practice_Sess','eightsec_accuracy_Practice_Sess','FalseAlarms_Practice_Sess','height_Practice_Sess','Hits_Practice_Sess',
                             'live_row_Practice_Sess','logfile_Practice_Sess','match_1s_accuracy_Practice_Sess','match_1s_avg_rt_Practice_Sess','match_8s_accuracy_Practice_Sess',
                             'match_8s_avg_rt_Practice_Sess','mismatch_1s_accuracy_Practice_Sess','mismatch_1s_avg_rt_Practice_Sess','mismatch_8s_accuracy_Practice_Sess',
                             'mismatch_8s_avg_rt_Practice_Sess','NNonResponses_Practice_Sess','Omissions_Practice_Sess','onesec_accuracy_Practice_Sess',
                             'QuinetteAccuracyScore_Practice_Sess','QuinetteProcessingScore_Practice_Sess','response_Practice_Sess','response_time_Practice_Sess',
                             'ResponsesGiven_Practice_Sess','total_correct_Practice_Sess','total_match_1s_rt_Practice_Sess','total_match_8s_rt_Practice_Sess',
                             'total_mismatch_1s_rt_Practice_Sess','total_mismatch_8s_rt_Practice_Sess','total_response_time_Practice_Sess','total_responses_Practice_Sess',
                             'width_Practice_Sess']

############################################################################################
############################################################################################
# Select Operation Span columns, convert formats, remove unwanted strings, and rename
df_Operation_Span_Practice = df_Operation_Span_Practice[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'height', 'letter', 'List_Prev_Letter', 'List_responses_memory',
     'live_row', 'logfile', 'response_average_time_memory', 'response_memory', 'response_processing',
     'response_time_memory', 'response_time_processing', 'response_total_time_memory', 'OP_part_process_time',
     'score_practice', 'score_operation_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4', 'score_subblock_5',
     'score_subblock_6', 'Tipo', 'total_correct', 'total_response_time',
     'total_responses', 'width']]
# Fix comma decimal separators before numeric conversion
df_Operation_Span_Practice[['acc', 'avg_rt']] = df_Operation_Span_Practice[['acc', 'avg_rt']].replace(',', '.')
# Convert selected fields to numeric or string types
df_Operation_Span_Practice = df_Operation_Span_Practice.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
# Identify unwanted string in processing responses
example_c = df_Operation_Span_Practice["response_processing"].iloc[2]
# Remove the unwanted string from the dataframe
df_Operation_Span_Practice = df_Operation_Span_Practice.replace(example_c, '')
# Rename Operation Span columns to practice-session format
df_Operation_Span_Practice.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                               'SubTaskName','acc_Practice_Sess','avg_rt_Practice_Sess','BlockChoice_Practice_Sess','correct_Practice_Sess',
                               'correct_response_Practice_Sess','height_Practice_Sess','letter_Practice_Sess','List_Prev_Letter_Practice_Sess',
                               'List_responses_memory_Practice_Sess','live_row_Practice_Sess','logfile_Practice_Sess','response_average_time_memory_Practice_Sess',
                               'response_memory_Practice_Sess','response_processing_Practice_Sess','response_time_memory_Practice_Sess',
                               'response_time_processing_Practice_Sess','response_total_time_memory_Practice_Sess','OP_part_process_time_Practice_Sess',
                               'score_practice_Practice_Sess','score_operation_span_Practice_Sess','score_subblock_2_Practice_Sess','score_subblock_3_Practice_Sess',
                               'score_subblock_4_Practice_Sess','score_subblock_5_Practice_Sess','score_subblock_6_Practice_Sess','Tipo_Practice_Sess',
                               'total_correct_Practice_Sess','total_response_time_Practice_Sess','total_responses_Practice_Sess','width_Practice_Sess']

# Reset index and drop the old index column
df_Reading_Span_Practice = df_Reading_Span_Practice.reset_index(drop=True)

# Configure pandas to show all columns when printing
pd.set_option("display.max_columns", None)
# Display cleaned and formatted Reading Span practice dataframe
df_Reading_Span_Practice

## 4.3. Generates database with raw scores in the five WM tasks (practice session)

In [ ]:
# Create empty dataframe to store raw scores for each task
df_raw_scores = pd.DataFrame()

# Insert subject number column into the raw scores dataframe
subj_nr = df_total_part["subject_nr"].unique()
df_raw_scores.insert(0,'subject_nr',subj_nr)

##############################################################################################################
# Load temperature data and map temperature values to participants
PROJ_ROOT = Path("..").resolve()

# Build the path to the temperature Excel file
excel_part_data_path = PROJ_ROOT / "Temperature" / "Body_Temperature_Collection.xlsx"
Temperature = pd.read_excel(excel_part_data_path, sheet_name='TempPractice')

# Convert subject numbers to integer for matching
aa = df_raw_scores["subject_nr"].astype("int")

# Iterate through temperature file and assign temperature values to matching subjects
ii = 0
k = 0
for i in Temperature["Subject ID"]:
    if int(i) in aa.values:
        df_raw_scores["Temperature (°C) Practice Sess"] = Temperature["Temperature (°C) Practice Sess"][ii]
    ii += 1
    k =+ 1

########################################################################################################
# Compute and insert raw scores for each WM task based on max performance per participant
RawRS = list(df_Reading_Span_Practice.groupby(['subject_nr'], sort=True)['score_reading_span_Practice_Sess'].max() * 20)
df_raw_scores["Reading Span Practice Session"] = RawRS

RawUT = list(df_WMU_Task_Practice.groupby(['subject_nr'], sort=True)['WMUExperimentalScore_Practice_Sess'].max())
df_raw_scores["Updating Task Practice Session"] = RawUT

RawSS = list(df_Symmetry_Span_Practice.groupby(['subject_nr'], sort=True)['score_symmetry_span_Practice_Sess'].max() * 20)
df_raw_scores["Symmetry Span Practice Session"] = RawSS

RawBT = list(df_Binding_Task_Practice.groupby(['subject_nr'], sort=True)['BindingRawScore_Practice_Sess'].max())
df_raw_scores["Binding Task Practice Session"] = RawBT

RawOS = list(df_Operation_Span_Practice.groupby(['subject_nr'], sort=True)['score_operation_span_Practice_Sess'].max() * 20)
df_raw_scores["Operation Span Practice Session"] = RawOS

# Convert subject number column to numeric and sort dataframe
df_raw_scores["subject_nr"] = pd.to_numeric(df_raw_scores["subject_nr"], errors="coerce")
df_raw_scores = df_raw_scores.sort_values(by="subject_nr").reset_index(drop=True)

# Convert all columns except subject number and temperature to integers
df_raw_scores = df_raw_scores.astype({
    col: 'int' 
    for col in df_raw_scores.columns 
    if col not in ['subject_nr', 'Temperature (°C) Practice Sess']
})

# Create a sorted copy of the raw scores dataframe
df_raw_scores_pract = df_raw_scores.sort_values(by="subject_nr")

# Show all columns when printing
pd.set_option("display.max_columns", None)

# Display final practice-session raw scores dataframe
df_raw_scores_pract

## 4.4. Generates database with normalized scores in the five WM tasks (practice session)

In [ ]:
########################################################################################################################################
# Create dataframe containing normalized WM scores and socio-demographic data
df_normalized_scores = df_raw_scores.copy(deep=True)

# Normalize Reading Span scores by maximum possible score
df_normalized_scores["Reading Span Practice Session"] = df_normalized_scores["Reading Span Practice Session"]/20
# Normalize Updating Task scores by maximum score
df_normalized_scores["Updating Task Practice Session"] = df_normalized_scores["Updating Task Practice Session"] / 36
# Normalize Symmetry Span scores by maximum possible score
df_normalized_scores["Symmetry Span Practice Session"] = df_normalized_scores["Symmetry Span Practice Session"] / 20
# Normalize Binding Task scores by maximum possible score
df_normalized_scores["Binding Task Practice Session"] = df_normalized_scores["Binding Task Practice Session"] / 12
# Normalize Operation Span scores by maximum possible score
df_normalized_scores["Operation Span Practice Session"] = df_normalized_scores["Operation Span Practice Session"] / 20

# Columns requiring rounding after numeric conversion
cols_to_round = [
    "Reading Span Practice Session",
    "Updating Task Practice Session",
    "Symmetry Span Practice Session",
    "Binding Task Practice Session",
    "Operation Span Practice Session",
]

# Convert selected columns to numeric and round values to two decimals
df_normalized_scores[cols_to_round] = (
    df_normalized_scores[cols_to_round]
    .apply(pd.to_numeric, errors="coerce")
    .round(2)
)

# Ensure subject number is numeric and sort rows
df_normalized_scores["subject_nr"] = pd.to_numeric(df_normalized_scores["subject_nr"], errors="coerce")
df_normalized_scores = df_normalized_scores.sort_values(by="subject_nr").reset_index(drop=True)

# Create sorted version of normalized scores dataframe
df_normalized_scores_pract = df_normalized_scores.sort_values(by="subject_nr")

# Display all columns when printing
pd.set_option("display.max_columns", None)

# Show normalized scores dataframe
df_normalized_scores_pract

# 5. Working memory (WM) tasks experimental session database
Generates DB with the performance data in the WM tasks collected in the experimental sessions.

## 5.1. Importing data

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Import utilities for filesystem paths
from pathlib import Path
# Import pandas for data manipulation
import pandas as pd

# Define project root directory one level above current folder
PROJ_ROOT = Path("..").resolve()
# Define directory containing experimental participant CSV files
data_dir = PROJ_ROOT / "data_participants_experimental"

# Collect all CSV files in the directory, sorted alphabetically
csv_files = sorted(data_dir.glob("*.csv"))

# Attempt to read CSV files using several common encodings and delimiters
def try_read(p):
    # List of read attempts covering typical CSV formats across systems
    tries = [
        dict(sep=None, engine="python", encoding="utf-8-sig"),
        dict(sep=";",  engine="python", encoding="utf-8-sig"),
        dict(sep=",",  engine="python", encoding="utf-8-sig"),

        dict(sep=None, engine="python", encoding="latin-1"),
        dict(sep=";",  engine="python", encoding="latin-1"),
        dict(sep=",",  engine="python", encoding="latin-1"),

        dict(sep="\t", engine="python", encoding="utf-16"),
        dict(sep=",",  engine="python", encoding="utf-16"),
        dict(sep=";",  engine="python", encoding="utf-16"),
    ]

    # Track last error in case all attempts fail
    last_err = None

    # Try reading using each configuration
    for kw in tries:
        try:
            return pd.read_csv(str(p), error_bad_lines=False, warn_bad_lines=False, **kw)
        except TypeError:
            # Retry with only compatible arguments if pandas version is older
            try:
                return pd.read_csv(str(p), **{k:v for k,v in kw.items() if k in ("sep","engine","encoding")})
            except Exception as e:
                last_err = e
        except Exception as e:
            last_err = e

    # Raise the last error if no attempt succeeded
    raise last_err

# List to collect all successfully loaded dataframes
dfs = []

# Loop through each CSV file and load it using try_read
for p in csv_files:
    df = try_read(p)
    dfs.append(df)

# Concatenate all participant dataframes into a single one
df_total_part = pd.concat(dfs, ignore_index=True)

# Reset the index to clean up the concatenation index
df_total_part.reset_index(inplace=True)

# Replace 'undefined' text entries with 'Nan' to mark missing values consistently
df_total_part = df_total_part.replace('undefined','Nan')

# Move Task_Name and subject_nr columns to be the first and second columns
first_column = df_total_part.pop('Task_Name')
second_column = df_total_part.pop('subject_nr')
df_total_part.insert(0, 'Task_Name', first_column)
df_total_part.insert(1, 'subject_nr', second_column)

# Ensure all columns are fully visible when printing the dataframe
pd.set_option("display.max_columns", None)

# Display the final combined experimental participants dataframe
df_total_part

## 5.2. Extracts and cleans experimental-session data for each WM task

In [ ]:
#Os seguintes 5 blocos de código criam 5 DataFrames distintas.
#Cada uma das DataFrames vai conter a informação referente a cada uma das sete tarefas de memória de trabalho (reading span, symmetry
# span, operation span, binding task e Updating Task) realizadas por todos os participantes.
df_Reading_Span = df_total_part[df_total_part['Task_Name'] == 'Reading Span']
df_Reading_Span = df_Reading_Span.sort_values(by=['selSNr'], kind='mergesort')
df_Reading_Span = df_Reading_Span.sort_values(by=['subject_nr'], kind='mergesort')
df_Reading_Span_1 = df_Reading_Span.query('selSNr == 1 and 1 <= subject_nr <= 5 or selSNr == 4 and 6 <= subject_nr <= 10 or selSNr == 3 and 11 <= subject_nr <= 15 or selSNr == 2 and 16 <= subject_nr <= 20 or selSNr == 1 and subject_nr == 21 or selSNr == 4 and subject_nr == 22 or selSNr == 3 and subject_nr == 23 or selSNr == 2 and subject_nr == 24 or selSNr == 1 and subject_nr == 25 or selSNr == 4 and subject_nr == 26 or selSNr == 3 and subject_nr == 27  or selSNr == 2 and subject_nr == 28')
df_Reading_Span_1 = df_Reading_Span_1.reset_index(drop=True)
df_Reading_Span_2 = df_Reading_Span.query('selSNr == 2 and 1 <= subject_nr <= 5 or selSNr == 1 and 6 <= subject_nr <= 10 or selSNr == 4 and 11 <= subject_nr <= 15 or selSNr == 3 and 16 <= subject_nr <= 20 or selSNr == 2 and subject_nr == 21 or selSNr == 1 and subject_nr == 22 or selSNr == 4 and subject_nr == 23 or selSNr == 3 and subject_nr == 24 or selSNr == 2 and subject_nr == 25 or selSNr == 1 and subject_nr == 26  or selSNr == 4 and subject_nr == 27  or selSNr == 3 and subject_nr == 28')
df_Reading_Span_2 = df_Reading_Span_2.reset_index(drop=True)
df_Reading_Span_3 = df_Reading_Span.query('selSNr == 3 and 1 <= subject_nr <= 5 or selSNr == 2 and 6 <= subject_nr <= 10 or selSNr == 1 and 11 <= subject_nr <= 15 or selSNr == 4 and 16 <= subject_nr <= 20 or selSNr == 3 and subject_nr == 21 or selSNr == 2 and subject_nr == 22 or selSNr == 1 and subject_nr == 23 or selSNr == 4 and subject_nr == 24 or selSNr == 3 and subject_nr == 25 or selSNr == 2 and subject_nr == 26  or selSNr == 1 and subject_nr == 27  or selSNr == 4 and subject_nr == 28')
df_Reading_Span_3 = df_Reading_Span_3.reset_index(drop=True)
df_Reading_Span_4 = df_Reading_Span.query('selSNr == 4 and 1 <= subject_nr <= 5 or selSNr == 3 and 6 <= subject_nr <= 10 or selSNr == 2 and 11 <= subject_nr <= 15 or selSNr == 1 and 16 <= subject_nr <= 20 or selSNr == 4 and subject_nr == 21 or selSNr == 3 and subject_nr == 22 or selSNr == 2 and subject_nr == 23 or selSNr == 1 and subject_nr == 24 or selSNr == 4 and subject_nr == 25 or selSNr == 3 and subject_nr == 26  or selSNr == 2 and subject_nr == 27  or selSNr == 1 and subject_nr == 28')
df_Reading_Span_4 = df_Reading_Span_4.reset_index(drop=True)

df_WMU_Task = df_total_part[df_total_part['Task_Name'] == 'Working Memory Updating Task']
df_WMU_Task = df_WMU_Task.sort_values(by=['selSNr'], kind='mergesort')
df_WMU_Task = df_WMU_Task.sort_values(by=['subject_nr'], kind='mergesort')
df_WMU_Task_1 = df_WMU_Task.query('selSNr == 1 and 1 <= subject_nr <= 5 or selSNr == 4 and 6 <= subject_nr <= 10 or selSNr == 3 and 11 <= subject_nr <= 15 or selSNr == 2 and 16 <= subject_nr <= 20 or selSNr == 1 and subject_nr == 21 or selSNr == 4 and subject_nr == 22 or selSNr == 3 and subject_nr == 23 or selSNr == 2 and subject_nr == 24 or selSNr == 1 and subject_nr == 25 or selSNr == 4 and subject_nr == 26  or selSNr == 3 and subject_nr == 27  or selSNr == 2 and subject_nr == 28')
df_WMU_Task_1 = df_WMU_Task_1.reset_index(drop=True)
df_WMU_Task_2 = df_WMU_Task.query('selSNr == 2 and 1 <= subject_nr <= 5 or selSNr == 1 and 6 <= subject_nr <= 10 or selSNr == 4 and 11 <= subject_nr <= 15 or selSNr == 3 and 16 <= subject_nr <= 20 or selSNr == 2 and subject_nr == 21 or selSNr == 1 and subject_nr == 22 or selSNr == 4 and subject_nr == 23 or selSNr == 3 and subject_nr == 24 or selSNr == 2 and subject_nr == 25 or selSNr == 1 and subject_nr == 26  or selSNr == 4 and subject_nr == 27  or selSNr == 3 and subject_nr == 28')
df_WMU_Task_2 = df_WMU_Task_2.reset_index(drop=True)
df_WMU_Task_3 = df_WMU_Task.query('selSNr == 3 and 1 <= subject_nr <= 5 or selSNr == 2 and 6 <= subject_nr <= 10 or selSNr == 1 and 11 <= subject_nr <= 15 or selSNr == 4 and 16 <= subject_nr <= 20 or selSNr == 3 and subject_nr == 21 or selSNr == 2 and subject_nr == 22 or selSNr == 1 and subject_nr == 23 or selSNr == 4 and subject_nr == 24 or selSNr == 3 and subject_nr == 25 or selSNr == 2 and subject_nr == 26  or selSNr == 1 and subject_nr == 27  or selSNr == 4 and subject_nr == 28')
df_WMU_Task_3 = df_WMU_Task_3.reset_index(drop=True)
df_WMU_Task_4 = df_WMU_Task.query('selSNr == 4 and 1 <= subject_nr <= 5 or selSNr == 3 and 6 <= subject_nr <= 10 or selSNr == 2 and 11 <= subject_nr <= 15 or selSNr == 1 and 16 <= subject_nr <= 20 or selSNr == 4 and subject_nr == 21 or selSNr == 3 and subject_nr == 22 or selSNr == 2 and subject_nr == 23 or selSNr == 1 and subject_nr == 24 or selSNr == 4 and subject_nr == 25 or selSNr == 3 and subject_nr == 26  or selSNr == 2 and subject_nr == 27  or selSNr == 1 and subject_nr == 28')
df_WMU_Task_4 = df_WMU_Task_4.reset_index(drop=True)

df_Symmetry_Span = df_total_part[df_total_part['Task_Name'] == 'Symmetry Span']
df_Symmetry_Span = df_Symmetry_Span.sort_values(by=['selSNr'], kind='mergesort')
df_Symmetry_Span = df_Symmetry_Span.sort_values(by=['subject_nr'], kind='mergesort')
df_Symmetry_Span_1 = df_Symmetry_Span.query('selSNr == 1 and 1 <= subject_nr <= 5 or selSNr == 4 and 6 <= subject_nr <= 10 or selSNr == 3 and 11 <= subject_nr <= 15 or selSNr == 2 and 16 <= subject_nr <= 20 or selSNr == 1 and subject_nr == 21 or selSNr == 4 and subject_nr == 22 or selSNr == 3 and subject_nr == 23 or selSNr == 2 and subject_nr == 24 or selSNr == 1 and subject_nr == 25 or selSNr == 4 and subject_nr == 26 or selSNr == 3 and subject_nr == 27  or selSNr == 2 and subject_nr == 28')
df_Symmetry_Span_1 = df_Symmetry_Span_1.reset_index(drop=True)
df_Symmetry_Span_2 = df_Symmetry_Span.query('selSNr == 2 and 1 <= subject_nr <= 5 or selSNr == 1 and 6 <= subject_nr <= 10 or selSNr == 4 and 11 <= subject_nr <= 15 or selSNr == 3 and 16 <= subject_nr <= 20 or selSNr == 2 and subject_nr == 21 or selSNr == 1 and subject_nr == 22 or selSNr == 4 and subject_nr == 23 or selSNr == 3 and subject_nr == 24 or selSNr == 2 and subject_nr == 25 or selSNr == 1 and subject_nr == 26 or selSNr == 4 and subject_nr == 27  or selSNr == 3 and subject_nr == 28')
df_Symmetry_Span_2 = df_Symmetry_Span_2.reset_index(drop=True)
df_Symmetry_Span_3 = df_Symmetry_Span.query('selSNr == 3 and 1 <= subject_nr <= 5 or selSNr == 2 and 6 <= subject_nr <= 10 or selSNr == 1 and 11 <= subject_nr <= 15 or selSNr == 4 and 16 <= subject_nr <= 20 or selSNr == 3 and subject_nr == 21 or selSNr == 2 and subject_nr == 22 or selSNr == 1 and subject_nr == 23 or selSNr == 4 and subject_nr == 24 or selSNr == 3 and subject_nr == 25 or selSNr == 2 and subject_nr == 26 or selSNr == 1 and subject_nr == 27  or selSNr == 4 and subject_nr == 28')
df_Symmetry_Span_3 = df_Symmetry_Span_3.reset_index(drop=True)
df_Symmetry_Span_4 = df_Symmetry_Span.query('selSNr == 4 and 1 <= subject_nr <= 5 or selSNr == 3 and 6 <= subject_nr <= 10 or selSNr == 2 and 11 <= subject_nr <= 15 or selSNr == 1 and 16 <= subject_nr <= 20 or selSNr == 4 and subject_nr == 21 or selSNr == 3 and subject_nr == 22 or selSNr == 2 and subject_nr == 23 or selSNr == 1 and subject_nr == 24 or selSNr == 4 and subject_nr == 25 or selSNr == 3 and subject_nr == 26 or selSNr == 2 and subject_nr == 27  or selSNr == 1 and subject_nr == 28')
df_Symmetry_Span_4 = df_Symmetry_Span_4.reset_index(drop=True)

df_Binding_Task = df_total_part[df_total_part['Task_Name'] == 'Binding Task']
df_Binding_Task = df_Binding_Task.sort_values(by=['selSNr'], kind='mergesort')
df_Binding_Task = df_Binding_Task.sort_values(by=['subject_nr'], kind='mergesort')
df_Binding_Task_1 = df_Binding_Task.query('selSNr == 1 and 1 <= subject_nr <= 5 or selSNr == 4 and 6 <= subject_nr <= 10 or selSNr == 3 and 11 <= subject_nr <= 15 or selSNr == 2 and 16 <= subject_nr <= 20 or selSNr == 1 and subject_nr == 21 or selSNr == 4 and subject_nr == 22 or selSNr == 3 and subject_nr == 23 or selSNr == 2 and subject_nr == 24 or selSNr == 1 and subject_nr == 25 or selSNr == 4 and subject_nr == 26 or selSNr == 3 and subject_nr == 27  or selSNr == 2 and subject_nr == 28')
df_Binding_Task_1 = df_Binding_Task_1.reset_index(drop=True)
df_Binding_Task_2 = df_Binding_Task.query('selSNr == 2 and 1 <= subject_nr <= 5 or selSNr == 1 and 6 <= subject_nr <= 10 or selSNr == 4 and 11 <= subject_nr <= 15 or selSNr == 3 and 16 <= subject_nr <= 20 or selSNr == 2 and subject_nr == 21 or selSNr == 1 and subject_nr == 22 or selSNr == 4 and subject_nr == 23 or selSNr == 3 and subject_nr == 24 or selSNr == 2 and subject_nr == 25 or selSNr == 1 and subject_nr == 26 or selSNr == 4 and subject_nr == 27  or selSNr == 3 and subject_nr == 28')
df_Binding_Task_2 = df_Binding_Task_2.reset_index(drop=True)
df_Binding_Task_3 = df_Binding_Task.query('selSNr == 3 and 1 <= subject_nr <= 5 or selSNr == 2 and 6 <= subject_nr <= 10 or selSNr == 1 and 11 <= subject_nr <= 15 or selSNr == 4 and 16 <= subject_nr <= 20 or selSNr == 3 and subject_nr == 21 or selSNr == 2 and subject_nr == 22 or selSNr == 1 and subject_nr == 23 or selSNr == 4 and subject_nr == 24 or selSNr == 3 and subject_nr == 25 or selSNr == 2 and subject_nr == 26 or selSNr == 1 and subject_nr == 27  or selSNr == 4 and subject_nr == 28')
df_Binding_Task_3 = df_Binding_Task_3.reset_index(drop=True)
df_Binding_Task_4 = df_Binding_Task.query('selSNr == 4 and 1 <= subject_nr <= 5 or selSNr == 3 and 6 <= subject_nr <= 10 or selSNr == 2 and 11 <= subject_nr <= 15 or selSNr == 1 and 16 <= subject_nr <= 20 or selSNr == 4 and subject_nr == 21 or selSNr == 3 and subject_nr == 22 or selSNr == 2 and subject_nr == 23 or selSNr == 1 and subject_nr == 24 or selSNr == 4 and subject_nr == 25 or selSNr == 3 and subject_nr == 26 or selSNr == 2 and subject_nr == 27  or selSNr == 1 and subject_nr == 28')
df_Binding_Task_4 = df_Binding_Task_4.reset_index(drop=True)

df_Operation_Span = df_total_part[df_total_part['Task_Name'] == 'Operation Span']
df_Operation_Span = df_Operation_Span.sort_values(by=['selSNr'], kind='mergesort')
df_Operation_Span = df_Operation_Span.sort_values(by=['subject_nr'], kind='mergesort')
df_Operation_Span_1 = df_Operation_Span.query('selSNr == 1 and 1 <= subject_nr <= 5 or selSNr == 4 and 6 <= subject_nr <= 10 or selSNr == 3 and 11 <= subject_nr <= 15 or selSNr == 2 and 16 <= subject_nr <= 20 or selSNr == 1 and subject_nr == 21 or selSNr == 4 and subject_nr == 22 or selSNr == 3 and subject_nr == 23 or selSNr == 2 and subject_nr == 24 or selSNr == 1 and subject_nr == 25 or selSNr == 4 and subject_nr == 26 or selSNr == 3 and subject_nr == 27  or selSNr == 2 and subject_nr == 28')
df_Operation_Span_1 = df_Operation_Span_1.reset_index(drop=True)
df_Operation_Span_2 = df_Operation_Span.query('selSNr == 2 and 1 <= subject_nr <= 5 or selSNr == 1 and 6 <= subject_nr <= 10 or selSNr == 4 and 11 <= subject_nr <= 15 or selSNr == 3 and 16 <= subject_nr <= 20 or selSNr == 2 and subject_nr == 21 or selSNr == 1 and subject_nr == 22 or selSNr == 4 and subject_nr == 23 or selSNr == 3 and subject_nr == 24 or selSNr == 2 and subject_nr == 25 or selSNr == 1 and subject_nr == 26 or selSNr == 4 and subject_nr == 27  or selSNr == 3 and subject_nr == 28')
df_Operation_Span_2 = df_Operation_Span_2.reset_index(drop=True)
df_Operation_Span_3 = df_Operation_Span.query('selSNr == 3 and 1 <= subject_nr <= 5 or selSNr == 2 and 6 <= subject_nr <= 10 or selSNr == 1 and 11 <= subject_nr <= 15 or selSNr == 4 and 16 <= subject_nr <= 20 or selSNr == 3 and subject_nr == 21 or selSNr == 2 and subject_nr == 22 or selSNr == 1 and subject_nr == 23 or selSNr == 4 and subject_nr == 24 or selSNr == 3 and subject_nr == 25 or selSNr == 2 and subject_nr == 26 or selSNr == 1 and subject_nr == 27  or selSNr == 4 and subject_nr == 28')
df_Operation_Span_3 = df_Operation_Span_3.reset_index(drop=True)
df_Operation_Span_4 = df_Operation_Span.query('selSNr == 4 and 1 <= subject_nr <= 5 or selSNr == 3 and 6 <= subject_nr <= 10 or selSNr == 2 and 11 <= subject_nr <= 15 or selSNr == 1 and 16 <= subject_nr <= 20 or selSNr == 4 and subject_nr == 21 or selSNr == 3 and subject_nr == 22 or selSNr == 2 and subject_nr == 23 or selSNr == 1 and subject_nr == 24 or selSNr == 4 and subject_nr == 25 or selSNr == 3 and subject_nr == 26 or selSNr == 2 and subject_nr == 27  or selSNr == 1 and subject_nr == 28')
df_Operation_Span_4 = df_Operation_Span_4.reset_index(drop=True)

#Os próximos 5 blocos de código selecionam as colunas com informação relevante de cada WM task e guardam estas colunas em DataFrames
#que só contêm informação relacionada com a mesma tarefa. Para além disso, estes nestes 5 blocos de código, são realizadas algumas
#conversões no formato dos dados (e.g., string to float) e são alterados os nomes de algumas colunas de forma a ficarem mais percétiveis.
#'selSNr',
df_Reading_Span_1 = df_Reading_Span_1[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'Frase', 'height', 'letter', 'List_Prev_Letter',
     'List_responses_memory', 'live_row', 'logfile', 'response_average_time_memory', 'response_memory',
     'response_processing', 'response_time_memory', 'response_time_processing', 'response_total_time_memory',
     'RP_part_process_time', 'score_practice', 'score_reading_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4',
     'score_subblock_5', 'score_subblock_6', 'Tipo', 'total_correct',
     'total_response_time', 'total_responses', 'width']]
df_Reading_Span_1[['acc', 'avg_rt']] = df_Reading_Span_1[['acc', 'avg_rt']].replace(',', '.')
df_Reading_Span_1 = df_Reading_Span_1.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_b = df_Reading_Span_1["response_processing"].iloc[2]
df_Reading_Span_1 = df_Reading_Span_1.replace(example_b, '')
#df_Reading_Span_1 = df_Reading_Span_1.sort_values(by=['selSNr'], kind='mergesort')
#df_Reading_Span_1 = df_Reading_Span_1.sort_values(by=['subject_nr'], kind='mergesort')
df_Reading_Span_1.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                             'SubTaskName','acc_Sess09h00','avg_rt_Sess09h00','BlockChoice_Sess09h00','correct_Sess09h00','correct_response_Sess09h00',
                             'Frase_Sess09h00','height_Sess09h00','letter_Sess09h00','List_Prev_Letter_Sess09h00','List_responses_memory_Sess09h00',
                             'live_row_Sess09h00','logfile_Sess09h00','response_average_time_memory_Sess09h00','response_memory_Sess09h00',
                             'response_processing_Sess09h00','response_time_memory_Sess09h00','response_time_processing_Sess09h00',
                             'response_total_time_memory_Sess09h00','RP_part_process_time_Sess09h00','score_practice_Sess09h00',
                             'score_reading_span_Sess09h00','score_subblock_2_Sess09h00','score_subblock_3_Sess09h00','score_subblock_4_Sess09h00',
                             'score_subblock_5_Sess09h00','score_subblock_6_Sess09h00','Tipo_Sess09h00','total_correct_Sess09h00',
                             'total_response_time_Sess09h00','total_responses_Sess09h00','width_Sess09h00']
#'selSNr',
df_Reading_Span_2 = df_Reading_Span_2[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'Frase', 'height', 'letter', 'List_Prev_Letter',
     'List_responses_memory', 'live_row', 'logfile', 'response_average_time_memory', 'response_memory',
     'response_processing', 'response_time_memory', 'response_time_processing', 'response_total_time_memory',
     'RP_part_process_time', 'score_practice', 'score_reading_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4',
     'score_subblock_5', 'score_subblock_6', 'Tipo', 'total_correct',
     'total_response_time', 'total_responses', 'width']]
df_Reading_Span_2[['acc', 'avg_rt']] = df_Reading_Span_2[['acc', 'avg_rt']].replace(',', '.')
df_Reading_Span_2 = df_Reading_Span_2.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_b = df_Reading_Span_2["response_processing"].iloc[2]
df_Reading_Span_2 = df_Reading_Span_2.replace(example_b, '')
df_Reading_Span_2.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
#df_Reading_Span_2 = df_Reading_Span_2.sort_values(by=['selSNr'], kind='mergesort')
#df_Reading_Span_2 = df_Reading_Span_2.sort_values(by=['subject_nr'], kind='mergesort')
df_Reading_Span_2.columns = ['acc_Sess13h00','avg_rt_Sess13h00','BlockChoice_Sess13h00','correct_Sess13h00','correct_response_Sess13h00',
                             'Frase_Sess13h00','height_Sess13h00','letter_Sess13h00','List_Prev_Letter_Sess13h00','List_responses_memory_Sess13h00',
                             'live_row_Sess13h00','logfile_Sess13h00','response_average_time_memory_Sess13h00','response_memory_Sess13h00',
                             'response_processing_Sess13h00','response_time_memory_Sess13h00','response_time_processing_Sess13h00',
                             'response_total_time_memory_Sess13h00','RP_part_process_time_Sess13h00','score_practice_Sess13h00',
                             'score_reading_span_Sess13h00','score_subblock_2_Sess13h00','score_subblock_3_Sess13h00','score_subblock_4_Sess13h00',
                             'score_subblock_5_Sess13h00','score_subblock_6_Sess13h00','Tipo_Sess13h00','total_correct_Sess13h00',
                             'total_response_time_Sess13h00','total_responses_Sess13h00','width_Sess13h00']

#'selSNr',
df_Reading_Span_3 = df_Reading_Span_3[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'Frase', 'height', 'letter', 'List_Prev_Letter',
     'List_responses_memory', 'live_row', 'logfile', 'response_average_time_memory', 'response_memory',
     'response_processing', 'response_time_memory', 'response_time_processing', 'response_total_time_memory',
     'RP_part_process_time', 'score_practice', 'score_reading_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4',
     'score_subblock_5', 'score_subblock_6', 'Tipo', 'total_correct',
     'total_response_time', 'total_responses', 'width']]
df_Reading_Span_3[['acc', 'avg_rt']] = df_Reading_Span_3[['acc', 'avg_rt']].replace(',', '.')
df_Reading_Span_3 = df_Reading_Span_3.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_b = df_Reading_Span_3["response_processing"].iloc[2]
df_Reading_Span_3 = df_Reading_Span_3.replace(example_b, '')
#df_Reading_Span_3 = df_Reading_Span_3.sort_values(by=['selSNr'], kind='mergesort')
#df_Reading_Span_3 = df_Reading_Span_3.sort_values(by=['subject_nr'], kind='mergesort')
df_Reading_Span_3.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Reading_Span_3.columns = ['acc_Sess17h00','avg_rt_Sess17h00','BlockChoice_Sess17h00','correct_Sess17h00','correct_response_Sess17h00',
                             'Frase_Sess17h00','height_Sess17h00','letter_Sess17h00','List_Prev_Letter_Sess17h00','List_responses_memory_Sess17h00',
                             'live_row_Sess17h00','logfile_Sess17h00','response_average_time_memory_Sess17h00','response_memory_Sess17h00',
                             'response_processing_Sess17h00','response_time_memory_Sess17h00','response_time_processing_Sess17h00',
                             'response_total_time_memory_Sess17h00','RP_part_process_time_Sess17h00','score_practice_Sess17h00',
                             'score_reading_span_Sess17h00','score_subblock_2_Sess17h00','score_subblock_3_Sess17h00','score_subblock_4_Sess17h00',
                             'score_subblock_5_Sess17h00','score_subblock_6_Sess17h00','Tipo_Sess17h00','total_correct_Sess17h00',
                             'total_response_time_Sess17h00','total_responses_Sess17h00','width_Sess17h00']
#'selSNr',
df_Reading_Span_4 = df_Reading_Span_4[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'Frase', 'height', 'letter', 'List_Prev_Letter',
     'List_responses_memory', 'live_row', 'logfile', 'response_average_time_memory', 'response_memory',
     'response_processing', 'response_time_memory', 'response_time_processing', 'response_total_time_memory',
     'RP_part_process_time', 'score_practice', 'score_reading_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4',
     'score_subblock_5', 'score_subblock_6', 'Tipo', 'total_correct',
     'total_response_time', 'total_responses', 'width']]
df_Reading_Span_4[['acc', 'avg_rt']] = df_Reading_Span_4[['acc', 'avg_rt']].replace(',', '.')
df_Reading_Span_4 = df_Reading_Span_4.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_b = df_Reading_Span_4["response_processing"].iloc[2]
df_Reading_Span_4 = df_Reading_Span_4.replace(example_b, '')
#df_Reading_Span_4 = df_Reading_Span_4.sort_values(by=['selSNr'], kind='mergesort')
#df_Reading_Span_4 = df_Reading_Span_4.sort_values(by=['subject_nr'], kind='mergesort')
df_Reading_Span_4.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Reading_Span_4.columns = ['acc_Sess21h00','avg_rt_Sess21h00','BlockChoice_Sess21h00','correct_Sess21h00','correct_response_Sess21h00',
                             'Frase_Sess21h00','height_Sess21h00','letter_Sess21h00','List_Prev_Letter_Sess21h00','List_responses_memory_Sess21h00',
                             'live_row_Sess21h00','logfile_Sess21h00','response_average_time_memory_Sess21h00','response_memory_Sess21h00',
                             'response_processing_Sess21h00','response_time_memory_Sess21h00','response_time_processing_Sess21h00',
                             'response_total_time_memory_Sess21h00','RP_part_process_time_Sess21h00','score_practice_Sess21h00',
                             'score_reading_span_Sess21h00','score_subblock_2_Sess21h00','score_subblock_3_Sess21h00','score_subblock_4_Sess21h00',
                             'score_subblock_5_Sess21h00','score_subblock_6_Sess21h00','Tipo_Sess21h00','total_correct_Sess21h00',
                             'total_response_time_Sess21h00','total_responses_Sess21h00','width_Sess21h00']

#'selSNr',

#print(df_Reading_Span_2.to_string())
df_Reading_Span_Experimental = pd.concat([df_Reading_Span_1,df_Reading_Span_2,df_Reading_Span_3,df_Reading_Span_4],axis=1)

df_Reading_Span_Experimental = df_Reading_Span_Experimental[['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName','acc_Sess09h00',
                                                   'acc_Sess13h00','acc_Sess17h00','acc_Sess21h00','avg_rt_Sess09h00','avg_rt_Sess13h00','avg_rt_Sess17h00','avg_rt_Sess21h00',
                                                   'BlockChoice_Sess09h00','BlockChoice_Sess13h00','BlockChoice_Sess17h00','BlockChoice_Sess21h00','correct_Sess09h00',
                                                   'correct_Sess13h00','correct_Sess17h00','correct_Sess21h00','correct_response_Sess09h00','correct_response_Sess13h00',
                                                   'correct_response_Sess17h00','correct_response_Sess21h00','Frase_Sess09h00','Frase_Sess13h00','Frase_Sess17h00','Frase_Sess21h00',
                                                   'height_Sess09h00','height_Sess13h00','height_Sess17h00','height_Sess21h00','letter_Sess09h00','letter_Sess13h00','letter_Sess17h00',
                                                   'letter_Sess21h00','List_Prev_Letter_Sess09h00','List_Prev_Letter_Sess13h00','List_Prev_Letter_Sess17h00',
                                                   'List_Prev_Letter_Sess21h00','List_responses_memory_Sess09h00','List_responses_memory_Sess13h00',
                                                   'List_responses_memory_Sess17h00','List_responses_memory_Sess21h00','live_row_Sess09h00','live_row_Sess13h00',
                                                   'live_row_Sess17h00','live_row_Sess21h00','logfile_Sess09h00','logfile_Sess13h00','logfile_Sess17h00','logfile_Sess21h00',
                                                   'response_average_time_memory_Sess09h00','response_average_time_memory_Sess13h00','response_average_time_memory_Sess17h00',
                                                   'response_average_time_memory_Sess21h00','response_memory_Sess09h00','response_memory_Sess13h00','response_memory_Sess17h00',
                                                   'response_memory_Sess21h00','response_processing_Sess09h00','response_processing_Sess13h00','response_processing_Sess17h00',
                                                   'response_processing_Sess21h00','response_time_memory_Sess09h00','response_time_memory_Sess13h00',
                                                   'response_time_memory_Sess17h00','response_time_memory_Sess21h00','response_time_processing_Sess09h00',
                                                   'response_time_processing_Sess13h00','response_time_processing_Sess17h00','response_time_processing_Sess21h00',
                                                   'response_total_time_memory_Sess09h00','response_total_time_memory_Sess13h00','response_total_time_memory_Sess17h00',
                                                   'response_total_time_memory_Sess21h00','RP_part_process_time_Sess09h00','RP_part_process_time_Sess13h00',
                                                   'RP_part_process_time_Sess17h00','RP_part_process_time_Sess21h00','score_practice_Sess09h00',
                                                   'score_practice_Sess13h00','score_practice_Sess17h00','score_practice_Sess21h00','score_reading_span_Sess09h00',
                                                   'score_reading_span_Sess13h00','score_reading_span_Sess17h00','score_reading_span_Sess21h00','score_subblock_2_Sess09h00',
                                                   'score_subblock_2_Sess13h00','score_subblock_2_Sess17h00','score_subblock_2_Sess21h00','score_subblock_3_Sess09h00',
                                                   'score_subblock_3_Sess13h00','score_subblock_3_Sess17h00','score_subblock_3_Sess21h00','score_subblock_4_Sess09h00',
                                                   'score_subblock_4_Sess13h00','score_subblock_4_Sess17h00','score_subblock_4_Sess21h00','score_subblock_5_Sess09h00',
                                                   'score_subblock_5_Sess13h00','score_subblock_5_Sess17h00','score_subblock_5_Sess21h00','score_subblock_6_Sess09h00',
                                                   'score_subblock_6_Sess13h00','score_subblock_6_Sess17h00','score_subblock_6_Sess21h00','Tipo_Sess09h00','Tipo_Sess13h00',
                                                   'Tipo_Sess17h00','Tipo_Sess21h00','total_correct_Sess09h00','total_correct_Sess13h00','total_correct_Sess17h00',
                                                   'total_correct_Sess21h00','total_response_time_Sess09h00','total_response_time_Sess13h00','total_response_time_Sess17h00',
                                                   'total_response_time_Sess21h00','total_responses_Sess09h00','total_responses_Sess13h00','total_responses_Sess17h00',
                                                   'total_responses_Sess21h00','width_Sess09h00','width_Sess13h00','width_Sess17h00','width_Sess21h00']]

############################################################################################
############################################################################################
#'toUpdate1_1', 'toUpdate1_2', 'toUpdate1_3', 'toUpdate2_1', 'toUpdate2_2','toUpdate2_3','correct_response1', 'correct_response2', 'correct_response3',
#'selSNr',
df_WMU_Task_1 = df_WMU_Task_1[
    ['subject_nr',  'CB_ref', 'practice', 'TrialNumber', 'correct1', 'correct2', 'correct3', 'digit1', 'digit2', 'digit3', 'height',
     'Index_List', 'live_row', 'logfile', 'response1', 'response2', 'response3', 'response_time1', 'responseavgRT',
     'total_correct_trial', 'TotalRtBlock',  'WMUExperimentalScore', 'WMUPracticeScore', 'width']]
df_WMU_Task_1 = df_WMU_Task_1.rename(columns={'response_time1': 'response_time'})
#df_WMU_Task_1 = df_WMU_Task_1.sort_values(by=['selSNr'], kind='mergesort')
#df_WMU_Task_1 = df_WMU_Task_1.sort_values(by=['subject_nr'], kind='mergesort')
for i in range(0, len(df_WMU_Task_1['responseavgRT'])):
    if df_WMU_Task_1['responseavgRT'].iloc[i] == 0:
        df_WMU_Task_1['responseavgRT'].iloc[i] = ''
df_WMU_Task_1.columns = ['subject_nr','CB_ref','practice','TrialNumber','correct1_Sess09h00',
                         'correct2_Sess09h00','correct3_Sess09h00','digit1_Sess09h00','digit2_Sess09h00','digit3_Sess09h00','height_Sess09h00',
                         'Index_List_Sess09h00','live_row_Sess09h00','logfile_Sess09h00','response1_Sess09h00','response2_Sess09h00','response3_Sess09h00',
                         'response_time1_Sess09h00','responseavgRT_Sess09h00','total_correct_trial_Sess09h00','TotalRtBlock_Sess09h00',
                         'WMUExperimentalScore_Sess09h00','WMUPracticeScore_Sess09h00','width_Sess09h00']

#WMU_cast_lis = ['toUpdate1_1', 'toUpdate1_2', 'toUpdate1_3', 'toUpdate2_1', 'toUpdate2_2', 'toUpdate2_3']
#for i in WMU_cast_lis:
#    for j in range(0, len(df_WMU_Task[i])):
#        if df_WMU_Task[i].iloc[j] > 0:
#            df_WMU_Task[i].iloc[j] = '+' + str(int(df_WMU_Task[i].iloc[j]))
#        else:
#            df_WMU_Task[i].iloc[j] = str(int(df_WMU_Task[i].iloc[j]))

#'selSNr',
df_WMU_Task_2 = df_WMU_Task_2[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'correct1', 'correct2', 'correct3', 'digit1', 'digit2', 'digit3', 'height',
     'Index_List', 'live_row', 'logfile', 'response1', 'response2', 'response3', 'response_time1', 'responseavgRT',
     'total_correct_trial', 'TotalRtBlock',  'WMUExperimentalScore', 'WMUPracticeScore', 'width']]
df_WMU_Task_2 = df_WMU_Task_2.rename(columns={'response_time1': 'response_time'})
#df_WMU_Task_2 = df_WMU_Task_2.sort_values(by=['selSNr'], kind='mergesort')
#df_WMU_Task_2 = df_WMU_Task_2.sort_values(by=['subject_nr'], kind='mergesort')
for i in range(0, len(df_WMU_Task_2['responseavgRT'])):
    if df_WMU_Task_2['responseavgRT'].iloc[i] == 0:
        df_WMU_Task_2['responseavgRT'].iloc[i] = ''
df_WMU_Task_2.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_WMU_Task_2.columns = ['correct1_Sess13h00','correct2_Sess13h00','correct3_Sess13h00','digit1_Sess13h00','digit2_Sess13h00','digit3_Sess13h00','height_Sess13h00',
                         'Index_List_Sess13h00','live_row_Sess13h00','logfile_Sess13h00','response1_Sess13h00','response2_Sess13h00','response3_Sess13h00',
                         'response_time1_Sess13h00','responseavgRT_Sess13h00','total_correct_trial_Sess13h00','TotalRtBlock_Sess13h00',
                         'WMUExperimentalScore_Sess13h00','WMUPracticeScore_Sess13h00','width_Sess13h00']
#WMU_cast_lis = ['toUpdate1_1', 'toUpdate1_2', 'toUpdate1_3', 'toUpdate2_1', 'toUpdate2_2', 'toUpdate2_3']
#for i in WMU_cast_lis:
#    for j in range(0, len(df_WMU_Task[i])):
#        if df_WMU_Task[i].iloc[j] > 0:
#            df_WMU_Task[i].iloc[j] = '+' + str(int(df_WMU_Task[i].iloc[j]))
#        else:
#            df_WMU_Task[i].iloc[j] = str(int(df_WMU_Task[i].iloc[j]))

#'selSNr',
df_WMU_Task_3 = df_WMU_Task_3[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'correct1', 'correct2', 'correct3', 'digit1', 'digit2', 'digit3', 'height',
     'Index_List', 'live_row', 'logfile', 'response1', 'response2', 'response3', 'response_time1', 'responseavgRT',
     'total_correct_trial', 'TotalRtBlock',  'WMUExperimentalScore', 'WMUPracticeScore', 'width']]
df_WMU_Task_3 = df_WMU_Task_3.rename(columns={'response_time1': 'response_time'})
#df_WMU_Task_3 = df_WMU_Task_3.sort_values(by=['selSNr'], kind='mergesort')
#df_WMU_Task_3 = df_WMU_Task_3.sort_values(by=['subject_nr'], kind='mergesort')
for i in range(0, len(df_WMU_Task_3['responseavgRT'])):
    if df_WMU_Task_3['responseavgRT'].iloc[i] == 0:
        df_WMU_Task_3['responseavgRT'].iloc[i] = ''
df_WMU_Task_3.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_WMU_Task_3.columns = ['correct1_Sess17h00','correct2_Sess17h00','correct3_Sess17h00','digit1_Sess17h00','digit2_Sess17h00','digit3_Sess17h00','height_Sess17h00',
                         'Index_List_Sess17h00','live_row_Sess17h00','logfile_Sess17h00','response1_Sess17h00','response2_Sess17h00','response3_Sess17h00',
                         'response_time1_Sess17h00','responseavgRT_Sess17h00','total_correct_trial_Sess17h00','TotalRtBlock_Sess17h00',
                         'WMUExperimentalScore_Sess17h00','WMUPracticeScore_Sess17h00','width_Sess17h00']
#WMU_cast_lis = ['toUpdate1_1', 'toUpdate1_2', 'toUpdate1_3', 'toUpdate2_1', 'toUpdate2_2', 'toUpdate2_3']
#for i in WMU_cast_lis:
#    for j in range(0, len(df_WMU_Task[i])):
#        if df_WMU_Task[i].iloc[j] > 0:
#            df_WMU_Task[i].iloc[j] = '+' + str(int(df_WMU_Task[i].iloc[j]))
#        else:
#            df_WMU_Task[i].iloc[j] = str(int(df_WMU_Task[i].iloc[j]))

#'selSNr',
df_WMU_Task_4 = df_WMU_Task_4[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'correct1', 'correct2', 'correct3', 'digit1', 'digit2', 'digit3', 'height',
     'Index_List', 'live_row', 'logfile', 'response1', 'response2', 'response3', 'response_time1', 'responseavgRT',
     'total_correct_trial', 'TotalRtBlock',  'WMUExperimentalScore', 'WMUPracticeScore', 'width']]
df_WMU_Task_4 = df_WMU_Task_4.rename(columns={'response_time1': 'response_time'})
#df_WMU_Task_4 = df_WMU_Task_4.sort_values(by=['selSNr'], kind='mergesort')
#df_WMU_Task_4 = df_WMU_Task_4.sort_values(by=['subject_nr'], kind='mergesort')
for i in range(0, len(df_WMU_Task_4['responseavgRT'])):
    if df_WMU_Task_4['responseavgRT'].iloc[i] == 0:
        df_WMU_Task_4['responseavgRT'].iloc[i] = ''
df_WMU_Task_4.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_WMU_Task_4.columns = ['correct1_Sess21h00','correct2_Sess21h00','correct3_Sess21h00','digit1_Sess21h00','digit2_Sess21h00','digit3_Sess21h00','height_Sess21h00',
                         'Index_List_Sess21h00','live_row_Sess21h00','logfile_Sess21h00','response1_Sess21h00','response2_Sess21h00','response3_Sess21h00',
                         'response_time1_Sess21h00','responseavgRT_Sess21h00','total_correct_trial_Sess21h00','TotalRtBlock_Sess21h00',
                         'WMUExperimentalScore_Sess21h00','WMUPracticeScore_Sess21h00','width_Sess21h00']
#WMU_cast_lis = ['toUpdate1_1', 'toUpdate1_2', 'toUpdate1_3', 'toUpdate2_1', 'toUpdate2_2', 'toUpdate2_3']
#for i in WMU_cast_lis:
#    for j in range(0, len(df_WMU_Task[i])):
#        if df_WMU_Task[i].iloc[j] > 0:
#            df_WMU_Task[i].iloc[j] = '+' + str(int(df_WMU_Task[i].iloc[j]))
#        else:
#            df_WMU_Task[i].iloc[j] = str(int(df_WMU_Task[i].iloc[j]))

df_WMU_Task_Experimental = pd.concat([df_WMU_Task_1,df_WMU_Task_2,df_WMU_Task_3,df_WMU_Task_4],axis=1)

df_WMU_Task_Experimental = df_WMU_Task_Experimental[['subject_nr','CB_ref','practice','TrialNumber','correct1_Sess09h00','correct1_Sess13h00',
                                       'correct1_Sess17h00','correct1_Sess21h00','correct2_Sess09h00','correct2_Sess13h00','correct2_Sess17h00',
                                       'correct2_Sess21h00','correct3_Sess09h00','correct3_Sess13h00','correct3_Sess17h00','correct3_Sess21h00',
                                       'digit1_Sess09h00','digit1_Sess13h00','digit1_Sess17h00','digit1_Sess21h00','digit2_Sess09h00','digit2_Sess13h00',
                                       'digit2_Sess17h00','digit2_Sess21h00','digit3_Sess09h00','digit3_Sess13h00','digit3_Sess17h00','digit3_Sess21h00',
                                       'height_Sess09h00','height_Sess13h00','height_Sess17h00','height_Sess21h00','Index_List_Sess09h00',
                                       'Index_List_Sess13h00','Index_List_Sess17h00','Index_List_Sess21h00','live_row_Sess09h00','live_row_Sess13h00',
                                       'live_row_Sess17h00','live_row_Sess21h00','logfile_Sess09h00','logfile_Sess13h00','logfile_Sess17h00','logfile_Sess21h00',
                                       'response1_Sess09h00','response1_Sess13h00','response1_Sess17h00','response1_Sess21h00','response2_Sess09h00','response2_Sess13h00',
                                       'response2_Sess17h00','response2_Sess21h00','response3_Sess09h00','response3_Sess13h00','response3_Sess17h00','response3_Sess21h00',
                                       'response_time1_Sess09h00','response_time1_Sess13h00','response_time1_Sess17h00','response_time1_Sess21h00',
                                       'responseavgRT_Sess09h00','responseavgRT_Sess13h00','responseavgRT_Sess17h00','responseavgRT_Sess21h00',
                                       'total_correct_trial_Sess09h00','total_correct_trial_Sess13h00','total_correct_trial_Sess17h00','total_correct_trial_Sess21h00',
                                       'TotalRtBlock_Sess09h00','TotalRtBlock_Sess13h00','TotalRtBlock_Sess17h00','TotalRtBlock_Sess21h00',
                                       'WMUExperimentalScore_Sess09h00', 'WMUExperimentalScore_Sess13h00','WMUExperimentalScore_Sess17h00', 'WMUExperimentalScore_Sess21h00',
                                       'WMUPracticeScore_Sess09h00','WMUPracticeScore_Sess13h00','WMUPracticeScore_Sess17h00','WMUPracticeScore_Sess21h00',
                                       'width_Sess09h00','width_Sess13h00','width_Sess17h00','width_Sess21h00']]
############################################################################################
############################################################################################
#'selSNr',
df_Symmetry_Span_1 = df_Symmetry_Span_1[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName',
     'aggregated_score_memory', 'average_response_time_processing', 'average_total_time_memory', 'correct',
     'correct_response', 'countDys', 'countSym', 'height', 'LeftHalfPos', 'List_SS_button', 'List_SS_Pos', 'live_row',
     'logfile', 'maxDys', 'maxSym', 'pressed_buttons', 'response_memory', 'response_processing', 'response_time_memory',
     'response_time_processing', 'response_total_time_memory', 'response_total_time_memory_full_task', 'RightHalfPos',
     'SP_part_process_time', 'SS_practice_score', 'score_symmetry_span', 'score_subblock_2', 'score_subblock_3',
     'score_subblock_4', 'score_subblock_5', 'score_subblock_6', 'SymType',
     'total_correct_processing', 'total_response_time_processing', 'width']]
df_Symmetry_Span_1 = df_Symmetry_Span_1.astype({'correct_response': 'str', 'response_processing': 'str'})
example_d = df_Symmetry_Span_1["response_processing"].iloc[2]
df_Symmetry_Span_1 = df_Symmetry_Span_1.replace(example_d, '')
#df_Symmetry_Span_1 = df_Symmetry_Span_1.sort_values(by=['selSNr'], kind='mergesort')
#df_Symmetry_Span_1 = df_Symmetry_Span_1.sort_values(by=['subject_nr'], kind='mergesort')
df_Symmetry_Span_1.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                              'SubTaskName','aggregated_score_memory_Sess09h00','average_response_time_processing_Sess09h00',
                              'average_total_time_memory_Sess09h00','correct_Sess09h00','correct_response_Sess09h00','countDys_Sess09h00',
                              'countSym_Sess09h00','height_Sess09h00','LeftHalfPos_Sess09h00','List_SS_button_Sess09h00','List_SS_Pos_Sess09h00',
                              'live_row_Sess09h00','logfile_Sess09h00','maxDys_Sess09h00','maxSym_Sess09h00','pressed_buttons_Sess09h00',
                              'response_memory_Sess09h00','response_processing_Sess09h00','response_time_memory_Sess09h00',
                              'response_time_processing_Sess09h00','response_total_time_memory_Sess09h00',
                              'response_total_time_memory_full_task_Sess09h00','RightHalfPos_Sess09h00','SP_part_process_time_Sess09h00',
                              'SS_practice_score_Sess09h00','score_symmetry_span_Sess09h00','score_subblock_2_Sess09h00','score_subblock_3_Sess09h00',
                              'score_subblock_4_Sess09h00','score_subblock_5_Sess09h00','score_subblock_6_Sess09h00','SymType_Sess09h00',
                              'total_correct_processing_Sess09h00','total_response_time_processing_Sess09h00','width_Sess09h00']

#'selSNr',
df_Symmetry_Span_2 = df_Symmetry_Span_2[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName',
     'aggregated_score_memory', 'average_response_time_processing', 'average_total_time_memory', 'correct',
     'correct_response', 'countDys', 'countSym', 'height', 'LeftHalfPos', 'List_SS_button', 'List_SS_Pos', 'live_row',
     'logfile', 'maxDys', 'maxSym', 'pressed_buttons', 'response_memory', 'response_processing', 'response_time_memory',
     'response_time_processing', 'response_total_time_memory', 'response_total_time_memory_full_task', 'RightHalfPos',
     'SP_part_process_time', 'SS_practice_score', 'score_symmetry_span', 'score_subblock_2', 'score_subblock_3',
     'score_subblock_4', 'score_subblock_5', 'score_subblock_6', 'SymType',
     'total_correct_processing', 'total_response_time_processing', 'width']]
df_Symmetry_Span_2 = df_Symmetry_Span_2.astype({'correct_response': 'str', 'response_processing': 'str'})
example_d = df_Symmetry_Span_2["response_processing"].iloc[2]
df_Symmetry_Span_2 = df_Symmetry_Span_2.replace(example_d, '')
#df_Symmetry_Span_2 = df_Symmetry_Span_2.sort_values(by=['selSNr'], kind='mergesort')
#df_Symmetry_Span_2 = df_Symmetry_Span_2.sort_values(by=['subject_nr'], kind='mergesort')
df_Symmetry_Span_2.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Symmetry_Span_2.columns = ['aggregated_score_memory_Sess13h00','average_response_time_processing_Sess13h00',
                              'average_total_time_memory_Sess13h00','correct_Sess13h00','correct_response_Sess13h00','countDys_Sess13h00',
                              'countSym_Sess13h00','height_Sess13h00','LeftHalfPos_Sess13h00','List_SS_button_Sess13h00','List_SS_Pos_Sess13h00',
                              'live_row_Sess13h00','logfile_Sess13h00','maxDys_Sess13h00','maxSym_Sess13h00','pressed_buttons_Sess13h00',
                              'response_memory_Sess13h00','response_processing_Sess13h00','response_time_memory_Sess13h00',
                              'response_time_processing_Sess13h00','response_total_time_memory_Sess13h00',
                              'response_total_time_memory_full_task_Sess13h00','RightHalfPos_Sess13h00','SP_part_process_time_Sess13h00',
                              'SS_practice_score_Sess13h00','score_symmetry_span_Sess13h00','score_subblock_2_Sess13h00','score_subblock_3_Sess13h00',
                              'score_subblock_4_Sess13h00','score_subblock_5_Sess13h00','score_subblock_6_Sess13h00','SymType_Sess13h00',
                              'total_correct_processing_Sess13h00','total_response_time_processing_Sess13h00','width_Sess13h00']

#'selSNr',
df_Symmetry_Span_3 = df_Symmetry_Span_3[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName',
     'aggregated_score_memory', 'average_response_time_processing', 'average_total_time_memory', 'correct',
     'correct_response', 'countDys', 'countSym', 'height', 'LeftHalfPos', 'List_SS_button', 'List_SS_Pos', 'live_row',
     'logfile', 'maxDys', 'maxSym', 'pressed_buttons', 'response_memory', 'response_processing', 'response_time_memory',
     'response_time_processing', 'response_total_time_memory', 'response_total_time_memory_full_task', 'RightHalfPos',
     'SP_part_process_time', 'SS_practice_score', 'score_symmetry_span', 'score_subblock_2', 'score_subblock_3',
     'score_subblock_4', 'score_subblock_5', 'score_subblock_6', 'SymType',
     'total_correct_processing', 'total_response_time_processing', 'width']]
df_Symmetry_Span_3 = df_Symmetry_Span_3.astype({'correct_response': 'str', 'response_processing': 'str'})
example_d = df_Symmetry_Span_3["response_processing"].iloc[2]
df_Symmetry_Span_3 = df_Symmetry_Span_3.replace(example_d, '')
#df_Symmetry_Span_3 = df_Symmetry_Span_3.sort_values(by=['selSNr'], kind='mergesort')
#df_Symmetry_Span_3 = df_Symmetry_Span_3.sort_values(by=['subject_nr'], kind='mergesort')
df_Symmetry_Span_3.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Symmetry_Span_3.columns = ['aggregated_score_memory_Sess17h00','average_response_time_processing_Sess17h00',
                              'average_total_time_memory_Sess17h00','correct_Sess17h00','correct_response_Sess17h00','countDys_Sess17h00',
                              'countSym_Sess17h00','height_Sess17h00','LeftHalfPos_Sess17h00','List_SS_button_Sess17h00','List_SS_Pos_Sess17h00',
                              'live_row_Sess17h00','logfile_Sess17h00','maxDys_Sess17h00','maxSym_Sess17h00','pressed_buttons_Sess17h00',
                              'response_memory_Sess17h00','response_processing_Sess17h00','response_time_memory_Sess17h00',
                              'response_time_processing_Sess17h00','response_total_time_memory_Sess17h00',
                              'response_total_time_memory_full_task_Sess17h00','RightHalfPos_Sess17h00','SP_part_process_time_Sess17h00',
                              'SS_practice_score_Sess17h00','score_symmetry_span_Sess17h00','score_subblock_2_Sess17h00','score_subblock_3_Sess17h00',
                              'score_subblock_4_Sess17h00','score_subblock_5_Sess17h00','score_subblock_6_Sess17h00','SymType_Sess17h00',
                              'total_correct_processing_Sess17h00','total_response_time_processing_Sess17h00','width_Sess17h00']

#'selSNr',
df_Symmetry_Span_4 = df_Symmetry_Span_4[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName',
     'aggregated_score_memory', 'average_response_time_processing', 'average_total_time_memory', 'correct',
     'correct_response', 'countDys', 'countSym', 'height', 'LeftHalfPos', 'List_SS_button', 'List_SS_Pos', 'live_row',
     'logfile', 'maxDys', 'maxSym', 'pressed_buttons', 'response_memory', 'response_processing', 'response_time_memory',
     'response_time_processing', 'response_total_time_memory', 'response_total_time_memory_full_task', 'RightHalfPos',
     'SP_part_process_time', 'SS_practice_score', 'score_symmetry_span', 'score_subblock_2', 'score_subblock_3',
     'score_subblock_4', 'score_subblock_5', 'score_subblock_6', 'SymType',
     'total_correct_processing', 'total_response_time_processing', 'width']]
df_Symmetry_Span_4 = df_Symmetry_Span_4.astype({'correct_response': 'str', 'response_processing': 'str'})
example_d = df_Symmetry_Span_4["response_processing"].iloc[2]
df_Symmetry_Span_4 = df_Symmetry_Span_4.replace(example_d, '')
#df_Symmetry_Span_4 = df_Symmetry_Span_4.sort_values(by=['selSNr'], kind='mergesort')
#df_Symmetry_Span_4 = df_Symmetry_Span_4.sort_values(by=['subject_nr'], kind='mergesort')
df_Symmetry_Span_4.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Symmetry_Span_4.columns = ['aggregated_score_memory_Sess21h00','average_response_time_processing_Sess21h00',
                              'average_total_time_memory_Sess21h00','correct_Sess21h00','correct_response_Sess21h00','countDys_Sess21h00',
                              'countSym_Sess21h00','height_Sess21h00','LeftHalfPos_Sess21h00','List_SS_button_Sess21h00','List_SS_Pos_Sess21h00',
                              'live_row_Sess21h00','logfile_Sess21h00','maxDys_Sess21h00','maxSym_Sess21h00','pressed_buttons_Sess21h00',
                              'response_memory_Sess21h00','response_processing_Sess21h00','response_time_memory_Sess21h00',
                              'response_time_processing_Sess21h00','response_total_time_memory_Sess21h00',
                              'response_total_time_memory_full_task_Sess21h00','RightHalfPos_Sess21h00','SP_part_process_time_Sess21h00',
                              'SS_practice_score_Sess21h00','score_symmetry_span_Sess21h00','score_subblock_2_Sess21h00','score_subblock_3_Sess21h00',
                              'score_subblock_4_Sess21h00','score_subblock_5_Sess21h00','score_subblock_6_Sess21h00','SymType_Sess21h00',
                              'total_correct_processing_Sess21h00','total_response_time_processing_Sess21h00','width_Sess21h00']

df_Symmetry_Span_Experimental = pd.concat([df_Symmetry_Span_1,df_Symmetry_Span_2,df_Symmetry_Span_3,df_Symmetry_Span_4],axis=1)

df_Symmetry_Span_Experimental = df_Symmetry_Span_Experimental[['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName',
                                                 'aggregated_score_memory_Sess09h00','aggregated_score_memory_Sess13h00',
                                                 'aggregated_score_memory_Sess17h00','aggregated_score_memory_Sess21h00',
                                                 'average_response_time_processing_Sess09h00','average_response_time_processing_Sess13h00',
                                                 'average_response_time_processing_Sess17h00','average_response_time_processing_Sess21h00',
                                                 'average_total_time_memory_Sess09h00','average_total_time_memory_Sess13h00',
                                                 'average_total_time_memory_Sess17h00','average_total_time_memory_Sess21h00',
                                                 'correct_Sess09h00','correct_Sess13h00','correct_Sess17h00','correct_Sess21h00',
                                                 'correct_response_Sess09h00','correct_response_Sess13h00','correct_response_Sess17h00',
                                                 'correct_response_Sess21h00','countDys_Sess09h00','countDys_Sess13h00','countDys_Sess17h00','countDys_Sess21h00',
                                                 'countSym_Sess09h00','countSym_Sess13h00','countSym_Sess17h00','countSym_Sess21h00','height_Sess09h00','height_Sess13h00','height_Sess17h00','height_Sess21h00',
                                                 'LeftHalfPos_Sess09h00','LeftHalfPos_Sess13h00','LeftHalfPos_Sess17h00','LeftHalfPos_Sess21h00',
                                                 'List_SS_button_Sess09h00','List_SS_button_Sess13h00','List_SS_button_Sess17h00','List_SS_button_Sess21h00',
                                                 'List_SS_Pos_Sess09h00','List_SS_Pos_Sess13h00','List_SS_Pos_Sess17h00','List_SS_Pos_Sess21h00','live_row_Sess09h00',
                                                 'live_row_Sess13h00','live_row_Sess17h00','live_row_Sess21h00','logfile_Sess09h00','logfile_Sess13h00','logfile_Sess17h00',
                                                 'logfile_Sess21h00','maxDys_Sess09h00','maxDys_Sess13h00','maxDys_Sess17h00','maxDys_Sess21h00','maxSym_Sess09h00',
                                                 'maxSym_Sess13h00','maxSym_Sess17h00','maxSym_Sess21h00','pressed_buttons_Sess09h00','pressed_buttons_Sess13h00',
                                                 'pressed_buttons_Sess17h00','pressed_buttons_Sess21h00','response_memory_Sess09h00',
                                                 'response_memory_Sess13h00','response_memory_Sess17h00','response_memory_Sess21h00',
                                                 'response_processing_Sess09h00','response_processing_Sess13h00','response_processing_Sess17h00',
                                                 'response_processing_Sess21h00','response_time_memory_Sess09h00','response_time_memory_Sess13h00','response_time_memory_Sess17h00',
                                                 'response_time_memory_Sess21h00','response_time_processing_Sess09h00','response_time_processing_Sess13h00',
                                                 'response_time_processing_Sess17h00','response_time_processing_Sess21h00','response_total_time_memory_Sess09h00',
                                                 'response_total_time_memory_Sess13h00','response_total_time_memory_Sess17h00','response_total_time_memory_Sess21h00',
                                                 'response_total_time_memory_full_task_Sess09h00','response_total_time_memory_full_task_Sess13h00',
                                                 'response_total_time_memory_full_task_Sess17h00','response_total_time_memory_full_task_Sess21h00',
                                                 'RightHalfPos_Sess09h00','RightHalfPos_Sess13h00','RightHalfPos_Sess17h00','RightHalfPos_Sess21h00',
                                                 'SP_part_process_time_Sess09h00','SP_part_process_time_Sess13h00','SP_part_process_time_Sess17h00',
                                                 'SP_part_process_time_Sess21h00','SS_practice_score_Sess09h00','SS_practice_score_Sess13h00','SS_practice_score_Sess17h00',
                                                 'SS_practice_score_Sess21h00','score_symmetry_span_Sess09h00','score_symmetry_span_Sess13h00',
                                                 'score_symmetry_span_Sess17h00','score_symmetry_span_Sess21h00','score_subblock_2_Sess09h00','score_subblock_2_Sess13h00',
                                                 'score_subblock_2_Sess17h00','score_subblock_2_Sess21h00','score_subblock_3_Sess09h00','score_subblock_3_Sess13h00',
                                                 'score_subblock_3_Sess17h00','score_subblock_3_Sess21h00','score_subblock_4_Sess09h00','score_subblock_4_Sess13h00',
                                                 'score_subblock_4_Sess17h00','score_subblock_4_Sess21h00','score_subblock_5_Sess09h00','score_subblock_5_Sess13h00',
                                                 'score_subblock_5_Sess17h00','score_subblock_5_Sess21h00','score_subblock_6_Sess09h00','score_subblock_6_Sess13h00',
                                                 'score_subblock_6_Sess17h00','score_subblock_6_Sess21h00','SymType_Sess09h00','SymType_Sess13h00','SymType_Sess17h00',
                                                 'SymType_Sess21h00','total_correct_processing_Sess09h00','total_correct_processing_Sess13h00','total_correct_processing_Sess17h00',
                                                 'total_correct_processing_Sess21h00','total_response_time_processing_Sess09h00','total_response_time_processing_Sess13h00',
                                                 'total_response_time_processing_Sess17h00','total_response_time_processing_Sess21h00','width_Sess09h00','width_Sess13h00',
                                                 'width_Sess17h00','width_Sess21h00']]

############################################################################################
############################################################################################
#'Probe', 'Target',
#'selSNr',
df_Binding_Task_1 = df_Binding_Task_1[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'acc', 'average_response_time',
     'BindingRawScore', 'correct', 'correct_response', 'counter', 'Delay', 'eightsec_accuracy', 'FalseAlarms', 'height',
     'Hits', 'live_row', 'logfile', 'match_1s_accuracy', 'match_1s_avg_rt', 'match_8s_accuracy', 'match_8s_avg_rt',
     'mismatch_1s_accuracy', 'mismatch_1s_avg_rt', 'mismatch_8s_accuracy', 'mismatch_8s_avg_rt', 'NNonResponses',
     'Omissions', 'onesec_accuracy', 'QuinetteAccuracyScore', 'QuinetteProcessingScore', 'response',
     'response_time', 'ResponsesGiven', 'total_correct', 'total_match_1s_rt', 'total_match_8s_rt',
     'total_mismatch_1s_rt', 'total_mismatch_8s_rt', 'total_response_time', 'total_responses', 'width']]
df_Binding_Task_1[['acc', 'average_response_time']] = df_Binding_Task_1[['acc', 'average_response_time']].replace(',', '.')
df_Binding_Task_1 = df_Binding_Task_1.astype(
    {'acc': 'float64', 'average_response_time': 'float64', 'correct_response': 'str', 'response': 'str'})
#df_Binding_Task_1 = df_Binding_Task_1.sort_values(by=['selSNr'], kind='mergesort')
#df_Binding_Task_1 = df_Binding_Task_1.sort_values(by=['subject_nr'], kind='mergesort')
df_Binding_Task_1.columns = ['subject_nr','CB_ref','practice','TrialNumber','acc_Sess09h00',
                             'average_response_time_Sess09h00','BindingRawScore_Sess09h00','correct_Sess09h00','correct_response_Sess09h00',
                             'counter_Sess09h00','Delay_Sess09h00','eightsec_accuracy_Sess09h00','FalseAlarms_Sess09h00','height_Sess09h00','Hits_Sess09h00',
                             'live_row_Sess09h00','logfile_Sess09h00','match_1s_accuracy_Sess09h00','match_1s_avg_rt_Sess09h00','match_8s_accuracy_Sess09h00',
                             'match_8s_avg_rt_Sess09h00','mismatch_1s_accuracy_Sess09h00','mismatch_1s_avg_rt_Sess09h00','mismatch_8s_accuracy_Sess09h00',
                             'mismatch_8s_avg_rt_Sess09h00','NNonResponses_Sess09h00','Omissions_Sess09h00','onesec_accuracy_Sess09h00',
                             'QuinetteAccuracyScore_Sess09h00','QuinetteProcessingScore_Sess09h00','response_Sess09h00','response_time_Sess09h00',
                             'ResponsesGiven_Sess09h00','total_correct_Sess09h00','total_match_1s_rt_Sess09h00','total_match_8s_rt_Sess09h00',
                             'total_mismatch_1s_rt_Sess09h00','total_mismatch_8s_rt_Sess09h00','total_response_time_Sess09h00','total_responses_Sess09h00',
                             'width_Sess09h00']

#'selSNr',
df_Binding_Task_2 = df_Binding_Task_2[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'acc', 'average_response_time',
     'BindingRawScore', 'correct', 'correct_response', 'counter', 'Delay', 'eightsec_accuracy', 'FalseAlarms', 'height',
     'Hits', 'live_row', 'logfile', 'match_1s_accuracy', 'match_1s_avg_rt', 'match_8s_accuracy', 'match_8s_avg_rt',
     'mismatch_1s_accuracy', 'mismatch_1s_avg_rt', 'mismatch_8s_accuracy', 'mismatch_8s_avg_rt', 'NNonResponses',
     'Omissions', 'onesec_accuracy', 'QuinetteAccuracyScore', 'QuinetteProcessingScore', 'response',
     'response_time', 'ResponsesGiven', 'total_correct', 'total_match_1s_rt', 'total_match_8s_rt',
     'total_mismatch_1s_rt', 'total_mismatch_8s_rt', 'total_response_time', 'total_responses', 'width']]
df_Binding_Task_2[['acc', 'average_response_time']] = df_Binding_Task_2[['acc', 'average_response_time']].replace(',', '.')
df_Binding_Task_2 = df_Binding_Task_2.astype(
    {'acc': 'float64', 'average_response_time': 'float64', 'correct_response': 'str', 'response': 'str'})
#df_Binding_Task_2 = df_Binding_Task_2.sort_values(by=['selSNr'], kind='mergesort')
#df_Binding_Task_2 = df_Binding_Task_2.sort_values(by=['subject_nr'], kind='mergesort')
df_Binding_Task_2.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_Binding_Task_2.columns = ['acc_Sess13h00','average_response_time_Sess13h00','BindingRawScore_Sess13h00','correct_Sess13h00','correct_response_Sess13h00',
                             'counter_Sess13h00','Delay_Sess13h00','eightsec_accuracy_Sess13h00','FalseAlarms_Sess13h00','height_Sess13h00','Hits_Sess13h00',
                             'live_row_Sess13h00','logfile_Sess13h00','match_1s_accuracy_Sess13h00','match_1s_avg_rt_Sess13h00','match_8s_accuracy_Sess13h00',
                             'match_8s_avg_rt_Sess13h00','mismatch_1s_accuracy_Sess13h00','mismatch_1s_avg_rt_Sess13h00','mismatch_8s_accuracy_Sess13h00',
                             'mismatch_8s_avg_rt_Sess13h00','NNonResponses_Sess13h00','Omissions_Sess13h00','onesec_accuracy_Sess13h00',
                             'QuinetteAccuracyScore_Sess13h00','QuinetteProcessingScore_Sess13h00','response_Sess13h00','response_time_Sess13h00',
                             'ResponsesGiven_Sess13h00','total_correct_Sess13h00','total_match_1s_rt_Sess13h00','total_match_8s_rt_Sess13h00',
                             'total_mismatch_1s_rt_Sess13h00','total_mismatch_8s_rt_Sess13h00','total_response_time_Sess13h00','total_responses_Sess13h00',
                             'width_Sess13h00']

#'selSNr',
df_Binding_Task_3 = df_Binding_Task_3[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'acc', 'average_response_time',
     'BindingRawScore', 'correct', 'correct_response', 'counter', 'Delay', 'eightsec_accuracy', 'FalseAlarms', 'height',
     'Hits', 'live_row', 'logfile', 'match_1s_accuracy', 'match_1s_avg_rt', 'match_8s_accuracy', 'match_8s_avg_rt',
     'mismatch_1s_accuracy', 'mismatch_1s_avg_rt', 'mismatch_8s_accuracy', 'mismatch_8s_avg_rt', 'NNonResponses',
     'Omissions', 'onesec_accuracy', 'QuinetteAccuracyScore', 'QuinetteProcessingScore', 'response',
     'response_time', 'ResponsesGiven', 'total_correct', 'total_match_1s_rt', 'total_match_8s_rt',
     'total_mismatch_1s_rt', 'total_mismatch_8s_rt', 'total_response_time', 'total_responses', 'width']]
df_Binding_Task_3[['acc', 'average_response_time']] = df_Binding_Task_3[['acc', 'average_response_time']].replace(',', '.')
df_Binding_Task_3 = df_Binding_Task_3.astype(
    {'acc': 'float64', 'average_response_time': 'float64', 'correct_response': 'str', 'response': 'str'})
#df_Binding_Task_3 = df_Binding_Task_3.sort_values(by=['selSNr'], kind='mergesort')
#df_Binding_Task_3 = df_Binding_Task_3.sort_values(by=['subject_nr'], kind='mergesort')
df_Binding_Task_3.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_Binding_Task_3.columns = ['acc_Sess17h00','average_response_time_Sess17h00','BindingRawScore_Sess17h00','correct_Sess17h00','correct_response_Sess17h00',
                             'counter_Sess17h00','Delay_Sess17h00','eightsec_accuracy_Sess17h00','FalseAlarms_Sess17h00','height_Sess17h00','Hits_Sess17h00',
                             'live_row_Sess17h00','logfile_Sess17h00','match_1s_accuracy_Sess17h00','match_1s_avg_rt_Sess17h00','match_8s_accuracy_Sess17h00',
                             'match_8s_avg_rt_Sess17h00','mismatch_1s_accuracy_Sess17h00','mismatch_1s_avg_rt_Sess17h00','mismatch_8s_accuracy_Sess17h00',
                             'mismatch_8s_avg_rt_Sess17h00','NNonResponses_Sess17h00','Omissions_Sess17h00','onesec_accuracy_Sess17h00',
                             'QuinetteAccuracyScore_Sess17h00','QuinetteProcessingScore_Sess17h00','response_Sess17h00','response_time_Sess17h00',
                             'ResponsesGiven_Sess17h00','total_correct_Sess17h00','total_match_1s_rt_Sess17h00','total_match_8s_rt_Sess17h00',
                             'total_mismatch_1s_rt_Sess17h00','total_mismatch_8s_rt_Sess17h00','total_response_time_Sess17h00','total_responses_Sess17h00',
                             'width_Sess17h00']

#'selSNr',
df_Binding_Task_4 = df_Binding_Task_4[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'acc', 'average_response_time',
     'BindingRawScore', 'correct', 'correct_response', 'counter', 'Delay', 'eightsec_accuracy', 'FalseAlarms', 'height',
     'Hits', 'live_row', 'logfile', 'match_1s_accuracy', 'match_1s_avg_rt', 'match_8s_accuracy', 'match_8s_avg_rt',
     'mismatch_1s_accuracy', 'mismatch_1s_avg_rt', 'mismatch_8s_accuracy', 'mismatch_8s_avg_rt', 'NNonResponses',
     'Omissions', 'onesec_accuracy', 'QuinetteAccuracyScore', 'QuinetteProcessingScore', 'response',
     'response_time', 'ResponsesGiven', 'total_correct', 'total_match_1s_rt', 'total_match_8s_rt',
     'total_mismatch_1s_rt', 'total_mismatch_8s_rt', 'total_response_time', 'total_responses', 'width']]
df_Binding_Task_4[['acc', 'average_response_time']] = df_Binding_Task_4[['acc', 'average_response_time']].replace(',', '.')
df_Binding_Task_4 = df_Binding_Task_4.astype(
    {'acc': 'float64', 'average_response_time': 'float64', 'correct_response': 'str', 'response': 'str'})
#df_Binding_Task_4 = df_Binding_Task_4.sort_values(by=['selSNr'], kind='mergesort')
#df_Binding_Task_4 = df_Binding_Task_4.sort_values(by=['subject_nr'], kind='mergesort')
df_Binding_Task_4.drop(['subject_nr','CB_ref','practice','TrialNumber'],axis=1,inplace=True)
df_Binding_Task_4.columns = ['acc_Sess21h00','average_response_time_Sess21h00','BindingRawScore_Sess21h00','correct_Sess21h00','correct_response_Sess21h00',
                             'counter_Sess21h00','Delay_Sess21h00','eightsec_accuracy_Sess21h00','FalseAlarms_Sess21h00','height_Sess21h00','Hits_Sess21h00',
                             'live_row_Sess21h00','logfile_Sess21h00','match_1s_accuracy_Sess21h00','match_1s_avg_rt_Sess21h00','match_8s_accuracy_Sess21h00',
                             'match_8s_avg_rt_Sess21h00','mismatch_1s_accuracy_Sess21h00','mismatch_1s_avg_rt_Sess21h00','mismatch_8s_accuracy_Sess21h00',
                             'mismatch_8s_avg_rt_Sess21h00','NNonResponses_Sess21h00','Omissions_Sess21h00','onesec_accuracy_Sess21h00',
                             'QuinetteAccuracyScore_Sess21h00','QuinetteProcessingScore_Sess21h00','response_Sess21h00','response_time_Sess21h00',
                             'ResponsesGiven_Sess21h00','total_correct_Sess21h00','total_match_1s_rt_Sess21h00','total_match_8s_rt_Sess21h00',
                             'total_mismatch_1s_rt_Sess21h00','total_mismatch_8s_rt_Sess21h00','total_response_time_Sess21h00','total_responses_Sess21h00',
                             'width_Sess21h00']

df_Binding_Task_Experimental = pd.concat([df_Binding_Task_1,df_Binding_Task_2,df_Binding_Task_3,df_Binding_Task_4],axis=1)

df_Binding_Task_Experimental = df_Binding_Task_Experimental[['subject_nr','CB_ref','practice','TrialNumber','acc_Sess09h00','acc_Sess13h00','acc_Sess17h00','acc_Sess21h00',
                                         'average_response_time_Sess09h00','average_response_time_Sess13h00','average_response_time_Sess17h00','average_response_time_Sess21h00',
                                         'BindingRawScore_Sess09h00','BindingRawScore_Sess13h00','BindingRawScore_Sess17h00','BindingRawScore_Sess21h00','correct_Sess09h00',
                                         'correct_Sess13h00','correct_Sess17h00','correct_Sess21h00','correct_response_Sess09h00','correct_response_Sess13h00',
                                         'correct_response_Sess17h00','correct_response_Sess21h00','counter_Sess09h00','counter_Sess13h00','counter_Sess17h00','counter_Sess21h00',
                                         'Delay_Sess09h00','Delay_Sess13h00','Delay_Sess17h00','Delay_Sess21h00','eightsec_accuracy_Sess09h00','eightsec_accuracy_Sess13h00',
                                         'eightsec_accuracy_Sess17h00','eightsec_accuracy_Sess21h00','FalseAlarms_Sess09h00','FalseAlarms_Sess13h00','FalseAlarms_Sess17h00',
                                         'FalseAlarms_Sess21h00','height_Sess09h00','height_Sess13h00','height_Sess17h00','height_Sess21h00','Hits_Sess09h00','Hits_Sess13h00',
                                         'Hits_Sess17h00','Hits_Sess21h00','live_row_Sess09h00','live_row_Sess13h00','live_row_Sess17h00','live_row_Sess21h00','logfile_Sess09h00',
                                         'logfile_Sess13h00','logfile_Sess17h00','logfile_Sess21h00','match_1s_accuracy_Sess09h00','match_1s_accuracy_Sess13h00',
                                         'match_1s_accuracy_Sess17h00','match_1s_accuracy_Sess21h00','match_1s_avg_rt_Sess09h00','match_1s_avg_rt_Sess13h00','match_1s_avg_rt_Sess17h00',
                                         'match_1s_avg_rt_Sess21h00','match_8s_accuracy_Sess09h00','match_8s_accuracy_Sess13h00','match_8s_accuracy_Sess17h00',
                                         'match_8s_accuracy_Sess21h00','match_8s_avg_rt_Sess09h00','match_8s_avg_rt_Sess13h00','match_8s_avg_rt_Sess17h00',
                                         'match_8s_avg_rt_Sess21h00','mismatch_1s_accuracy_Sess09h00','mismatch_1s_accuracy_Sess13h00','mismatch_1s_accuracy_Sess17h00',
                                         'mismatch_1s_accuracy_Sess21h00','mismatch_1s_avg_rt_Sess09h00','mismatch_1s_avg_rt_Sess13h00','mismatch_1s_avg_rt_Sess17h00',
                                         'mismatch_1s_avg_rt_Sess21h00','mismatch_8s_accuracy_Sess09h00','mismatch_8s_accuracy_Sess13h00','mismatch_8s_accuracy_Sess17h00',
                                         'mismatch_8s_accuracy_Sess21h00','mismatch_8s_avg_rt_Sess09h00','mismatch_8s_avg_rt_Sess13h00','mismatch_8s_avg_rt_Sess17h00',
                                         'mismatch_8s_avg_rt_Sess21h00','NNonResponses_Sess09h00','NNonResponses_Sess13h00','NNonResponses_Sess17h00',
                                         'NNonResponses_Sess21h00','Omissions_Sess09h00','Omissions_Sess13h00','Omissions_Sess17h00',
                                         'Omissions_Sess21h00','onesec_accuracy_Sess09h00','onesec_accuracy_Sess13h00','onesec_accuracy_Sess17h00',
                                         'onesec_accuracy_Sess21h00','QuinetteAccuracyScore_Sess09h00','QuinetteAccuracyScore_Sess13h00','QuinetteAccuracyScore_Sess17h00',
                                         'QuinetteAccuracyScore_Sess21h00','QuinetteProcessingScore_Sess09h00','QuinetteProcessingScore_Sess13h00',
                                         'QuinetteProcessingScore_Sess17h00','QuinetteProcessingScore_Sess21h00','response_Sess09h00',
                                         'response_Sess13h00','response_Sess17h00','response_Sess21h00','response_time_Sess09h00','response_time_Sess13h00',
                                         'response_time_Sess17h00','response_time_Sess21h00','ResponsesGiven_Sess09h00','ResponsesGiven_Sess13h00',
                                         'ResponsesGiven_Sess17h00','ResponsesGiven_Sess21h00','total_correct_Sess09h00','total_correct_Sess13h00','total_correct_Sess17h00',
                                         'total_correct_Sess21h00','total_match_1s_rt_Sess09h00','total_match_1s_rt_Sess13h00','total_match_1s_rt_Sess17h00',
                                         'total_match_1s_rt_Sess21h00','total_match_8s_rt_Sess09h00','total_match_8s_rt_Sess13h00','total_match_8s_rt_Sess17h00',
                                         'total_match_8s_rt_Sess21h00','total_mismatch_1s_rt_Sess09h00','total_mismatch_1s_rt_Sess13h00','total_mismatch_1s_rt_Sess17h00',
                                         'total_mismatch_1s_rt_Sess21h00','total_mismatch_8s_rt_Sess09h00','total_mismatch_8s_rt_Sess13h00','total_mismatch_8s_rt_Sess17h00',
                                         'total_mismatch_8s_rt_Sess21h00','total_response_time_Sess09h00','total_response_time_Sess13h00','total_response_time_Sess17h00',
                                         'total_response_time_Sess21h00','total_responses_Sess09h00','total_responses_Sess13h00','total_responses_Sess17h00','total_responses_Sess21h00',
                                         'width_Sess09h00','width_Sess13h00','width_Sess17h00','width_Sess21h00']]

############################################################################################
############################################################################################
#'selSNr',
df_Operation_Span_1 = df_Operation_Span_1[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'height', 'letter', 'List_Prev_Letter', 'List_responses_memory',
     'live_row', 'logfile', 'response_average_time_memory', 'response_memory', 'response_processing',
     'response_time_memory', 'response_time_processing', 'response_total_time_memory', 'OP_part_process_time',
     'score_practice', 'score_operation_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4', 'score_subblock_5',
     'score_subblock_6', 'Tipo', 'total_correct', 'total_response_time',
     'total_responses', 'width']]
df_Operation_Span_1[['acc', 'avg_rt']] = df_Operation_Span_1[['acc', 'avg_rt']].replace(',', '.')
df_Operation_Span_1 = df_Operation_Span_1.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_c = df_Operation_Span_1["response_processing"].iloc[2]
df_Operation_Span_1 = df_Operation_Span_1.replace(example_c, '')
#df_Operation_Span_1 = df_Operation_Span_1.sort_values(by=['selSNr'], kind='mergesort')
#df_Operation_Span_1 = df_Operation_Span_1.sort_values(by=['subject_nr'], kind='mergesort')
df_Operation_Span_1.columns = ['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco',
                               'SubTaskName','acc_Sess09h00','avg_rt_Sess09h00','BlockChoice_Sess09h00','correct_Sess09h00',
                               'correct_response_Sess09h00','height_Sess09h00','letter_Sess09h00','List_Prev_Letter_Sess09h00',
                               'List_responses_memory_Sess09h00','live_row_Sess09h00','logfile_Sess09h00','response_average_time_memory_Sess09h00',
                               'response_memory_Sess09h00','response_processing_Sess09h00','response_time_memory_Sess09h00',
                               'response_time_processing_Sess09h00','response_total_time_memory_Sess09h00','OP_part_process_time_Sess09h00',
                               'score_practice_Sess09h00','score_operation_span_Sess09h00','score_subblock_2_Sess09h00','score_subblock_3_Sess09h00',
                               'score_subblock_4_Sess09h00','score_subblock_5_Sess09h00','score_subblock_6_Sess09h00','Tipo_Sess09h00',
                               'total_correct_Sess09h00','total_response_time_Sess09h00','total_responses_Sess09h00','width_Sess09h00']

#'selSNr',
df_Operation_Span_2 = df_Operation_Span_2[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'height', 'letter', 'List_Prev_Letter', 'List_responses_memory',
     'live_row', 'logfile', 'response_average_time_memory', 'response_memory', 'response_processing',
     'response_time_memory', 'response_time_processing', 'response_total_time_memory', 'OP_part_process_time',
     'score_practice', 'score_operation_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4', 'score_subblock_5',
     'score_subblock_6', 'Tipo', 'total_correct', 'total_response_time',
     'total_responses', 'width']]
df_Operation_Span_2[['acc', 'avg_rt']] = df_Operation_Span_2[['acc', 'avg_rt']].replace(',', '.')
df_Operation_Span_2 = df_Operation_Span_2.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_c = df_Operation_Span_2["response_processing"].iloc[2]
df_Operation_Span_2 = df_Operation_Span_2.replace(example_c, '')
#df_Operation_Span_2 = df_Operation_Span_2.sort_values(by=['selSNr'], kind='mergesort')
#df_Operation_Span_2 = df_Operation_Span_2.sort_values(by=['subject_nr'], kind='mergesort')
df_Operation_Span_2.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Operation_Span_2.columns = ['acc_Sess13h00','avg_rt_Sess13h00','BlockChoice_Sess13h00','correct_Sess13h00',
                               'correct_response_Sess13h00','height_Sess13h00','letter_Sess13h00','List_Prev_Letter_Sess13h00',
                               'List_responses_memory_Sess13h00','live_row_Sess13h00','logfile_Sess13h00','response_average_time_memory_Sess13h00',
                               'response_memory_Sess13h00','response_processing_Sess13h00','response_time_memory_Sess13h00',
                               'response_time_processing_Sess13h00','response_total_time_memory_Sess13h00','OP_part_process_time_Sess13h00',
                               'score_practice_Sess13h00','score_operation_span_Sess13h00','score_subblock_2_Sess13h00','score_subblock_3_Sess13h00',
                               'score_subblock_4_Sess13h00','score_subblock_5_Sess13h00','score_subblock_6_Sess13h00','Tipo_Sess13h00',
                               'total_correct_Sess13h00','total_response_time_Sess13h00','total_responses_Sess13h00','width_Sess13h00']

#'selSNr',
df_Operation_Span_3 = df_Operation_Span_3[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'height', 'letter', 'List_Prev_Letter', 'List_responses_memory',
     'live_row', 'logfile', 'response_average_time_memory', 'response_memory', 'response_processing',
     'response_time_memory', 'response_time_processing', 'response_total_time_memory', 'OP_part_process_time',
     'score_practice', 'score_operation_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4', 'score_subblock_5',
     'score_subblock_6', 'Tipo', 'total_correct', 'total_response_time',
     'total_responses', 'width']]
df_Operation_Span_3[['acc', 'avg_rt']] = df_Operation_Span_3[['acc', 'avg_rt']].replace(',', '.')
df_Operation_Span_3 = df_Operation_Span_3.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_c = df_Operation_Span_3["response_processing"].iloc[2]
df_Operation_Span_3 = df_Operation_Span_3.replace(example_c, '')
#df_Operation_Span_3 = df_Operation_Span_3.sort_values(by=['selSNr'], kind='mergesort')
#df_Operation_Span_3 = df_Operation_Span_3.sort_values(by=['subject_nr'], kind='mergesort')
df_Operation_Span_3.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Operation_Span_3.columns = ['acc_Sess17h00','avg_rt_Sess17h00','BlockChoice_Sess17h00','correct_Sess17h00',
                               'correct_response_Sess17h00','height_Sess17h00','letter_Sess17h00','List_Prev_Letter_Sess17h00',
                               'List_responses_memory_Sess17h00','live_row_Sess17h00','logfile_Sess17h00','response_average_time_memory_Sess17h00',
                               'response_memory_Sess17h00','response_processing_Sess17h00','response_time_memory_Sess17h00',
                               'response_time_processing_Sess17h00','response_total_time_memory_Sess17h00','OP_part_process_time_Sess17h00',
                               'score_practice_Sess17h00','score_operation_span_Sess17h00','score_subblock_2_Sess17h00','score_subblock_3_Sess17h00',
                               'score_subblock_4_Sess17h00','score_subblock_5_Sess17h00','score_subblock_6_Sess17h00','Tipo_Sess17h00',
                               'total_correct_Sess17h00','total_response_time_Sess17h00','total_responses_Sess17h00','width_Sess17h00']

#'selSNr',
df_Operation_Span_4 = df_Operation_Span_4[
    ['subject_nr', 'CB_ref', 'practice', 'TrialNumber', 'Sub_bloco', 'SubTaskName', 'acc', 'avg_rt',
     'BlockChoice', 'correct', 'correct_response', 'height', 'letter', 'List_Prev_Letter', 'List_responses_memory',
     'live_row', 'logfile', 'response_average_time_memory', 'response_memory', 'response_processing',
     'response_time_memory', 'response_time_processing', 'response_total_time_memory', 'OP_part_process_time',
     'score_practice', 'score_operation_span', 'score_subblock_2', 'score_subblock_3', 'score_subblock_4', 'score_subblock_5',
     'score_subblock_6', 'Tipo', 'total_correct', 'total_response_time',
     'total_responses', 'width']]
df_Operation_Span_4[['acc', 'avg_rt']] = df_Operation_Span_4[['acc', 'avg_rt']].replace(',', '.')
df_Operation_Span_4 = df_Operation_Span_4.astype(
    {'acc': 'float64', 'avg_rt': 'float64', 'correct_response': 'str', 'response_processing': 'str'})
example_c = df_Operation_Span_4["response_processing"].iloc[2]
df_Operation_Span_4 = df_Operation_Span_4.replace(example_c, '')
#df_Operation_Span_4 = df_Operation_Span_4.sort_values(by=['selSNr'], kind='mergesort')
#df_Operation_Span_4 = df_Operation_Span_4.sort_values(by=['subject_nr'], kind='mergesort')
df_Operation_Span_4.drop(['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName'],axis=1,inplace=True)
df_Operation_Span_4.columns = ['acc_Sess21h00','avg_rt_Sess21h00','BlockChoice_Sess21h00','correct_Sess21h00',
                               'correct_response_Sess21h00','height_Sess21h00','letter_Sess21h00','List_Prev_Letter_Sess21h00',
                               'List_responses_memory_Sess21h00','live_row_Sess21h00','logfile_Sess21h00','response_average_time_memory_Sess21h00',
                               'response_memory_Sess21h00','response_processing_Sess21h00','response_time_memory_Sess21h00',
                               'response_time_processing_Sess21h00','response_total_time_memory_Sess21h00','OP_part_process_time_Sess21h00',
                               'score_practice_Sess21h00','score_operation_span_Sess21h00','score_subblock_2_Sess21h00','score_subblock_3_Sess21h00',
                               'score_subblock_4_Sess21h00','score_subblock_5_Sess21h00','score_subblock_6_Sess21h00','Tipo_Sess21h00',
                               'total_correct_Sess21h00','total_response_time_Sess21h00','total_responses_Sess21h00','width_Sess21h00']

df_Operation_Span_Experimental = pd.concat([df_Operation_Span_1,df_Operation_Span_2,df_Operation_Span_3,df_Operation_Span_4],axis=1)


df_Operation_Span_Experimental = df_Operation_Span_Experimental[['subject_nr','CB_ref','practice','TrialNumber','Sub_bloco','SubTaskName',
                                                   'acc_Sess09h00','acc_Sess13h00','acc_Sess17h00','acc_Sess21h00','avg_rt_Sess09h00','avg_rt_Sess13h00','avg_rt_Sess17h00','avg_rt_Sess21h00',
                                                   'BlockChoice_Sess09h00','BlockChoice_Sess13h00','BlockChoice_Sess17h00','BlockChoice_Sess21h00','correct_Sess09h00',
                                                   'correct_Sess13h00','correct_Sess17h00','correct_Sess21h00','correct_response_Sess09h00','correct_response_Sess13h00',
                                                   'correct_response_Sess17h00','correct_response_Sess21h00','height_Sess09h00','height_Sess13h00','height_Sess17h00','height_Sess21h00',
                                                   'letter_Sess09h00','letter_Sess13h00','letter_Sess17h00','letter_Sess21h00','List_Prev_Letter_Sess09h00','List_Prev_Letter_Sess13h00',
                                                   'List_Prev_Letter_Sess17h00','List_Prev_Letter_Sess21h00','List_responses_memory_Sess09h00','List_responses_memory_Sess13h00',
                                                   'List_responses_memory_Sess17h00','List_responses_memory_Sess21h00','live_row_Sess09h00','live_row_Sess13h00','live_row_Sess17h00',
                                                   'live_row_Sess21h00','logfile_Sess09h00','logfile_Sess13h00','logfile_Sess17h00','logfile_Sess21h00',
                                                   'response_average_time_memory_Sess09h00','response_average_time_memory_Sess13h00','response_average_time_memory_Sess17h00',
                                                   'response_average_time_memory_Sess21h00','response_memory_Sess09h00','response_memory_Sess13h00','response_memory_Sess17h00',
                                                   'response_memory_Sess21h00','response_processing_Sess09h00','response_processing_Sess13h00','response_processing_Sess17h00',
                                                   'response_processing_Sess21h00','response_time_memory_Sess09h00','response_time_memory_Sess13h00','response_time_memory_Sess17h00',
                                                   'response_time_memory_Sess21h00','response_time_processing_Sess09h00','response_time_processing_Sess13h00',
                                                   'response_time_processing_Sess17h00','response_time_processing_Sess21h00','response_total_time_memory_Sess09h00',
                                                   'response_total_time_memory_Sess13h00','response_total_time_memory_Sess17h00','response_total_time_memory_Sess21h00',
                                                   'OP_part_process_time_Sess09h00','OP_part_process_time_Sess13h00','OP_part_process_time_Sess17h00',
                                                   'OP_part_process_time_Sess21h00','score_practice_Sess09h00','score_practice_Sess13h00','score_practice_Sess17h00',
                                                   'score_practice_Sess21h00','score_operation_span_Sess09h00','score_operation_span_Sess13h00','score_operation_span_Sess17h00',
                                                   'score_operation_span_Sess21h00','score_subblock_2_Sess09h00','score_subblock_2_Sess13h00','score_subblock_2_Sess17h00',
                                                   'score_subblock_2_Sess21h00','score_subblock_3_Sess09h00','score_subblock_3_Sess13h00','score_subblock_3_Sess17h00',
                                                   'score_subblock_3_Sess21h00','score_subblock_4_Sess09h00','score_subblock_4_Sess13h00','score_subblock_4_Sess17h00',
                                                   'score_subblock_4_Sess21h00','score_subblock_5_Sess09h00','score_subblock_5_Sess13h00','score_subblock_5_Sess17h00',
                                                   'score_subblock_5_Sess21h00','score_subblock_6_Sess09h00','score_subblock_6_Sess13h00','score_subblock_6_Sess17h00',
                                                   'score_subblock_6_Sess21h00','Tipo_Sess09h00','Tipo_Sess13h00','Tipo_Sess17h00','Tipo_Sess21h00','total_correct_Sess09h00',
                                                   'total_correct_Sess13h00','total_correct_Sess17h00','total_correct_Sess21h00','total_response_time_Sess09h00',
                                                   'total_response_time_Sess13h00','total_response_time_Sess17h00','total_response_time_Sess21h00','total_responses_Sess09h00',
                                                   'total_responses_Sess13h00','total_responses_Sess17h00','total_responses_Sess21h00','width_Sess09h00','width_Sess13h00',
                                                   'width_Sess17h00','width_Sess21h00']]


# Reset index and drop the old index column
df_Reading_Span_Experimental = df_Reading_Span_Experimental.reset_index(drop=True)

In [ ]:
# Show all dataframe columns when printing
pd.set_option("display.max_columns", None)

# Display the experimental Reading Span dataframe
df_Reading_Span_Experimental

## 5.3. Generates database with raw scores in the five WM tasks (experimental session)

In [ ]:
# Initialize an empty DataFrame to aggregate all raw scores
df_raw_scores = pd.DataFrame()

# Get unique subject IDs from the total participants table
subj_nr = df_total_part["subject_nr"].unique()

# Add subject IDs as the first column of the raw scores DataFrame
df_raw_scores.insert(0,'subject_nr',subj_nr)

# Define project root and load experimental body temperature data
PROJ_ROOT = Path("..").resolve()
excel_part_data_path = PROJ_ROOT / "Temperature" / "Body_Temperature_Collection.xlsx"
Temperature = pd.read_excel(excel_part_data_path,sheet_name='TempExperimental')

# Remove redundant subject identifier from the temperature sheet
Temperature.drop(columns="Subject ID", inplace=True)

# Add 09h00 temperature values to the raw scores table
df_raw_scores["Temperature (°C) Sess 09h00"] = list(Temperature["Temperature (°C) Sess 09h00"])

# Add 13h00 temperature values to the raw scores table
df_raw_scores["Temperature (°C) Sess 13h00"] = list(Temperature["Temperature (°C) Sess 13h00"])

# Add 17h00 temperature values to the raw scores table
df_raw_scores["Temperature (°C) Sess 17h00"] = list(Temperature["Temperature (°C) Sess 17h00"])

# Add 21h00 temperature values to the raw scores table
df_raw_scores["Temperature (°C) Sess 21h00"] = list(Temperature["Temperature (°C) Sess 21h00"])

# Compute max Reading Span (09h00) per subject (scaled ×20) and store
RawRS1 = list(df_Reading_Span_1.groupby(['subject_nr'], sort=True)['score_reading_span_Sess09h00'].max() * 20)

# Insert Reading Span 09h00 scores into the aggregate table
df_raw_scores["Reading Span Session 09h00"] = RawRS1

# Define metadata columns to mirror into subsequent Reading Span DataFrames
cols_to_add = [
    'subject_nr', 'CB_ref', 'practice', 
    'TrialNumber', 'Sub_bloco', 'SubTaskName'
]

# Copy metadata columns from RS1 to RS2 before computing RS2 scores
for col in cols_to_add:
    df_Reading_Span_2.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Reading_Span_1[col].values
    )

# Compute max Reading Span (13h00) per subject (scaled ×20) and store
RawRS2 = list(df_Reading_Span_2.groupby(['subject_nr'], sort=True)['score_reading_span_Sess13h00'].max() * 20)

# Insert Reading Span 13h00 scores into the aggregate table
df_raw_scores["Reading Span Session 13h00"] = RawRS2

# Reuse the same metadata set for RS3
cols_to_add = [
    'subject_nr', 'CB_ref', 'practice', 
    'TrialNumber', 'Sub_bloco', 'SubTaskName'
]

# Copy metadata columns from RS1 to RS3 before computing RS3 scores
for col in cols_to_add:
    df_Reading_Span_3.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Reading_Span_1[col].values
    )

# Compute max Reading Span (17h00) per subject (scaled ×20) and store
RawRS3 = list(df_Reading_Span_3.groupby(['subject_nr'], sort=True)['score_reading_span_Sess17h00'].max() * 20)

# Insert Reading Span 17h00 scores into the aggregate table
df_raw_scores["Reading Span Session 17h00"] = RawRS3

# Reuse the same metadata set for RS4
cols_to_add = [
    'subject_nr', 'CB_ref', 'practice', 
    'TrialNumber', 'Sub_bloco', 'SubTaskName'
]

# Copy metadata columns from RS1 to RS4 before computing RS4 scores
for col in cols_to_add:
    df_Reading_Span_4.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Reading_Span_1[col].values
    )

# Compute max Reading Span (21h00) per subject (scaled ×20) and store
RawRS4 = list(df_Reading_Span_4.groupby(['subject_nr'], sort=True)['score_reading_span_Sess21h00'].max() * 20)

# Insert Reading Span 21h00 scores into the aggregate table
df_raw_scores["Reading Span Session 21h00"] = RawRS4

# Compute max Updating Task (09h00) per subject and store
RawUT1 = list(df_WMU_Task_1.groupby(['subject_nr'], sort=True)['WMUExperimentalScore_Sess09h00'].max())

# Insert Updating Task 09h00 scores into the aggregate table
df_raw_scores["Updating Task Session 09h00"] = RawUT1

# Define minimal metadata to mirror for WMU tasks
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from WMU1 to WMU2 before computing WMU2 scores
for col in cols_to_add:
    df_WMU_Task_2.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_WMU_Task_1[col].values
    )

# Compute max Updating Task (13h00) per subject and store
RawUT2 = list(df_WMU_Task_2.groupby(['subject_nr'], sort=True)['WMUExperimentalScore_Sess13h00'].max())

# Insert Updating Task 13h00 scores into the aggregate table
df_raw_scores["Updating Task Session 13h00"] = RawUT2

# Reuse subject ID mirroring for WMU3
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from WMU1 to WMU3 before computing WMU3 scores
for col in cols_to_add:
    df_WMU_Task_3.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_WMU_Task_1[col].values
    )

# Compute max Updating Task (17h00) per subject and store
RawUT3 = list(df_WMU_Task_3.groupby(['subject_nr'], sort=True)['WMUExperimentalScore_Sess17h00'].max())

# Insert Updating Task 17h00 scores into the aggregate table
df_raw_scores["Updating Task Session 17h00"] = RawUT3

# Reuse subject ID mirroring for WMU4
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from WMU1 to WMU4 before computing WMU4 scores
for col in cols_to_add:
    df_WMU_Task_4.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_WMU_Task_1[col].values
    )

# Compute max Updating Task (21h00) per subject and store
RawUT4 = list(df_WMU_Task_4.groupby(['subject_nr'], sort=True)['WMUExperimentalScore_Sess21h00'].max())

# Insert Updating Task 21h00 scores into the aggregate table
df_raw_scores["Updating Task Session 21h00"] = RawUT4

# Compute max Symmetry Span (09h00) per subject (scaled ×20) and store
RawSS1 = list(df_Symmetry_Span_1.groupby(['subject_nr'], sort=True)['score_symmetry_span_Sess09h00'].max() * 20)

# Insert Symmetry Span 09h00 scores into the aggregate table
df_raw_scores["Symmetry Span Session 09h00"] = RawSS1

# Define minimal metadata to mirror for Symmetry Span tasks
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from SS1 to SS2 before computing SS2 scores
for col in cols_to_add:
    df_Symmetry_Span_2.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Symmetry_Span_1[col].values
    )

# Compute max Symmetry Span (13h00) per subject (scaled ×20) and store
RawSS2 = list(df_Symmetry_Span_2.groupby(['subject_nr'], sort=True)['score_symmetry_span_Sess13h00'].max() * 20)

# Insert Symmetry Span 13h00 scores into the aggregate table
df_raw_scores["Symmetry Span Session 13h00"] = RawSS2

# Reuse subject ID mirroring for SS3
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from SS1 to SS3 before computing SS3 scores
for col in cols_to_add:
    df_Symmetry_Span_3.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Symmetry_Span_1[col].values
    )

# Compute max Symmetry Span (17h00) per subject (scaled ×20) and store
RawSS3 = list(df_Symmetry_Span_3.groupby(['subject_nr'], sort=True)['score_symmetry_span_Sess17h00'].max() * 20)

# Insert Symmetry Span 17h00 scores into the aggregate table
df_raw_scores["Symmetry Span Session 17h00"] = RawSS3

# Reuse subject ID mirroring for SS4
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from SS1 to SS4 before computing SS4 scores
for col in cols_to_add:
    df_Symmetry_Span_4.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Symmetry_Span_1[col].values
    )

# Compute max Symmetry Span (21h00) per subject (scaled ×20) and store
RawSS4 = list(df_Symmetry_Span_4.groupby(['subject_nr'], sort=True)['score_symmetry_span_Sess21h00'].max() * 20)

# Insert Symmetry Span 21h00 scores into the aggregate table
df_raw_scores["Symmetry Span Session 21h00"] = RawSS4

# Compute max Binding Task (09h00) per subject and store
RawBT1 = list(df_Binding_Task_1.groupby(['subject_nr'], sort=True)['BindingRawScore_Sess09h00'].max())

# Insert Binding Task 09h00 scores into the aggregate table
df_raw_scores["Binding Task Session 09h00"] = RawBT1

# Define minimal metadata to mirror for Binding tasks
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from BT1 to BT2 before computing BT2 scores
for col in cols_to_add:
    df_Binding_Task_2.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Binding_Task_1[col].values
    )

# Compute max Binding Task (13h00) per subject and store
RawBT2 = list(df_Binding_Task_2.groupby(['subject_nr'], sort=True)['BindingRawScore_Sess13h00'].max())

# Insert Binding Task 13h00 scores into the aggregate table
df_raw_scores["Binding Task Session 13h00"] = RawBT2

# Reuse subject ID mirroring for BT3
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from BT1 to BT3 before computing BT3 scores
for col in cols_to_add:
    df_Binding_Task_3.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Binding_Task_1[col].values
    )

# Compute max Binding Task (17h00) per subject and store
RawBT3 = list(df_Binding_Task_3.groupby(['subject_nr'], sort=True)['BindingRawScore_Sess17h00'].max())

# Insert Binding Task 17h00 scores into the aggregate table
df_raw_scores["Binding Task Session 17h00"] = RawBT3

# Reuse subject ID mirroring for BT4
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from BT1 to BT4 before computing BT4 scores
for col in cols_to_add:
    df_Binding_Task_4.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Binding_Task_1[col].values
    )

# Compute max Binding Task (21h00) per subject and store
RawBT4 = list(df_Binding_Task_4.groupby(['subject_nr'], sort=True)['BindingRawScore_Sess21h00'].max())

# Insert Binding Task 21h00 scores into the aggregate table
df_raw_scores["Binding Task Session 21h00"] = RawBT4

# Compute max Operation Span (09h00) per subject (scaled ×20) and store
RawOS1 = list(df_Operation_Span_1.groupby(['subject_nr'], sort=True)['score_operation_span_Sess09h00'].max() * 20)

# Insert Operation Span 09h00 scores into the aggregate table
df_raw_scores["Operation Span Session 09h00"] = RawOS1

# Define minimal metadata to mirror for Operation Span tasks
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from OS1 to OS2 before computing OS2 scores
for col in cols_to_add:
    df_Operation_Span_2.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Operation_Span_1[col].values
    )

# Compute max Operation Span (13h00) per subject (scaled ×20) and store
RawOS2 = list(df_Operation_Span_2.groupby(['subject_nr'], sort=True)['score_operation_span_Sess13h00'].max() * 20)

# Insert Operation Span 13h00 scores into the aggregate table
df_raw_scores["Operation Span Session 13h00"] = RawOS2

# Reuse subject ID mirroring for OS3
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from OS1 to OS3 before computing OS3 scores
for col in cols_to_add:
    df_Operation_Span_3.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Operation_Span_1[col].values
    )

# Compute max Operation Span (17h00) per subject (scaled ×20) and store
RawOS3 = list(df_Operation_Span_3.groupby(['subject_nr'], sort=True)['score_operation_span_Sess17h00'].max() * 20)

# Insert Operation Span 17h00 scores into the aggregate table
df_raw_scores["Operation Span Session 17h00"] = RawOS3

# Reuse subject ID mirroring for OS4
cols_to_add = [
    'subject_nr'
]

# Copy subject IDs from OS1 to OS4 before computing OS4 scores
for col in cols_to_add:
    df_Operation_Span_4.insert(
        loc=0,                          # insert at the beginning
        column=col, 
        value=df_Operation_Span_1[col].values
    )

# Compute max Operation Span (21h00) per subject (scaled ×20) and store
RawOS4 = list(df_Operation_Span_4.groupby(['subject_nr'], sort=True)['score_operation_span_Sess21h00'].max() * 20)

# Insert Operation Span 21h00 scores into the aggregate table
df_raw_scores["Operation Span Session 21h00"] = RawOS4

# Determine each subject’s counterbalancing order based on ID ranges/specific IDs
list_counterbalancing = []
for i in range(0,len(df_raw_scores)):
    if 1 <= df_raw_scores.loc[i,'subject_nr'] <= 5 or df_raw_scores.loc[i,'subject_nr'] == 21 or df_raw_scores.loc[i,'subject_nr'] == 25:
        aaa = '09h00,13h00,17h00,21h00'
        list_counterbalancing.append(aaa)
    elif 6 <= df_raw_scores.loc[i,'subject_nr'] <= 10 or df_raw_scores.loc[i,'subject_nr'] == 22 or df_raw_scores.loc[i,'subject_nr'] == 26:
        aaa = '13h00,17h00,21h00,09h00'
        list_counterbalancing.append(aaa)
    elif 11 <= df_raw_scores.loc[i,'subject_nr'] <= 15 or df_raw_scores.loc[i,'subject_nr'] == 23  or df_raw_scores.loc[i,'subject_nr'] == 27:
        aaa = '17h00,21h00,09h00,13h00'
        list_counterbalancing.append(aaa)
    elif 16 <= df_raw_scores.loc[i,'subject_nr'] <= 20 or df_raw_scores.loc[i,'subject_nr'] == 24  or df_raw_scores.loc[i,'subject_nr'] == 28:
        aaa = '21h00,09h00,13h00,17h00'
        list_counterbalancing.append(aaa)

# Insert the derived counterbalancing sequence as the second column
df_raw_scores.insert(1, 'Counterbalancing', list_counterbalancing)

# Cast all non-ID, non-text, non-temperature columns to integer type
df_raw_scores = df_raw_scores.astype({
    col: 'int' 
    for col in df_raw_scores.columns 
    if col not in ['subject_nr', 'Counterbalancing','Temperature (°C) Sess 09h00','Temperature (°C) Sess 13h00','Temperature (°C) Sess 17h00',
                   'Temperature (°C) Sess 21h00']
})

# Sort the final experimental table by subject ID
df_raw_scores_exper = df_raw_scores.sort_values(by="subject_nr")

# Display the sorted table
df_raw_scores_exper

# Ensure all columns are visible when displaying DataFrames
pd.set_option("display.max_columns", None)

# Display the sorted table again with the updated display option
df_raw_scores_exper

## 5.4. Generates database with normalized scores in the five WM tasks (experimental session)

In [ ]:
# Create a deep copy of the raw scores table to store normalized values
df_normalized_scores_exper = df_raw_scores_exper.copy(deep=True)

# Normalize Reading Span 09h00 scores by dividing by maximum possible value
df_normalized_scores_exper["Reading Span Session 09h00"] = df_normalized_scores_exper["Reading Span Session 09h00"]/20
# Normalize Reading Span 13h00 scores
df_normalized_scores_exper["Reading Span Session 13h00"] = df_normalized_scores_exper["Reading Span Session 13h00"]/20
# Normalize Reading Span 17h00 scores
df_normalized_scores_exper["Reading Span Session 17h00"] = df_normalized_scores_exper["Reading Span Session 17h00"]/20
# Normalize Reading Span 21h00 scores
df_normalized_scores_exper["Reading Span Session 21h00"] = df_normalized_scores_exper["Reading Span Session 21h00"]/20

# Normalize Updating Task 09h00 scores by dividing by maximum possible value
df_normalized_scores_exper["Updating Task Session 09h00"] = df_normalized_scores_exper["Updating Task Session 09h00"] / 36
# Normalize Updating Task 13h00 scores
df_normalized_scores_exper["Updating Task Session 13h00"] = df_normalized_scores_exper["Updating Task Session 13h00"] / 36
# Normalize Updating Task 17h00 scores
df_normalized_scores_exper["Updating Task Session 17h00"] = df_normalized_scores_exper["Updating Task Session 17h00"] / 36
# Normalize Updating Task 21h00 scores
df_normalized_scores_exper["Updating Task Session 21h00"] = df_normalized_scores_exper["Updating Task Session 21h00"] / 36

# Normalize Symmetry Span 09h00 scores by dividing by maximum possible value
df_normalized_scores_exper["Symmetry Span Session 09h00"] = df_normalized_scores_exper["Symmetry Span Session 09h00"] / 20
# Normalize Symmetry Span 13h00 scores
df_normalized_scores_exper["Symmetry Span Session 13h00"] = df_normalized_scores_exper["Symmetry Span Session 13h00"] / 20
# Normalize Symmetry Span 17h00 scores
df_normalized_scores_exper["Symmetry Span Session 17h00"] = df_normalized_scores_exper["Symmetry Span Session 17h00"] / 20
# Normalize Symmetry Span 21h00 scores
df_normalized_scores_exper["Symmetry Span Session 21h00"] = df_normalized_scores_exper["Symmetry Span Session 21h00"] / 20

# Normalize Binding Task 09h00 scores by dividing by maximum possible value
df_normalized_scores_exper["Binding Task Session 09h00"] = df_normalized_scores_exper["Binding Task Session 09h00"] / 12
# Normalize Binding Task 13h00 scores
df_normalized_scores_exper["Binding Task Session 13h00"] = df_normalized_scores_exper["Binding Task Session 13h00"] / 12
# Normalize Binding Task 17h00 scores
df_normalized_scores_exper["Binding Task Session 17h00"] = df_normalized_scores_exper["Binding Task Session 17h00"] / 12
# Normalize Binding Task 21h00 scores
df_normalized_scores_exper["Binding Task Session 21h00"] = df_normalized_scores_exper["Binding Task Session 21h00"] / 12

# Normalize Operation Span 09h00 scores by dividing by maximum possible value
df_normalized_scores_exper["Operation Span Session 09h00"] = df_normalized_scores_exper["Operation Span Session 09h00"] / 20
# Normalize Operation Span 13h00 scores
df_normalized_scores_exper["Operation Span Session 13h00"] = df_normalized_scores_exper["Operation Span Session 13h00"] / 20
# Normalize Operation Span 17h00 scores
df_normalized_scores_exper["Operation Span Session 17h00"] = df_normalized_scores_exper["Operation Span Session 17h00"] / 20
# Normalize Operation Span 21h00 scores
df_normalized_scores_exper["Operation Span Session 21h00"] = df_normalized_scores_exper["Operation Span Session 21h00"] / 20

# List of normalized score columns that need to be rounded
cols_to_round = [
    "Reading Span Session 09h00",
    "Reading Span Session 13h00",
    "Reading Span Session 17h00",
    "Reading Span Session 21h00",
    "Updating Task Session 09h00",
    "Updating Task Session 13h00",
    "Updating Task Session 17h00",
    "Updating Task Session 21h00",
    "Symmetry Span Session 09h00",
    "Symmetry Span Session 13h00",
    "Symmetry Span Session 17h00",
    "Symmetry Span Session 21h00",
    "Binding Task Session 09h00",
    "Binding Task Session 13h00",
    "Binding Task Session 17h00",
    "Binding Task Session 21h00",
    "Operation Span Session 09h00",
    "Operation Span Session 13h00",
    "Operation Span Session 17h00",
    "Operation Span Session 21h00",
]

# Convert selected columns to numeric and round them to two decimals
df_normalized_scores_exper[cols_to_round] = (
    df_normalized_scores_exper[cols_to_round]
    .apply(pd.to_numeric, errors="coerce")  # safely convert to numeric
    .round(2)  # round to 2 decimal places
)

# Ensure full column visibility when displaying DataFrames
pd.set_option("display.max_columns", None)

# Display the final normalized table
df_normalized_scores_exper


# 6. Viewing and/or downloading the final processed databases
Downloading of viewing the final processed DBs

## 6.1. Downloading the final processed databases

In [ ]:
# Import FileLink to allow generating a downloadable link for the Excel file
from IPython.display import FileLink

# Create a multi-sheet Excel file and write all datasets into separate sheets
with pd.ExcelWriter('BD_all_data_combined.xlsx') as writer:
    df_excel_Screening.to_excel(writer, sheet_name='Screening_Part', index=False)
    SD_df_excel_data_part.to_excel(writer, sheet_name='Sleep Diary', index=False)
    AD_df_excel_data_part.to_excel(writer, sheet_name='Activity Diary', index=False)
    df_Actigraphy.to_excel(writer,sheet_name="Actigraphy", index=False)
    df_raw_scores_exper.to_excel(writer, sheet_name='Raw Scores', index=False)
    df_normalized_scores_exper.to_excel(writer, sheet_name='Normalized Scores', index=False)
    df_Reading_Span_Experimental.to_excel(writer, sheet_name='Reading Span', index=False)
    df_WMU_Task_Experimental.to_excel(writer, sheet_name='Updating Task', index=False)
    df_Symmetry_Span_Experimental.to_excel(writer, sheet_name='Symmetry Span', index=False)
    df_Binding_Task_Experimental.to_excel(writer, sheet_name='Binding Task', index=False)
    df_Operation_Span_Experimental.to_excel(writer, sheet_name='Operation Span', index=False)
    df_raw_scores_pract.to_excel(writer, sheet_name='Practice Raw Scores', index=False)
    df_normalized_scores_pract.to_excel(writer, sheet_name='Practice Normalized Scores', index=False)
    df_Reading_Span_Practice.to_excel(writer, sheet_name='Practice RS', index=False)
    df_WMU_Task_Practice.to_excel(writer, sheet_name='Practice UT', index=False)
    df_Symmetry_Span_Practice.to_excel(writer, sheet_name='Practice SS', index=False)
    df_Binding_Task_Practice.to_excel(writer, sheet_name='Practice BT', index=False)
    df_Operation_Span_Practice.to_excel(writer, sheet_name='Practice OS', index=False)

## 6.2. Viewing the final processed databases

### 6.2.1 Screening database

In [ ]:
pd.set_option("display.max_columns",None)
df_excel_Screening

### 6.2.2. Sleep diary database

In [ ]:
pd.set_option("display.max_columns",None)
SD_df_excel_data_part

### 6.2.3. Activity diary database

In [ ]:
pd.set_option("display.max_columns",None)
AD_df_excel_data_part

### 6.2.4. Actigraphy database

In [ ]:
pd.set_option("display.max_columns",None)
df_Actigraphy

### 6.2.5. WM tasks raw scores experimental sessions database

In [ ]:
pd.set_option("display.max_columns",None)
df_raw_scores_exper

### 6.2.6. WM tasks normalized scores experimental sessions database

In [ ]:
pd.set_option("display.max_columns",None)
df_normalized_scores_exper

### 6.2.7. WM tasks raw scores practice sessions database

In [ ]:
pd.set_option("display.max_columns",None)
df_raw_scores_pract

### 6.2.8. WM tasks normalized scores practice sessions database

In [ ]:
pd.set_option("display.max_columns",None)
df_normalized_scores_pract